In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

! pip install readability

In [2]:
!pip install textblob
from textblob import TextBlob

In [3]:
stopwords_audit = '''ERNST
YOUNG
DELOITTE
TOUCHE
KPMG
PRICEWATERHOUSECOOPERS
PRICEWATERHOUSE
COOPERS'''
stopwords_audit=stopwords_audit.replace('\n',',')
stopwords_audit = stopwords_audit.split(',')
stopwords_audit


['ERNST',
 'YOUNG',
 'DELOITTE',
 'TOUCHE',
 'KPMG',
 'PRICEWATERHOUSECOOPERS',
 'PRICEWATERHOUSE',
 'COOPERS']

In [4]:
stopwords_cuurencies = '''AFGHANI  | Afghanistan 
ARIARY | Madagascar 
BAHT | Thailand 
BALBOA | Panama 
BIRR | Ethiopia 
BOLIVAR | Venezuela 
BOLIVIANO  | Bolivia 
CEDI | Ghana 
COLON  | Costa Rica 
CÓRDOBA  | Nicaragua 
DALASI | Gambia 
DENAR | Macedonia (Former Yug. Rep.) 
DINAR | Algeria 
DIRHAM  | Morocco 
DOBRA | São Tom and Príncipe 
DONG | Vietnam 
DRAM | Armenia 
ESCUDO  | Cape Verde 
EURO  | Belgium 
FLORIN | Aruba 
FORINT | Hungary 
GOURDE | Haiti 
GUARANI | Paraguay 
GULDEN | Netherlands Antilles 
HRYVNIA  | Ukraine 
KINA | Papua New Guinea 
KIP | Laos 
KONVERTIBILNA MARKA  | Bosnia-Herzegovina 
KORUNA  | Czech Republic 
KRONA | Sweden 
KRONE | Denmark 
KROON | Estonia 
KUNA | Croatia 
KWACHA | Zambia 
KWANZA | Angola 
KYAT | Myanmar 
LARI | Georgia 
LATS | Latvia 
LEK | Albania 
LEMPIRA | Honduras 
LEONE | Sierra Leone 
LEU | Romania 
LEV | Bulgaria 
LILANGENI  | Swaziland 
LIRA | Lebanon 
LITAS | Lithuania 
LOTI | Lesotho 
MANAT | Azerbaijan 
METICAL | Mozambique 
NAIRA | Nigeria 
NAKFA | Eritrea 
NEW LIRA | Turkey 
NEW SHEQEL  | Israel 
NGULTRUM  | Bhutan 
NUEVO SOL | Peru 
OUGUIYA  | Mauritania 
PATACA | Macau 
PESO  | Mexico 
POUND  | Egypt 
PULA  | Botswana 
QUETZAL | Guatemala 
RAND | South Africa 
REAL  | Brazil 
RENMINBI  | China 
RIAL | Iran 
RIEL | Cambodia 
RINGGIT | Malaysia 
RIYAL | Saudi Arabia 
RUBLE | Russia 
RUFIYAA | Maldives 
RUPEE  | India 
RUPEE  | Pakistan 
RUPIAH  | Indonesia 
SHILLING  | Uganda 
SOM | Uzbekistan 
SOMONI  | Tajikistan 
SPECIAL DRAWING RIGHTS  | International Monetary Fund 
TAKA | Bangladesh 
TALA | Western Samoa 
TENGE | Kazakhstan 
TUGRIK  | Mongolia 
VATU | Vanuatu 
WON  | Korea, South 
YEN | Japan 
ZLOTY | Poland '''
stopwords_cuurencies = stopwords_cuurencies.replace('\n',',')
stopwords_cuurencies = stopwords_cuurencies.replace('|',',')
stopwords_cuurencies = stopwords_cuurencies.replace(' ','')
stopwords_cuurencies = stopwords_cuurencies.split(',')
stopwords_cuurencies

['AFGHANI',
 'Afghanistan',
 'ARIARY',
 'Madagascar',
 'BAHT',
 'Thailand',
 'BALBOA',
 'Panama',
 'BIRR',
 'Ethiopia',
 'BOLIVAR',
 'Venezuela',
 'BOLIVIANO',
 'Bolivia',
 'CEDI',
 'Ghana',
 'COLON',
 'CostaRica',
 'CÓRDOBA',
 'Nicaragua',
 'DALASI',
 'Gambia',
 'DENAR',
 'Macedonia(FormerYug.Rep.)',
 'DINAR',
 'Algeria',
 'DIRHAM',
 'Morocco',
 'DOBRA',
 'SãoTomandPríncipe',
 'DONG',
 'Vietnam',
 'DRAM',
 'Armenia',
 'ESCUDO',
 'CapeVerde',
 'EURO',
 'Belgium',
 'FLORIN',
 'Aruba',
 'FORINT',
 'Hungary',
 'GOURDE',
 'Haiti',
 'GUARANI',
 'Paraguay',
 'GULDEN',
 'NetherlandsAntilles',
 'HRYVNIA',
 'Ukraine',
 'KINA',
 'PapuaNewGuinea',
 'KIP',
 'Laos',
 'KONVERTIBILNAMARKA',
 'Bosnia-Herzegovina',
 'KORUNA',
 'CzechRepublic',
 'KRONA',
 'Sweden',
 'KRONE',
 'Denmark',
 'KROON',
 'Estonia',
 'KUNA',
 'Croatia',
 'KWACHA',
 'Zambia',
 'KWANZA',
 'Angola',
 'KYAT',
 'Myanmar',
 'LARI',
 'Georgia',
 'LATS',
 'Latvia',
 'LEK',
 'Albania',
 'LEMPIRA',
 'Honduras',
 'LEONE',
 'SierraLeone',


In [5]:
stopwords_date_num = '''HUNDRED  | Denominations
THOUSAND
MILLION
BILLION
TRILLION
DATE  | Time related
ANNUAL
ANNUALLY
ANNUM
YEAR
YEARLY
QUARTER
QUARTERLY
QTR
MONTH
MONTHLY
WEEK
WEEKLY
DAY
DAILY
JANUARY  | Calendar
FEBRUARY
MARCH
APRIL
MAY
JUNE
JULY
AUGUST
SEPTEMBER
OCTOBER
NOVEMBER
DECEMBER
JAN
FEB
MAR
APR
MAY
JUN
JUL
AUG
SEP
SEPT
OCT
NOV
DEC
MONDAY
TUESDAY
WEDNESDAY
THURSDAY
FRIDAY
SATURDAY
SUNDAY
ONE  | Numbers
TWO
THREE
FOUR
FIVE
SIX
SEVEN
EIGHT
NINE
TEN
ELEVEN
TWELVE
THIRTEEN
FOURTEEN
FIFTEEN
SIXTEEN
SEVENTEEN
EIGHTEEN
NINETEEN
TWENTY
THIRTY
FORTY
FIFTY
SIXTY
SEVENTY
EIGHTY
NINETY
FIRST
SECOND
THIRD
FOURTH
FIFTH
SIXTH
SEVENTH
EIGHTH
NINTH
TENTH
I  | Roman numerals
II
III
IV
V
VI
VII
VIII
IX
X
XI
XII
XIII
XIV
XV
XVI
XVII
XVIII
XIX
XX'''
stopwords_date_num = stopwords_date_num.replace('\n',',')
stopwords_date_num = stopwords_date_num.replace('|',',')
stopwords_date_num = stopwords_date_num.replace(' ','')
stopwords_date_num = stopwords_date_num.split(',')
stopwords_date_num

['HUNDRED',
 'Denominations',
 'THOUSAND',
 'MILLION',
 'BILLION',
 'TRILLION',
 'DATE',
 'Timerelated',
 'ANNUAL',
 'ANNUALLY',
 'ANNUM',
 'YEAR',
 'YEARLY',
 'QUARTER',
 'QUARTERLY',
 'QTR',
 'MONTH',
 'MONTHLY',
 'WEEK',
 'WEEKLY',
 'DAY',
 'DAILY',
 'JANUARY',
 'Calendar',
 'FEBRUARY',
 'MARCH',
 'APRIL',
 'MAY',
 'JUNE',
 'JULY',
 'AUGUST',
 'SEPTEMBER',
 'OCTOBER',
 'NOVEMBER',
 'DECEMBER',
 'JAN',
 'FEB',
 'MAR',
 'APR',
 'MAY',
 'JUN',
 'JUL',
 'AUG',
 'SEP',
 'SEPT',
 'OCT',
 'NOV',
 'DEC',
 'MONDAY',
 'TUESDAY',
 'WEDNESDAY',
 'THURSDAY',
 'FRIDAY',
 'SATURDAY',
 'SUNDAY',
 'ONE',
 'Numbers',
 'TWO',
 'THREE',
 'FOUR',
 'FIVE',
 'SIX',
 'SEVEN',
 'EIGHT',
 'NINE',
 'TEN',
 'ELEVEN',
 'TWELVE',
 'THIRTEEN',
 'FOURTEEN',
 'FIFTEEN',
 'SIXTEEN',
 'SEVENTEEN',
 'EIGHTEEN',
 'NINETEEN',
 'TWENTY',
 'THIRTY',
 'FORTY',
 'FIFTY',
 'SIXTY',
 'SEVENTY',
 'EIGHTY',
 'NINETY',
 'FIRST',
 'SECOND',
 'THIRD',
 'FOURTH',
 'FIFTH',
 'SIXTH',
 'SEVENTH',
 'EIGHTH',
 'NINTH',
 'TENTH',
 'I',


In [6]:
stopwords_generic = '''ABOUT
ABOVE
AFTER
AGAIN
ALL
AM
AMONG
AN
AND
ANY
ARE
AS
AT
BE
BECAUSE
BEEN
BEFORE
BEING
BELOW
BETWEEN
BOTH
BUT
BY
CAN
DID
DO
DOES
DOING
DOWN
DURING
EACH
FEW
FOR
FROM
FURTHER
HAD
HAS
HAVE
HAVING
HE
HER
HERE
HERS
HERSELF
HIM
HIMSELF
HIS
HOW
IF
IN
INTO
IS
IT
ITS
ITSELF
JUST
ME
MORE
MOST
MY
MYSELF
NO
NOR
NOT
NOW
OF
OFF
ON
ONCE
ONLY
OR
OTHER
OUR
OURS
OURSELVES
OUT
OVER
OWN
SAME
SHE
SHOULD
SO
SOME
SUCH
THAN
THAT
THE
THEIR
THEIRS
THEM
THEMSELVES
THEN
THERE
THESE
THEY
THIS
THOSE
THROUGH
TO
TOO
UNDER
UNTIL
UP
VERY
WAS
WE
WERE
WHAT
WHEN
WHERE
WHICH
WHILE
WHO
WHOM
WHY
WITH
YOU
YOUR
YOURS
YOURSELF
YOURSELVES'''
stopwords_generic = stopwords_generic.replace('\n',',')
stopwords_generic = stopwords_generic.split(',')
stopwords_generic

['ABOUT',
 'ABOVE',
 'AFTER',
 'AGAIN',
 'ALL',
 'AM',
 'AMONG',
 'AN',
 'AND',
 'ANY',
 'ARE',
 'AS',
 'AT',
 'BE',
 'BECAUSE',
 'BEEN',
 'BEFORE',
 'BEING',
 'BELOW',
 'BETWEEN',
 'BOTH',
 'BUT',
 'BY',
 'CAN',
 'DID',
 'DO',
 'DOES',
 'DOING',
 'DOWN',
 'DURING',
 'EACH',
 'FEW',
 'FOR',
 'FROM',
 'FURTHER',
 'HAD',
 'HAS',
 'HAVE',
 'HAVING',
 'HE',
 'HER',
 'HERE',
 'HERS',
 'HERSELF',
 'HIM',
 'HIMSELF',
 'HIS',
 'HOW',
 'IF',
 'IN',
 'INTO',
 'IS',
 'IT',
 'ITS',
 'ITSELF',
 'JUST',
 'ME',
 'MORE',
 'MOST',
 'MY',
 'MYSELF',
 'NO',
 'NOR',
 'NOT',
 'NOW',
 'OF',
 'OFF',
 'ON',
 'ONCE',
 'ONLY',
 'OR',
 'OTHER',
 'OUR',
 'OURS',
 'OURSELVES',
 'OUT',
 'OVER',
 'OWN',
 'SAME',
 'SHE',
 'SHOULD',
 'SO',
 'SOME',
 'SUCH',
 'THAN',
 'THAT',
 'THE',
 'THEIR',
 'THEIRS',
 'THEM',
 'THEMSELVES',
 'THEN',
 'THERE',
 'THESE',
 'THEY',
 'THIS',
 'THOSE',
 'THROUGH',
 'TO',
 'TOO',
 'UNDER',
 'UNTIL',
 'UP',
 'VERY',
 'WAS',
 'WE',
 'WERE',
 'WHAT',
 'WHEN',
 'WHERE',
 'WHICH',
 'WHILE',
 '

In [7]:
stopwords_generic_long = '''a
a's
able
about
above
according
accordingly
across
actually
after
afterwards
again
against
ain't
all
allow
allows
almost
alone
along
already
also
although
always
am
among
amongst
an
and
another
any
anybody
anyhow
anyone
anything
anyway
anyways
anywhere
apart
appear
appreciate
appropriate
are
aren't
around
as
aside
ask
asking
associated
at
available
away
awfully
b
be
became
because
become
becomes
becoming
been
before
beforehand
behind
being
believe
below
beside
besides
best
better
between
beyond
both
brief
but
by
c
c'mon
c's
came
can
can't
cannot
cant
cause
causes
certain
certainly
changes
clearly
co
com
come
comes
concerning
consequently
consider
considering
contain
containing
contains
corresponding
could
couldn't
course
currently
d
definitely
described
despite
did
didn't
different
do
does
doesn't
doing
don't
done
down
downwards
during
e
each
edu
eg
eight
either
else
elsewhere
enough
entirely
especially
et
etc
even
ever
every
everybody
everyone
everything
everywhere
ex
exactly
example
except
f
far
few
fifth
first
five
followed
following
follows
for
former
formerly
forth
four
from
further
furthermore
g
get
gets
getting
given
gives
go
goes
going
gone
got
gotten
greetings
h
had
hadn't
happens
hardly
has
hasn't
have
haven't
having
he
he's
hello
help
hence
her
here
here's
hereafter
hereby
herein
hereupon
hers
herself
hi
him
himself
his
hither
hopefully
how
howbeit
however
i
i'd
i'll
i'm
i've
ie
if
ignored
immediate
in
inasmuch
inc
indeed
indicate
indicated
indicates
inner
insofar
instead
into
inward
is
isn't
it
it'd
it'll
it's
its
itself
j
just
k
keep
keeps
kept
know
knows
known
l
last
lately
later
latter
latterly
least
less
lest
let
let's
like
liked
likely
little
look
looking
looks
ltd
m
mainly
many
may
maybe
me
mean
meanwhile
merely
might
more
moreover
most
mostly
much
must
my
myself
n
name
namely
nd
near
nearly
necessary
need
needs
neither
never
nevertheless
new
next
nine
no
nobody
non
none
noone
nor
normally
not
nothing
novel
now
nowhere
o
obviously
of
off
often
oh
ok
okay
old
on
once
one
ones
only
onto
or
other
others
otherwise
ought
our
ours
ourselves
out
outside
over
overall
own
p
particular
particularly
per
perhaps
placed
please
plus
possible
presumably
probably
provides
q
que
quite
qv
r
rather
rd
re
really
reasonably
regarding
regardless
regards
relatively
respectively
right
s
said
same
saw
say
saying
says
second
secondly
see
seeing
seem
seemed
seeming
seems
seen
self
selves
sensible
sent
serious
seriously
seven
several
shall
she
should
shouldn't
since
six
so
some
somebody
somehow
someone
something
sometime
sometimes
somewhat
somewhere
soon
sorry
specified
specify
specifying
still
sub
such
sup
sure
t
t's
take
taken
tell
tends
th
than
thank
thanks
thanx
that
that's
thats
the
their
theirs
them
themselves
then
thence
there
there's
thereafter
thereby
therefore
therein
theres
thereupon
these
they
they'd
they'll
they're
they've
think
third
this
thorough
thoroughly
those
though
three
through
throughout
thru
thus
to
together
too
took
toward
towards
tried
tries
truly
try
trying
twice
two
u
un
under
unfortunately
unless
unlikely
until
unto
up
upon
us
use
used
useful
uses
using
usually
uucp
v
value
various
very
via
viz
vs
w
want
wants
was
wasn't
way
we
we'd
we'll
we're
we've
welcome
well
went
were
weren't
what
what's
whatever
when
whence
whenever
where
where's
whereafter
whereas
whereby
wherein
whereupon
wherever
whether
which
while
whither
who
who's
whoever
whole
whom
whose
why
will
willing
wish
with
within
without
won't
wonder
would
would
wouldn't
x
y
yes
yet
you
you'd
you'll
you're
you've
your
yours
yourself
yourselves
z
zero'''
stopwords_generic_long = stopwords_generic_long.replace('\n',',')
stopwords_generic_long = stopwords_generic_long.split(',')
stopwords_generic_long

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'c

In [8]:
stopwords_geographic = '''UNITED  | Geographic
STATE
NORTH
SOUTH
EAST
NORTHEAST
NORTHWEST
SOUTHEAST
SOUTHWEST
WEST
OCEAN
SEA
LAKE
RIVER
CREEK
GULF
MOUNTAIN
STREET
BOULEVARD
BLVD
PARKWAY
CITY
COUNTY
COUNTRY
PACIFIC
ATLANTIC
INDIAN
MEDITERRANEAN
COMMONWEALTH
AMERICA
AMERICAN
YORK  | Cities
CHICAGO
LAS
VEGAS
LOS
ANGELES
MILWAUKEE
SUNNYVALE
FREMONT
CINCINNATI
PHILADELPHIA
MIAMI
DALLAS
FORT
BOSTON
HOUSTON
WASHINGTON
ATLANTA
DETROIT
SAN
FRANSICO
PHOENIX
SEATTLE
DIEGO
MINNEAPOLIS
MEMPHIS
DENVER
ST
LOUIS
PITTSBURGH
MANHATTAN
HOLLYWOOD
COLUMBUS
INDIANAPOLIS
MUMBAI
KARACHI
ONTARIO
TORONTO
CAMBRIDGE
DELHI
SAO
PAULO
SHANGHAI
MOSCOW
SEOUL
ISTANBUL
TOKYO
JAKARTA
BEIJING
LONDON
LUXEMBOURG
SINGAPORE
REPUBLIC  | Countries
CHINA
INDIA
INDONESIA
BRAZIL
BRAZILIAN
PAKISTAN
BANGLADESH
RUSSIA
NIGERIA
NOVA
SCOTIA
JAPAN
MALAYSIA
MEXICO
MEXICAN
PHILIPPINES
VIETNAM
GERMANY
FRANCE
KOREA
SPAIN
ARGENTINA
BERMUDA
COLUMBIA
PUERTO
CANADA
CANADIAN
AFRICA
AFRICAN
CHILE
CHILEAN
EUROPE
EUROPEAN
USA
ITALY
ITALIAN
AUSTRALIA
NETHERLANDS
NORWAY
PORTUGAL
TAIWAN
ARGENTINA
SWITZERLAND
SWISS
SUISSE
DENMARK
CZECK
HUNGARY
FINLAND
SCOTLAND
COSTA
RICA
SWEDEN
BELGIUM
AUSTRIA
TURKEY
POLAND
INDONESIA
THAILAND
VENEZUELA
ASIA
ZEALAND
JAPANESE
LATIN
BRITISH
CHINESE
ALABAMA   | States
ALASKA
ARIZONA
ARKANSAS
CALIFORNIA
COLORADO
CONNECTICUT
DELAWARE
FLORIDA
GEORGIA
HAWAII
IDAHO
ILLINOIS
INDIANA
IOWA
KANSAS
KENTUCKY
LOUISIANA
MAINE
MARYLAND
MASSACHUSETTS
MICHIGAN
MINNESOTA
MISSISSIPPI
MISSOURI
MONTANA
NEBRASKA
NEVADA
HAMPSHIRE
JERSEY
MEXICO
NEW
YORK
OHIO
OKLAHOMA
OREGON
PENNSYLVANIA
RHODE
ISLAND
CAROLINA
DAKOTA
TENNESSEE
TEXAS
UTAH
VERMONT
VIRGINIA
WASHINGTON
WISCONSIN
WYOMING'''
stopwords_geographic = stopwords_geographic.replace('\n',',')
stopwords_geographic = stopwords_geographic.replace(' ','')
stopwords_geographic = stopwords_geographic.replace('|',',')
stopwords_geographic = stopwords_geographic.split(',')
stopwords_geographic

['UNITED',
 'Geographic',
 'STATE',
 'NORTH',
 'SOUTH',
 'EAST',
 'NORTHEAST',
 'NORTHWEST',
 'SOUTHEAST',
 'SOUTHWEST',
 'WEST',
 'OCEAN',
 'SEA',
 'LAKE',
 'RIVER',
 'CREEK',
 'GULF',
 'MOUNTAIN',
 'STREET',
 'BOULEVARD',
 'BLVD',
 'PARKWAY',
 'CITY',
 'COUNTY',
 'COUNTRY',
 'PACIFIC',
 'ATLANTIC',
 'INDIAN',
 'MEDITERRANEAN',
 'COMMONWEALTH',
 'AMERICA',
 'AMERICAN',
 'YORK',
 'Cities',
 'CHICAGO',
 'LAS',
 'VEGAS',
 'LOS',
 'ANGELES',
 'MILWAUKEE',
 'SUNNYVALE',
 'FREMONT',
 'CINCINNATI',
 'PHILADELPHIA',
 'MIAMI',
 'DALLAS',
 'FORT',
 'BOSTON',
 'HOUSTON',
 'WASHINGTON',
 'ATLANTA',
 'DETROIT',
 'SAN',
 'FRANSICO',
 'PHOENIX',
 'SEATTLE',
 'DIEGO',
 'MINNEAPOLIS',
 'MEMPHIS',
 'DENVER',
 'ST',
 'LOUIS',
 'PITTSBURGH',
 'MANHATTAN',
 'HOLLYWOOD',
 'COLUMBUS',
 'INDIANAPOLIS',
 'MUMBAI',
 'KARACHI',
 'ONTARIO',
 'TORONTO',
 'CAMBRIDGE',
 'DELHI',
 'SAO',
 'PAULO',
 'SHANGHAI',
 'MOSCOW',
 'SEOUL',
 'ISTANBUL',
 'TOKYO',
 'JAKARTA',
 'BEIJING',
 'LONDON',
 'LUXEMBOURG',
 'SINGAPORE',

In [9]:
stopwords_names = '''SMITH  | Surnames from 1990 census > .002%.  www.census.gov.genealogy/names/dist.all.last
JOHNSON
WILLIAMS
JONES
BROWN
DAVIS
MILLER
WILSON
MOORE
TAYLOR
ANDERSON
THOMAS
JACKSON
WHITE
HARRIS
MARTIN
THOMPSON
GARCIA
MARTINEZ
ROBINSON
CLARK
RODRIGUEZ
LEWIS
LEE
WALKER
HALL
ALLEN
YOUNG
HERNANDEZ
KING
WRIGHT
LOPEZ
HILL
SCOTT
GREEN
ADAMS
BAKER
GONZALEZ
NELSON
CARTER
MITCHELL
PEREZ
ROBERTS
TURNER
PHILLIPS
CAMPBELL
PARKER
EVANS
EDWARDS
COLLINS
STEWART
SANCHEZ
MORRIS
ROGERS
REED
COOK
MORGAN
BELL
MURPHY
BAILEY
RIVERA
COOPER
RICHARDSON
COX
HOWARD
WARD
TORRES
PETERSON
GRAY
RAMIREZ
JAMES
WATSON
BROOKS
KELLY
SANDERS
PRICE
BENNETT
WOOD
BARNES
ROSS
HENDERSON
COLEMAN
JENKINS
PERRY
POWELL
LONG
PATTERSON
HUGHES
FLORES
WASHINGTON
BUTLER
SIMMONS
FOSTER
GONZALES
BRYANT
ALEXANDER
RUSSELL
GRIFFIN
DIAZ
HAYES
MYERS
FORD
HAMILTON
GRAHAM
SULLIVAN
WALLACE
WOODS
COLE
WEST
JORDAN
OWENS
REYNOLDS
FISHER
ELLIS
HARRISON
GIBSON
MCDONALD
CRUZ
MARSHALL
ORTIZ
GOMEZ
MURRAY
FREEMAN
WELLS
WEBB
SIMPSON
STEVENS
TUCKER
PORTER
HUNTER
HICKS
CRAWFORD
HENRY
BOYD
MASON
MORALES
KENNEDY
WARREN
DIXON
RAMOS
REYES
BURNS
GORDON
SHAW
HOLMES
RICE
ROBERTSON
HUNT
BLACK
DANIELS
PALMER
MILLS
NICHOLS
GRANT
KNIGHT
FERGUSON
ROSE
STONE
HAWKINS
DUNN
PERKINS
HUDSON
SPENCER
GARDNER
STEPHENS
PAYNE
PIERCE
BERRY
MATTHEWS
ARNOLD
WAGNER
WILLIS
RAY
WATKINS
OLSON
CARROLL
DUNCAN
SNYDER
HART
CUNNINGHAM
BRADLEY
LANE
ANDREWS
RUIZ
HARPER
FOX
RILEY
ARMSTRONG
CARPENTER
WEAVER
GREENE
LAWRENCE
ELLIOTT
CHAVEZ
SIMS
AUSTIN
PETERS
KELLEY
FRANKLIN
LAWSON
FIELDS
GUTIERREZ
RYAN
SCHMIDT
CARR
VASQUEZ
CASTILLO
WHEELER
CHAPMAN
OLIVER
MONTGOMERY
RICHARDS
WILLIAMSON
JOHNSTON
BANKS
MEYER
BISHOP
MCCOY
HOWELL
ALVAREZ
MORRISON
HANSEN
FERNANDEZ
GARZA
HARVEY
LITTLE
BURTON
STANLEY
NGUYEN
GEORGE
JACOBS
REID
KIM
FULLER
LYNCH
DEAN
GILBERT
GARRETT
ROMERO
WELCH
LARSON
FRAZIER
BURKE
HANSON
DAY
MENDOZA
MORENO
BOWMAN
MEDINA
FOWLER
BREWER
HOFFMAN
CARLSON
SILVA
PEARSON
HOLLAND
DOUGLAS
FLEMING
JENSEN
VARGAS
BYRD
DAVIDSON
HOPKINS
MAY
TERRY
HERRERA
WADE
SOTO
WALTERS
CURTIS
NEAL
CALDWELL
LOWE
JENNINGS
BARNETT
GRAVES
JIMENEZ
HORTON
SHELTON
BARRETT
OBRIEN
CASTRO
SUTTON
GREGORY
MCKINNEY
LUCAS
MILES
CRAIG
RODRIQUEZ
CHAMBERS
HOLT
LAMBERT
FLETCHER
WATTS
BATES
HALE
RHODES
PENA
BECK
NEWMAN
HAYNES
MCDANIEL
MENDEZ
BUSH
VAUGHN
PARKS
DAWSON
SANTIAGO
NORRIS
HARDY
LOVE
STEELE
CURRY
POWERS
SCHULTZ
BARKER
GUZMAN
PAGE
MUNOZ
BALL
KELLER
CHANDLER
WEBER
LEONARD
WALSH
LYONS
RAMSEY
WOLFE
SCHNEIDER
MULLINS
BENSON
SHARP
BOWEN
DANIEL
BARBER
CUMMINGS
HINES
BALDWIN
GRIFFITH
VALDEZ
HUBBARD
SALAZAR
REEVES
WARNER
STEVENSON
BURGESS
SANTOS
TATE
CROSS
GARNER
MANN
MACK
MOSS
THORNTON
DENNIS
MCGEE
FARMER
DELGADO
AGUILAR
VEGA
GLOVER
MANNING
COHEN
HARMON
RODGERS
ROBBINS
NEWTON
TODD
BLAIR
HIGGINS
INGRAM
REESE
CANNON
STRICKLAND
TOWNSEND
POTTER
GOODWIN
WALTON
ROWE
HAMPTON
ORTEGA
PATTON
SWANSON
JOSEPH
FRANCIS
GOODMAN
MALDONADO
YATES
BECKER
ERICKSON
HODGES
RIOS
CONNER
ADKINS
WEBSTER
NORMAN
MALONE
HAMMOND
FLOWERS
COBB
MOODY
QUINN
BLAKE
MAXWELL
POPE
FLOYD
OSBORNE
PAUL
MCCARTHY
GUERRERO
LINDSEY
ESTRADA
SANDOVAL
GIBBS
TYLER
GROSS
FITZGERALD
STOKES
DOYLE
SHERMAN
SAUNDERS
WISE
COLON
GILL
ALVARADO
GREER
PADILLA
SIMON
WATERS
NUNEZ
BALLARD
SCHWARTZ
MCBRIDE
HOUSTON
CHRISTENSEN
KLEIN
PRATT
BRIGGS
PARSONS
MCLAUGHLIN
ZIMMERMAN
FRENCH
BUCHANAN
MORAN
COPELAND
ROY
PITTMAN
BRADY
MCCORMICK
HOLLOWAY
BROCK
POOLE
FRANK
LOGAN
OWEN
BASS
MARSH
DRAKE
WONG
JEFFERSON
PARK
MORTON
ABBOTT
SPARKS
PATRICK
NORTON
HUFF
CLAYTON
MASSEY
LLOYD
FIGUEROA
CARSON
BOWERS
ROBERSON
BARTON
TRAN
LAMB
HARRINGTON
CASEY
BOONE
CORTEZ
CLARKE
MATHIS
SINGLETON
WILKINS
CAIN
BRYAN
UNDERWOOD
HOGAN
MCKENZIE
COLLIER
LUNA
PHELPS
MCGUIRE
ALLISON
BRIDGES
WILKERSON
NASH
SUMMERS
ATKINS
WILCOX
PITTS
CONLEY
MARQUEZ
BURNETT
RICHARD
COCHRAN
CHASE
DAVENPORT
HOOD
GATES
CLAY
AYALA
SAWYER
ROMAN
VAZQUEZ
DICKERSON
HODGE
ACOSTA
FLYNN
ESPINOZA
NICHOLSON
MONROE
WOLF
MORROW
KIRK
RANDALL
ANTHONY
WHITAKER
OCONNOR
SKINNER
WARE
MOLINA
KIRBY
HUFFMAN
BRADFORD
CHARLES
GILMORE
DOMINGUEZ
ONEAL
BRUCE
LANG
COMBS
KRAMER
HEATH
HANCOCK
GALLAGHER
GAINES
SHAFFER
SHORT
WIGGINS
MATHEWS
MCCLAIN
FISCHER
WALL
SMALL
MELTON
HENSLEY
BOND
DYER
CAMERON
GRIMES
CONTRERAS
CHRISTIAN
WYATT
BAXTER
SNOW
MOSLEY
SHEPHERD
LARSEN
HOOVER
BEASLEY
GLENN
PETERSEN
WHITEHEAD
MEYERS
KEITH
GARRISON
VINCENT
SHIELDS
HORN
SAVAGE
OLSEN
SCHROEDER
HARTMAN
WOODARD
MUELLER
KEMP
DELEON
BOOTH
PATEL
CALHOUN
WILEY
EATON
CLINE
NAVARRO
HARRELL
LESTER
HUMPHREY
PARRISH
DURAN
HUTCHINSON
HESS
DORSEY
BULLOCK
ROBLES
BEARD
DALTON
AVILA
VANCE
RICH
BLACKWELL
YORK
JOHNS
BLANKENSHIP
TREVINO
SALINAS
CAMPOS
PRUITT
MOSES
CALLAHAN
GOLDEN
MONTOYA
HARDIN
GUERRA
MCDOWELL
CAREY
STAFFORD
GALLEGOS
HENSON
WILKINSON
BOOKER
MERRITT
MIRANDA
ATKINSON
ORR
DECKER
HOBBS
PRESTON
TANNER
KNOX
PACHECO
STEPHENSON
GLASS
ROJAS
SERRANO
MARKS
HICKMAN
ENGLISH
SWEENEY
STRONG
PRINCE
MCCLURE
CONWAY
WALTER
ROTH
MAYNARD
FARRELL
LOWERY
HURST
NIXON
WEISS
TRUJILLO
ELLISON
SLOAN
JUAREZ
WINTERS
MCLEAN
RANDOLPH
LEON
BOYER
VILLARREAL
MCCALL
GENTRY
CARRILLO
KENT
AYERS
LARA
SHANNON
SEXTON
PACE
HULL
LEBLANC
BROWNING
VELASQUEZ
LEACH
CHANG
HOUSE
SELLERS
HERRING
NOBLE
FOLEY
BARTLETT
MERCADO
LANDRY
DURHAM
WALLS
BARR
MCKEE
BAUER
RIVERS
EVERETT
BRADSHAW
PUGH
VELEZ
RUSH
ESTES
DODSON
MORSE
SHEPPARD
WEEKS
CAMACHO
BEAN
BARRON
LIVINGSTON
MIDDLETON
SPEARS
BRANCH
BLEVINS
CHEN
KERR
MCCONNELL
HATFIELD
HARDING
ASHLEY
SOLIS
HERMAN
FROST
GILES
BLACKBURN
WILLIAM
PENNINGTON
WOODWARD
FINLEY
MCINTOSH
KOCH
BEST
SOLOMON
MCCULLOUGH
DUDLEY
NOLAN
BLANCHARD
RIVAS
BRENNAN
MEJIA
KANE
BENTON
JOYCE
BUCKLEY
HALEY
VALENTINE
MADDOX
RUSSO
MCKNIGHT
BUCK
MOON
MCMILLAN
CROSBY
BERG
DOTSON
MAYS
ROACH
CHURCH
CHAN
RICHMOND
MEADOWS
FAULKNER
ONEILL
KNAPP
KLINE
BARRY
OCHOA
JACOBSON
GAY
AVERY
HENDRICKS
HORNE
SHEPARD
HEBERT
CHERRY
CARDENAS
MCINTYRE
WHITNEY
WALLER
HOLMAN
DONALDSON
CANTU
TERRELL
MORIN
GILLESPIE
FUENTES
TILLMAN
SANFORD
BENTLEY
PECK
KEY
SALAS
ROLLINS
GAMBLE
DICKSON
BATTLE
SANTANA
CABRERA
CERVANTES
HOWE
HINTON
HURLEY
SPENCE
ZAMORA
YANG
MCNEIL
SUAREZ
CASE
PETTY
GOULD
MCFARLAND
SAMPSON
CARVER
BRAY
ROSARIO
MACDONALD
STOUT
HESTER
MELENDEZ
DILLON
FARLEY
HOPPER
GALLOWAY
POTTS
BERNARD
JOYNER
STEIN
AGUIRRE
OSBORN
MERCER
BENDER
FRANCO
ROWLAND
SYKES
BENJAMIN
TRAVIS
PICKETT
CRANE
SEARS
MAYO
DUNLAP
HAYDEN
WILDER
MCKAY
COFFEY
MCCARTY
EWING
COOLEY
VAUGHAN
BONNER
COTTON
HOLDER
STARK
FERRELL
CANTRELL
FULTON
LYNN
LOTT
CALDERON
ROSA
POLLARD
HOOPER
BURCH
MULLEN
FRY
RIDDLE
LEVY
DAVID
DUKE
ODONNELL
GUY
MICHAEL
BRITT
FREDERICK
DAUGHERTY
BERGER
DILLARD
ALSTON
JARVIS
FRYE
RIGGS
CHANEY
ODOM
DUFFY
FITZPATRICK
VALENZUELA
MERRILL
MAYER
ALFORD
MCPHERSON
ACEVEDO
DONOVAN
BARRERA
ALBERT
COTE
REILLY
COMPTON
RAYMOND
MOONEY
MCGOWAN
CRAFT
CLEVELAND
CLEMONS
WYNN
NIELSEN
BAIRD
STANTON
SNIDER
ROSALES
BRIGHT
WITT
STUART
HAYS
HOLDEN
RUTLEDGE
KINNEY
CLEMENTS
CASTANEDA
SLATER
HAHN
EMERSON
CONRAD
BURKS
DELANEY
PATE
LANCASTER
SWEET
JUSTICE
TYSON
SHARPE
WHITFIELD
TALLEY
MACIAS
IRWIN
BURRIS
RATLIFF
MCCRAY
MADDEN
KAUFMAN
BEACH
GOFF
CASH
BOLTON
MCFADDEN
LEVINE
GOOD
BYERS
KIRKLAND
KIDD
WORKMAN
CARNEY
DALE
MCLEOD
HOLCOMB
ENGLAND
FINCH
HEAD
BURT
HENDRIX
SOSA
HANEY
FRANKS
SARGENT
NIEVES
DOWNS
RASMUSSEN
BIRD
HEWITT
LINDSAY
LE
FOREMAN
VALENCIA
ONEIL
DELACRUZ
VINSON
DEJESUS
HYDE
FORBES
GILLIAM
GUTHRIE
WOOTEN
HUBER
BARLOW
BOYLE
MCMAHON
BUCKNER
ROCHA
PUCKETT
LANGLEY
KNOWLES
COOKE
VELAZQUEZ
WHITLEY
NOEL
VANG
SHEA
ROUSE
HARTLEY
MAYFIELD
ELDER
RANKIN
HANNA
COWAN
LUCERO
ARROYO
SLAUGHTER
HAAS
OCONNELL
MINOR
KENDRICK
SHIRLEY
KENDALL
BOUCHER
ARCHER
BOGGS
ODELL
DOUGHERTY
ANDERSEN
NEWELL
CROWE
WANG
FRIEDMAN
BLAND
SWAIN
HOLLEY
FELIX
PEARCE
CHILDS
YARBROUGH
GALVAN
PROCTOR
MEEKS
LOZANO
MORA
RANGEL
BACON
VILLANUEVA
SCHAEFER
ROSADO
HELMS
BOYCE
GOSS
STINSON
SMART
LAKE
IBARRA
HUTCHINS
COVINGTON
REYNA
GREGG
WERNER
CROWLEY
HATCHER
MACKEY
BUNCH
WOMACK
POLK
JAMISON
DODD
CHILDRESS
CHILDERS
CAMP
VILLA
DYE
SPRINGER
MAHONEY
DAILEY
BELCHER
LOCKHART
GRIGGS
COSTA
CONNOR
BRANDT
WINTER
WALDEN
MOSER
TRACY
TATUM
MCCANN
AKERS
LUTZ
PRYOR
LAW
OROZCO
MCALLISTER
LUGO
DAVIES
SHOEMAKER
MADISON
RUTHERFORD
NEWSOME
MAGEE
CHAMBERLAIN
BLANTON
SIMMS
GODFREY
FLANAGAN
CRUM
CORDOVA
ESCOBAR
DOWNING
SINCLAIR
DONAHUE
KRUEGER
MCGINNIS
GORE
FARRIS
WEBBER
CORBETT
ANDRADE
STARR
LYON
YODER
HASTINGS
MCGRATH
SPIVEY
KRAUSE
HARDEN
CRABTREE
KIRKPATRICK
HOLLIS
BRANDON
ARRINGTON
ERVIN
CLIFTON
RITTER
MCGHEE
BOLDEN
MALONEY
GAGNON
DUNBAR
PONCE
PIKE
MAYES
HEARD
BEATTY
MOBLEY
KIMBALL
BUTTS
MONTES
HERBERT
GRADY
ELDRIDGE
BRAUN
HAMM
GIBBONS
SEYMOUR
MOYER
MANLEY
HERRON
PLUMMER
ELMORE
CRAMER
GARY
RUCKER
HILTON
BLUE
PIERSON
FONTENOT
FIELD
RUBIO
GRACE
GOLDSTEIN
ELKINS
WILLS
NOVAK
JOHN
HICKEY
WORLEY
GORMAN
KATZ
DICKINSON
BROUSSARD
FRITZ
WOODRUFF
CROW
CHRISTOPHER
BRITTON
FORREST
NANCE
LEHMAN
BINGHAM
ZUNIGA
WHALEY
SHAFER
COFFMAN
STEWARD
DELAROSA
NIX
NEELY
NUMBERS
MATA
MANUEL
DAVILA
MCCABE
KESSLER
EMERY
BOWLING
HINKLE
WELSH
PAGAN
GOLDBERG
GOINS
CROUCH
CUEVAS
QUINONES
MCDERMOTT
HENDRICKSON
SAMUELS
DENTON
BERGERON
LAM
IVEY
LOCKE
HAINES
THURMAN
SNELL
HOSKINS
BYRNE
MILTON
WINSTON
ARTHUR
ARIAS
STANFORD
ROE
CORBIN
BELTRAN
CHAPPELL
HURT
DOWNEY
DOOLEY
TUTTLE
COUCH
PAYTON
MCELROY
CROCKETT
GROVES
CLEMENT
LESLIE
CARTWRIGHT
DICKEY
MCGILL
DUBOIS
MUNIZ
ERWIN
SELF
TOLBERT
DEMPSEY
CISNEROS
SEWELL
LATHAM
GARLAND
VIGIL
TAPIA
STERLING
RAINEY
NORWOOD
LACY
STROUD
MEADE
AMOS
TIPTON
LORD
KUHN
HILLIARD
BONILLA
TEAGUE
COURTNEY
GUNN
HO
GREENWOOD
CORREA
REECE
WESTON
POE
TRENT
PINEDA
PHIPPS
FREY
KAISER
AMES
PAIGE
GUNTER
SCHMITT
MILLIGAN
ESPINOSA
CARLTON
BOWDEN
VICKERS
LOWRY
PRITCHARD
COSTELLO
PIPER
MCCLELLAN
LOVELL
DREW
SHEEHAN
QUICK
HATCH
DOBSON
SINGH
JEFFRIES
HOLLINGSWORTH
SORENSEN
MEZA
FINK
DONNELLY
BURRELL
BRUNO
TOMLINSON
COLBERT
BILLINGS
RITCHIE
HELTON
SUTHERLAND
PEOPLES
MCQUEEN
GASTON
THOMASON
MCKINLEY
GIVENS
CROCKER
VOGEL
ROBISON
DUNHAM
COKER
SWARTZ
KEYS
LILLY
LADNER
HANNAH
WILLARD
RICHTER
HARGROVE
EDMONDS
BRANTLEY
ALBRIGHT
MURDOCK
BOSWELL
MULLER
QUINTERO
PADGETT
KENNEY
DALY
CONNOLLY
PIERRE
INMAN
QUINTANA
LUND
BARNARD
VILLEGAS
SIMONS
LAND
HUGGINS
TIDWELL
SANDERSON
BULLARD
MCCLENDON
DUARTE
DRAPER
MEREDITH
MARRERO
DWYER
ABRAMS
STOVER
GOODE
FRASER
CREWS
BERNAL
SMILEY
GODWIN
FISH
CONKLIN
MCNEAL
BACA
ESPARZA
CROWDER
BOWER
NICHOLAS
CHUNG
BREWSTER
MCNEILL
DICK
RODRIGUES
LEAL
COATES
RAINES
MCCAIN
MCCORD
MINER
HOLBROOK
SWIFT
DUKES
CARLISLE
ALDRIDGE
ACKERMAN
STARKS
RICKS
HOLLIDAY
FERRIS
HAIRSTON
SHEFFIELD
LANGE
FOUNTAIN
MARINO
DOSS
BETTS
KAPLAN
CARMICHAEL
BLOOM
RUFFIN
PENN
KERN
BOWLES
SIZEMORE
LARKIN
DUPREE
JEWELL
SILVER
SEALS
METCALF
HUTCHISON
HENLEY
FARR
CASTLE
MCCAULEY
HANKINS
GUSTAFSON
DEAL
CURRAN
ASH
WADDELL
RAMEY
CATES
POLLOCK
MAJOR
IRVIN
CUMMINS
MESSER
HELLER
DEWITT
LIN
FUNK
CORNETT
PALACIOS
GALINDO
CANO
HATHAWAY
SINGER
PHAM
ENRIQUEZ
AARON
SALGADO
PELLETIER
PAINTER
WISEMAN
BLOUNT
HAND
FELICIANO
TEMPLE
HOUSER
DOHERTY
MEAD
MCGRAW
TONEY
SWAN
MELVIN
CAPPS
BLANCO
BLACKMON
WESLEY
THOMSON
MCMANUS
FAIR
BURKETT
POST
GLEASON
RUDOLPH
OTT
DICKENS
CORMIER
VOSS
RUSHING
ROSENBERG
HURD
DUMAS
BENITEZ
ARELLANO
STORY
MARIN
CAUDILL
BRAGG
JARAMILLO
HUERTA
GIPSON
COLVIN
BIGGS
VELA
PLATT
CASSIDY
TOMPKINS
MCCOLLUM
KAY
GABRIEL
DOLAN
DALEY
CRUMP
STREET
SNEED
KILGORE
GROVE
GRIMM
DAVISON
BRUNSON
PRATER
MARCUM
DEVINE
KYLE
DODGE
STRATTON
ROSAS
CHOI
TRIPP
LEDBETTER
LAY
HIGHTOWER
HAYWOOD
FELDMAN
EPPS
YEAGER
POSEY
SYLVESTER
SCRUGGS
COPE
STUBBS
RICHEY
OVERTON
TROTTER
SPRAGUE
CORDERO
BUTCHER
BURGER
STILES
BURGOS
WOODSON
HORNER
BASSETT
PURCELL
HASKINS
GEE
AKINS
ABRAHAM
HOYT
ZIEGLER
SPAULDING
HADLEY
GRUBBS
SUMNER
MURILLO
ZAVALA
SHOOK
LOCKWOOD
JARRETT
DRISCOLL
DAHL
THORPE
SHERIDAN
REDMOND
PUTNAM
MCWILLIAMS
MCRAE
CORNELL
FELTON
ROMANO
JOINER
SADLER
HEDRICK
HAGER
HAGEN
FITCH
COULTER
THACKER
MANSFIELD
LANGSTON
GUIDRY
FERREIRA
CORLEY
CONN
ROSSI
LACKEY
CODY
BAEZ
SAENZ
MCNAMARA
DARNELL
MICHEL
MCMULLEN
MCKENNA
MCDONOUGH
LINK
ENGEL
BROWNE
ROPER
PEACOCK
EUBANKS
DRUMMOND
STRINGER
PRITCHETT
PARHAM
MIMS
LANDERS
HAM
GRAYSON
STACY
SCHAFER
EGAN
TIMMONS
OHARA
KEEN
HAMLIN
FINN
CORTES
MCNAIR
LOUIS
CLIFFORD
NADEAU
MOSELEY
MICHAUD
ROSEN
OAKES
KURTZ
JEFFERS
CALLOWAY
BEAL
BAUTISTA
WINN
SUGGS
STERN
STAPLETON
LYLES
LAIRD
MONTANO
DIAMOND
DAWKINS
ROLAND
HAGAN
GOLDMAN
BRYSON
BARAJAS
LOVETT
SEGURA
METZ
LOCKETT
LANGFORD
HINSON
EASTMAN
ROCK
HOOKS
WOODY
SMALLWOOD
SHAPIRO
CROWELL
WHALEN
TRIPLETT
HOOKER
CHATMAN
ALDRICH
CAHILL
YOUNGBLOOD
YBARRA
STALLINGS
SHEETS
SAMUEL
REEDER
PERSON
PACK
LACEY
CONNELLY
BATEMAN
ABERNATHY
WINKLER
WILKES
MASTERS
HACKETT
GRANGER
GILLIS
SCHMITZ
SAPP
NAPIER
SOUZA
LANIER
GOMES
WEIR
OTERO
LEDFORD
BURROUGHS
BABCOCK
VENTURA
SIEGEL
DUGAN
CLINTON
CHRISTIE
BLEDSOE
ATWOOD
WRAY
VARNER
SPANGLER
OTTO
ANAYA
STALEY
KRAFT
FOURNIER
EDDY
BELANGER
WOLFF
THORNE
BYNUM
BURNETTE
BOYKIN
SWENSON
PURVIS
PINA
KHAN
DUVALL
DARBY
XIONG
KAUFFMAN
ALI
YU
HEALY
ENGLE
CORONA
BENOIT
VALLE
STEINER
SPICER
SHAVER
RANDLE
LUNDY
DOW
CHIN
CALVERT
STATON
NEFF
KEARNEY
DARDEN
OAKLEY
MEDEIROS
MCCRACKEN
CRENSHAW
BLOCK
BEAVER
PERDUE
DILL
WHITTAKER
TOBIN
CORNELIUS
WASHBURN
HOGUE
GOODRICH
EASLEY
BRAVO
DENNISON
VERA
SHIPLEY
KERNS
JORGENSEN
CRAIN
ABEL
VILLALOBOS
MAURER
LONGORIA
KEENE
COON
SIERRA
WITHERSPOON
STAPLES
PETTIT
KINCAID
EASON
MADRID
ECHOLS
LUSK
WU
STAHL
CURRIE
THAYER
SHULTZ
SHERWOOD
MCNALLY
SEAY
NORTH
MAHER
KENNY
HOPE
GAGNE
BARROW
NAVA
MYLES
MORELAND
HONEYCUTT
HEARN
DIGGS
CARON
WHITTEN
WESTBROOK
STOVALL
RAGLAND
QUEEN
MUNSON
MEIER
LOONEY
KIMBLE
JOLLY
HOBSON
LONDON
GODDARD
CULVER
BURR
PRESLEY
NEGRON
CONNELL
TOVAR
MARCUS
HUDDLESTON
HAMMER
ASHBY
SALTER
ROOT
PENDLETON
OLEARY
NICKERSON
MYRICK
JUDD
JACOBSEN
ELLIOT
BAIN
ADAIR
STARNES
SHELDON
MATOS
LIGHT
BUSBY
HERNDON
HANLEY
BELLAMY
JACK
DOTY
BARTLEY
YAZZIE
ROWELL
PARSON
GIFFORD
CULLEN
CHRISTIANSEN
BENAVIDES
BARNHART
TALBOT
MOCK
CRANDALL
CONNORS
BONDS
WHITT
GAGE
BERGMAN
ARREDONDO
ADDISON
MARION
LUJAN
DOWDY
JERNIGAN
HUYNH
BOUCHARD
DUTTON
RHOADES
OUELLETTE
KISER
RUBIN
HERRINGTON
HARE
DENNY
BLACKMAN
BABB
ALLRED
RUDD
PAULSON
OGDEN
KOENIG
JACOB
IRVING
GEIGER
BEGAY
PARRA
CHAMPION
LASSITER
HAWK
ESPOSITO
CHO
WALDRON
VERNON
RANSOM
PRATHER
KEENAN
JEAN
GROVER
CHACON
VICK
SANDS
ROARK
PARR
MAYBERRY
GREENBERG
COLEY
BRUNER
WHITMAN
SKAGGS
SHIPMAN
MEANS
LEARY
HUTTON
ROMO
MEDRANO
LADD
KRUSE
FRIEND
DARLING
ASKEW
VALENTIN
SCHULZ
ALFARO
TABOR
MOHR
GALLO
BERMUDEZ
PEREIRA
ISAAC
BLISS
REAVES
FLINT
COMER
BOSTON
WOODALL
NAQUIN
GUEVARA
EARL
DELONG
CARRIER
PICKENS
BRAND
TILLEY
SCHAFFER
READ
LIM
KNUTSON
FENTON
DORAN
CHU
VOGT
VANN
PRESCOTT
MCLAIN
LANDIS
CORCORAN
AMBROSE
ZAPATA
HYATT
HEMPHILL
FAULK
CALL
DOVE
BOUDREAUX
ARAGON
WHITLOCK
TREJO
TACKETT
SHEARER
SALDANA
HANKS
GOLD
DRIVER
MCKINNON
KOEHLER
CHAMPAGNE
BOURGEOIS
POOL
KEYES
GOODSON
FOOTE
EARLY
LUNSFORD
GOLDSMITH
FLOOD
WINSLOW
SAMS
REAGAN
MCCLOUD
HOUGH
ESQUIVEL
NAYLOR
LOOMIS
CORONADO
LUDWIG
BRASWELL
BEARDEN
SHERRILL
HUANG
FAGAN
EZELL
EDMONDSON
CYR
CRONIN
NUNN
LEMON
GUILLORY
GRIER
DUBOSE
TRAYLOR
RYDER
DOBBINS
COYLE
APONTE
WHITMORE
SMALLS
ROWAN
MALLOY
CARDONA
BRAXTON
BORDEN
HUMPHRIES
CARRASCO
RUFF
METZGER
HUNTLEY
HINOJOSA
FINNEY
MADSEN
HONG
HILLS
ERNST
DOZIER
BURKHART
BOWSER
PERALTA
DAIGLE
WHITTINGTON
SORENSON
SAUCEDO
ROCHE
REDDING
LOYD
FUGATE
AVALOS
WAITE
LIND
HUSTON
HAY
BENEDICT
HAWTHORNE
HAMBY
BOYLES
BOLES
REGAN
FAUST
CROOK
BEAM
BARGER
HINDS
GALLARDO
ELIAS
WILLOUGHBY
WILLINGHAM
WILBURN
ECKERT
BUSCH
ZEPEDA
WORTHINGTON
TINSLEY
RUSS
LI
HOFF
HAWLEY
CARMONA
VARELA
RECTOR
NEWCOMB
MALLORY
KINSEY
DUBE
WHATLEY
STRANGE
RAGSDALE
IVY
BERNSTEIN
BECERRA
YOST
MATTSON
LY
FELDER
CHEEK
LUKE
HANDY
GROSSMAN
GAUTHIER
ESCOBEDO
BRADEN
BECKMAN
MOTT
HILLMAN
GIL
FLAHERTY
DYKES
DOE
STOCKTON
STEARNS
LOFTON
KITCHEN
COATS
CAVAZOS
BEAVERS
BARRIOS
TANG
PARISH
MOSHER
LINCOLN
CARDWELL
COLES
BURNHAM
WELLER
LEMONS
BEEBE
AGUILERA
RING
PARNELL
HARMAN
COUTURE
ALLEY
SCHUMACHER
REDD
DOBBS
BLUM
BLALOCK
MERCHANT
ENNIS
DENSON
COTTRELL
CHESTER
BRANNON
BAGLEY
AVILES
WATT
SOUSA
ROSENTHAL
ROONEY
DIETZ
BLANK
PAQUETTE
MCCLELLAND
DUFF
VELASCO
LENTZ
GRUBB
BURROWS
BARBOUR
ULRICH
SHOCKLEY
RADER
GERMAN
BEYER
MIXON
LAYTON
ALTMAN
ALONZO
WEATHERS
TITUS
STONER
SQUIRES
SHIPP
PRIEST
LIPSCOMB
CUTLER
CABALLERO
ZIMMER
WILLETT
THURSTON
STOREY
MEDLEY
LYLE
EPPERSON
SHAH
MCMILLIAN
BAGGETT
TORREZ
LAWS
HIRSCH
DENT
COREY
POIRIER
PEACHEY
JACQUES
FARRAR
CREECH
BARTH
TRIMBLE
FRANCE
DUPRE
ALBRECHT
SAMPLE
LAWLER
CRISP
CONROY
CHADWICK
WETZEL
NESBITT
MURRY
JAMESON
WILHELM
PATTEN
MINTON
MATSON
KIMBROUGH
IVERSON
GUINN
GALE
FORTUNE
CROFT
TOTH
PULLIAM
NUGENT
NEWBY
LITTLEJOHN
DIAS
CANALES
BERNIER
BARON
BARNEY
SINGLETARY
RENTERIA
PRUETT
MCHUGH
MABRY
LANDRUM
BROWER
WELDON
STODDARD
RUTH
CAGLE
STJOHN
SCALES
KOHLER
KELLOGG
HOPSON
GANT
THARP
GANN
ZEIGLER
PRINGLE
HAMMONS
FAIRCHILD
DEATON
CHAVIS
CARNES
ROWLEY
MATLOCK
LIBBY
KEARNS
IRIZARRY
CARRINGTON
STARKEY
PEPPER
LOPES
JARRELL
FAY
CRAVEN
BEVERLY
BAUM
SPAIN
LITTLEFIELD
LINN
HUMPHREYS
HOOK
HIGH
ETHERIDGE
CUELLAR
CHASTAIN
CHANCE
BUNDY
SPEER
SKELTON
QUIROZ
PYLE
PORTILLO
PONDER
MOULTON
MACHADO
LIU
KILLIAN
HUTSON
HITCHCOCK
ELLSWORTH
DOWLING
CLOUD
BURDICK
SPANN
PEDERSEN
LEVIN
LEGGETT
HAYWARD
HACKER
DIETRICH
BEAULIEU
BARKSDALE
WAKEFIELD
SNOWDEN
PARIS
BRISCOE
BOWIE
BERMAN
OGLE
MCGREGOR
LAUGHLIN
HELM
BURDEN
WHEATLEY
SCHREIBER
PRESSLEY
PARRIS
NG
ALANIZ
AGEE
URBAN
SWANN
SNODGRASS
SCHUSTER
RADFORD
MONK
MATTINGLY
MAIN
LAMAR
HARP
GIRARD
CHENEY
YANCEY
WAGONER
RIDLEY
LOMBARDO
LAU
HUDGINS
GASKINS
DUCKWORTH
COE
COBURN
WILLEY
PRADO
NEWBERRY
MAGANA
HAMMONDS
ELAM
WHIPPLE
SLADE
SERNA
OJEDA
LILES
DORMAN
DIEHL
ANGEL
UPTON
REARDON
MICHAELS
KELSEY
GOETZ
ELLER
BAUMAN
BAER
AUGUSTINE
LAYNE
HUMMEL
BRENNER
AMAYA
ADAMSON
ORNELAS
DOWELL
CLOUTIER
CHRISTY
CASTELLANOS
WING
WELLMAN
SAYLOR
OROURKE
MOYA
MONTALVO
KILPATRICK
HARLEY
DURBIN
SHELL
OLDHAM
KANG
GARVIN
FOSS
BRANHAM
BARTHOLOMEW
TEMPLETON
MAGUIRE
HOLTON
ALONSO
RIDER
MONAHAN
MCCORMACK
BEATY
ANDERS
STREETER
NIETO
NIELSON
MOFFETT
LANKFORD
KEATING
HECK
GATLIN
DELATORRE
CALLAWAY
ADCOCK
WORRELL
UNGER
ROBINETTE
NOWAK
JETER
BRUNNER
ASHTON
STEEN
PARROTT
OVERSTREET
NOBLES
MONTANEZ
LUTHER
CLEVENGER
BRINKLEY
TRAHAN
QUARLES
PICKERING
PEDERSON
JANSEN
GRANTHAM
GILCHRIST
CRESPO
AIKEN
SCHELL
SCHAEFFER
LORENZ
LEYVA
HARMS
DYSON
WALLIS
PEASE
LEAVITT
HYMAN
CHENG
CAVANAUGH
BATTS
WARDEN
SEAMAN
ROCKWELL
QUEZADA
PAXTON
LINDER
HOUCK
FONTAINE
DURANT
CARUSO
ADLER
PIMENTEL
MIZE
LYTLE
DONALD
CLEARY
CASON
ACKER
SWITZER
SALMON
ISAACS
HIGGINBOTHAM
HAN
WATERMAN
VANDYKE
STAMPER
SISK
SHULER
RIDDICK
REDMAN
MCMAHAN
LEVESQUE
HATTON
BRONSON
BOLLINGER
ARNETT
OKEEFE
GERBER
GANNON
FARNSWORTH
BAUGHMAN
SILVERMAN
SATTERFIELD
ROYAL
MCCRARY
KOWALSKI
JOY
GRIGSBY
GRECO
CABRAL
TROUT
RINEHART
MAHON
LINTON
GOODEN
CURLEY
BAUGH
WYMAN
WEINER
SCHWAB
SCHULER
MORRISSEY
MAHAN
COY
BUNN
ANDREW
THRASHER
SPEAR
WAGGONER
SHELLEY
ROBERT
QUALLS
PURDY
MCWHORTER
MAULDIN
MARK
JORDON
GILMAN
PERRYMAN
NEWSOM
MENARD
MARTINO
GRAF
BILLINGSLEY
ARTIS
SIMPKINS
SALISBURY
QUINTANILLA
GILLILAND
FRALEY
FOUST
CROUSE
SCARBOROUGH
NGO
GRISSOM
FULTZ
RICO
MARLOW
MARKHAM
MADRIGAL
LAWTON
BARFIELD
WHITING
VARNEY
SCHWARZ
HUEY
GOOCH
ARCE
WHEAT
TRUONG
POULIN
MACKENZIE
LEONE
HURTADO
SELBY
GAITHER
FORTNER
CULPEPPER
COUGHLIN
BRINSON
BOUDREAU
BARKLEY
BALES
STEPP
HOLM
TAN
SCHILLING
MORRELL
KAHN
HEATON
GAMEZ
DOUGLASS
CAUSEY
BROTHERS
TURPIN
SHANKS
SCHRADER
MEEK
ISOM
HARDISON
CARRANZA
YANEZ
WAY
SCROGGINS
SCHOFIELD
RUNYON
RATCLIFF
MURRELL
MOELLER
IRBY
CURRIER
BUTTERFIELD
YEE
RALSTON
PULLEN
PINSON
ESTEP
EAST
CARBONE
LANCE
HAWKS
ELLINGTON
CASILLAS
SPURLOCK
SIKES
MOTLEY
MCCARTNEY
KRUGER
ISBELL
HOULE
FRANCISCO
BURK
BONE
TOMLIN
SHELBY
QUIGLEY
NEUMANN
LOVELACE
FENNELL
COLBY
CHEATHAM
BUSTAMANTE
SKIDMORE
HIDALGO
FORMAN
CULP
BOWENS
BETANCOURT
AQUINO
ROBB
REA
MILNER
MARTEL
GRESHAM
WILES
RICKETTS
GAVIN
DOWD
COLLAZO
BOSTIC
BLAKELY
SHERROD
POWER
KENYON
GANDY
EBERT
DELOACH
CARY
BULL
ALLARD
SAUER
ROBINS
OLIVARES
GILLETTE
CHESTNUT
BOURQUE
PAINE
LYMAN
HITE
HAUSER
DEVORE
CRAWLEY
CHAPA
VU
TOBIAS
TALBERT
POINDEXTER
MILLARD
MEADOR
MCDUFFIE
MATTOX
KRAUS
HARKINS
CHOATE
BESS
WREN
SLEDGE
SANBORN
OUTLAW
KINDER
GEARY
CORNWELL
BARCLAY
ADAM
ABNEY
SEWARD
RHOADS
HOWLAND
FORTIER
EASTER
BENNER
VINES
TUBBS
TROUTMAN
RAPP
NOE
MCCURDY
HARDER
DELUCA
WESTMORELAND
SOUTH
HAVENS
GUAJARDO
ELY
CLARY
SEAL
MEEHAN
HERZOG
GUILLEN
ASHCRAFT
WAUGH
RENNER
MILAM
JUNG
ELROD
CHURCHILL
BUFORD
BREAUX
BOLIN
ASHER
WINDHAM
TIRADO
PEMBERTON
NOLEN
NOLAND
KNOTT
EMMONS
CORNISH
CHRISTENSON
BROWNLEE
BARBEE
WALDROP
PITT
OLVERA
LOMBARDI
GRUBER
GAFFNEY
EGGLESTON
BANDA
ARCHULETA
STILL
SLONE
PREWITT
PFEIFFER
NETTLES
MENA
MCADAMS
HENNING
GARDINER
CROMWELL
CHISHOLM
BURLESON
BOX
VEST
OGLESBY
MCCARTER
MALCOLM
LUMPKIN
LARUE
GREY
WOFFORD
VANHORN
THORN
TEEL
SWAFFORD
STCLAIR
STANFIELD
OCAMPO
HERRMANN
HANNON
ARSENAULT
ROUSH
MCALISTER
HIATT
GUNDERSON
FORSYTHE
DUGGAN
DELVALLE
CINTRON
WILKS
WEINSTEIN
URIBE
RIZZO
NOYES
MCLENDON
GURLEY
BETHEA
WINSTEAD
MAPLES
HARRY
GUYTON
GIORDANO
ALDERMAN
VALDES
POLANCO
PAPPAS
LIVELY
GROGAN
GRIFFITHS
BOBO
AREVALO
WHITSON
SOWELL
RENDON
MATTHEW
JULIAN
FERNANDES
FARROW
EDMOND
BENAVIDEZ
AYRES
ALICEA
STUMP
SMALLEY
SEITZ
SCHULTE
GILLEY
GALLANT
DEWEY
CASPER
CANFIELD
WOLFORD
OMALLEY
MCNUTT
MCNULTY
MCGOVERN
HARDMAN
HARBIN
COWART
CHAVARRIA
BRINK
BECKETT
BAGWELL
ARMSTEAD
ANGLIN
ABREU
REYNOSO
KREBS
JETT
HOFFMANN
GREENFIELD
FORTE
BURNEY
BROOME
SISSON
PARENT
JUDE
YOUNGER
TRAMMELL
PARTRIDGE
MARVIN
MACE
LOMAX
LEMIEUX
GOSSETT
FRANTZ
FOGLE
COONEY
BROUGHTON
PENCE
PAULSEN
NEIL
MUNCY
MCARTHUR
HOLLINS
EDWARD
BEAUCHAMP
WITHERS
OSORIO
MULLIGAN
HOYLE
FOY
DOCKERY
COCKRELL
BEGLEY
AMADOR
ROBY
RAINS
LINDQUIST
GENTILE
EVERHART
BOHANNON
WYLIE
THAO
SOMMERS
PURNELL
PALMA
FORTIN
DUNNING
BREEDEN
VAIL
PHELAN
PHAN
MARX
COSBY
COLBURN
CHONG
BOLING
BIDDLE
LEDESMA
GADDIS
DENNEY
CHOW
BUENO
BERRIOS
WICKER
TOLLIVER
THIBODEAUX
NAGLE
LAVOIE
FISK
DO
CRIST
BARBOSA
REEDY
MARCH
LOCKLEAR
KOLB
HIMES
BEHRENS
BECKWITH
BECKHAM
WEEMS
WAHL
SHORTER
SHACKELFORD
REES
MUSE
FREE
CERDA
VALADEZ
THIBODEAU
SAAVEDRA
RIDGEWAY
REITER
MCHENRY
MAJORS
LACHANCE
KEATON
ISRAEL
FERRARA
FALCON
CLEMENS
BLOCKER
APPLEGATE
PAZ
NEEDHAM
MOJICA
KUYKENDALL
HAMEL
ESCAMILLA
DOUGHTY
BURCHETT
AINSWORTH
WILBUR
VIDAL
UPCHURCH
THIGPEN
STRAUSS
SPRUILL
SOWERS
RIGGINS
RICKER
MCCOMBS
HARLOW
GARNETT
BUFFINGTON
YI
SOTELO
OLIVAS
NEGRETE
MOREY
MACON
LOGSDON
LAPOINTE
FLORENCE
CATHEY
BIGELOW
BELLO
WESTFALL
STUBBLEFIELD
PEAK
LINDLEY
JEFFREY
HEIN
HAWES
FARRINGTON
EDGE
BREEN
BIRCH
WILDE
STEED
SEPULVEDA
REINHARDT
PROFFITT
MINTER
MESSINA
MCNABB
MAIER
KEELER
GAMBOA
DONOHUE
DEXTER
BASHAM
SHINN
ORLANDO
CROOKS
COTA
BORDERS
BILLS
BACHMAN
TISDALE
TAVARES
SCHMID
PICKARD
JASPER
GULLEY
FONSECA
DELOSSANTOS
CONDON
CLANCY
BATISTA
WICKS
WADSWORTH
NEW
MARTELL
LO
LITTLETON
ISON
HAAG
FOLSOM
BRUMFIELD
BROYLES
BRITO
MIRELES
MCDONNELL
LECLAIR
HAMBLIN
GOUGH
FANNING
BINDER
WINFIELD
WHITWORTH
SORIANO
PALUMBO
NEWKIRK
MANGUM
HUTCHERSON
COMSTOCK
CECIL
CARLIN
BEALL
BAIR
WENDT
WATTERS
WALLING
PUTMAN
OTOOLE
OLIVA
MORLEY
MARES
LEMUS
KEENER
JEFFERY
HUNDLEY
DIAL
DAMICO
BILLUPS
STROTHER
MCFARLANE
LAMM
EAVES
CRUTCHER
CARABALLO
CANTY
ATWELL
TAFT
SILER
RUST
RAWLS
RAWLINGS
PRIETO
NILES
MCNEELY
MCAFEE
HULSEY
HARLAN
HACKNEY
GALVEZ
ESCALANTE
DELAGARZA
CRIDER
CHARLTON
BANDY
WILBANKS
STOWE
STEINBERG
SAMSON
RENFRO
MASTERSON
MASSIE
LANHAM
HASKELL
HAMRICK
FORT
DEHART
CARD
BURDETTE
BRANSON
BOURNE
BABIN
ALEMAN
WORTHY
TIBBS
SWEAT
SMOOT
SLACK
PARADIS
PACKARD
MULL
LUCE
HOUGHTON
GANTT
FURMAN
DANNER
CHRISTIANSON
BURGE
BRODERICK
ASHFORD
ARNDT
ALMEIDA
STALLWORTH
SHADE
SEARCY
SAGER
NOONAN
MCLEMORE
MCINTIRE
MAXEY
LAVIGNE
JOBE
IRELAND
FERRER
FALK
EDGAR
COFFIN
BYRNES
ARANDA
APODACA
STAMPS
ROUNDS
PEEK
OLMSTEAD
LEWANDOWSKI
KAMINSKI
HER
DUNAWAY
BRUNS
BRACKETT
AMATO
REICH
MCCLUNG
LACROIX
KOONTZ
HERRICK
HARDESTY
FLANDERS
COUSINS
CLOSE
CATO
CADE
VICKERY
SHANK
NAGEL
DUPUIS
CROTEAU
COTTER
CABLE
STUCKEY
STINE
PORTERFIELD
PAULEY
NYE
MOFFITT
LU
KNUDSEN
HARDWICK
GOFORTH
DUPONT
BLUNT
BARROWS
BARNHILL
SHULL
RASH
RALPH
PENNY
LORENZO
LOFTIS
LEMAY
KITCHENS
HORVATH
GRENIER
FUCHS
FAIRBANKS
CULBERTSON
CALKINS
BURNSIDE
BEATTIE
ASHWORTH
ALBERTSON
WERTZ
VO
VAUGHT
VALLEJO
TYREE
TURK
TUCK
TIJERINA
SAGE
PICARD
PETERMAN
OTIS
MARROQUIN
MARR
LANTZ
HOANG
DEMARCO
DAILY
CONE
BERUBE
BARNETTE
WHARTON
STINNETT
SLOCUM
SCANLON
SANDER
PINTO
MANCUSO
LIMA
JUDGE
HEADLEY
EPSTEIN
COUNTS
CLARKSON
CARNAHAN
BRICE
BOREN
ARTEAGA
ADAME
ZOOK
WHITTLE
WHITEHURST
WENZEL
SAXTON
RHEA
REDDICK
PUENTE
HAZEL
HANDLEY
HAGGERTY
EARLEY
DEVLIN
DALLAS
CHAFFIN
CADY
AHMED
ACUNA
SOLANO
SIGLER
POLLACK
PENDERGRASS
OSTRANDER
JANES
FRANCOIS
FINE
CRUTCHFIELD
CORDELL
CHAMBERLIN
BRUBAKER
BAPTISTE
WILLSON
REIS
NEELEY
MULLIN
MERCIER
LIRA
LAYMAN
KEELING
HIGDON
GUEST
FORRESTER
ESPINAL
DION
CHAPIN
CARL
WARFIELD
TOLEDO
PULIDO
PEEBLES
NAGY
MONTAGUE
MELLO
LEAR
JAEGER
HOGG
GRAFF
FURR
DERRICK
CAVE
CANADA
SOLIZ
POORE
MENDENHALL
MCLAURIN
MAESTAS
LOW
GABLE
BELT
BARRAZA
TILLERY
SNEAD
POND
NEILL
MCCULLOCH
MCCORKLE
LIGHTFOOT
HUTCHINGS
HOLLOMAN
HARNESS
DORN
COUNCIL
BOCK
ZIELINSKI
TURLEY
TREADWELL
STPIERRE
STARLING
SOMERS
OSWALD
MERRICK
MARQUIS
IVORY
EASTERLING
BIVENS
TRUITT
POSTON
PARRY
ONTIVEROS
OLIVAREZ
NEVILLE
MOREAU
MEDLIN
MA
LENZ
KNOWLTON
FAIRLEY
COBBS
CHISOLM
BANNISTER
WOODWORTH
TOLER
OCASIO
NORIEGA
NEUMAN
MOYE
MILBURN
MCCLANAHAN
LILLEY
HANES
FLANNERY
DELLINGER
DANIELSON
CONTI
BLODGETT
BEERS
WEATHERFORD
STRAIN
KARR
HITT
DENHAM
CUSTER
COBLE
CLOUGH
CASTEEL
BOLDUC
BATCHELOR
AMMONS
WHITLOW
TIERNEY
STATEN
SIBLEY
SEIFERT
SCHUBERT
SALCEDO
MATTISON
LANEY
HAGGARD
GROOMS
DIX
DEES
CROMER
COOKS
COLSON
CASWELL
ZARATE
SWISHER
STACEY
SHIN
RAGAN
PRIDGEN
MCVEY
MATHENY
LEIGH
LAFLEUR
FRANZ
FERRARO
DUGGER
WHITESIDE
RIGSBY
MCMURRAY
LEHMANN
LARGE
JACOBY
HILDEBRAND
HENDRICK
HEADRICK
GOAD
FINCHER
DRURY
BORGES
ARCHIBALD
ALBERS
WOODCOCK
TRAPP
SOARES
SEATON
RICHIE
MONSON
LUCKETT
LINDBERG
KOPP
KEETON
HSU
HEALEY
GARVEY
GADDY
FAIN
BURCHFIELD
BADGER
WENTWORTH
STRAND
STACK
SPOONER
SAUCIER
SALES
RUBY
RICCI
PLUNKETT
PANNELL
NESS
LEGER
HOY
FREITAS
FONG
ELIZONDO
DUVAL
CHUN
CALVIN
BEAUDOIN
URBINA
STOCK
RICKARD
PARTIN
MOE
MCGREW
MCCLINTOCK
LEDOUX
FORSYTH
FAISON
DEVRIES
BERTRAND
WASSON
TILTON
SCARBROUGH
PRIDE
OH
LEUNG
LARRY
IRVINE
GARBER
DENNING
CORRAL
COLLEY
CASTLEBERRY
BOWLIN
BOGAN
BEALE
BAINES
TRUE
TRICE
RAYBURN
PARKINSON
PAK
NUNES
MCMILLEN
LEAHY
LEA
KIMMEL
HIGGS
FULMER
CARDEN
BEDFORD
TAGGART
SPEARMAN
REGISTER
PRICHARD
MORRILL
KOONCE
HEINZ
HEDGES
GUENTHER
GRICE
FINDLEY
EARLE
DOVER
CREIGHTON
BOOTHE
BAYER
ARREOLA
VITALE
VALLES
SEE
RANEY
PETER
OSGOOD
LOWELL
HANLON
BURLEY
BOUNDS
WORDEN
WEATHERLY
VETTER
TANAKA
STILTNER
SELL
NEVAREZ
MOSBY
MONTERO
MELANCON
HARTER
HAMER
GOBLE
GLADDEN
GIST
GINN
AKIN
ZARAGOZA
TOWNS
TARVER
SAMMONS
ROYSTER
OREILLY
MUIR
MOREHEAD
LUSTER
KINGSLEY
KELSO
GRISHAM
GLYNN
BAUMANN
ALVES
YOUNT
TAMAYO
TAM
PATERSON
OATES
MENENDEZ
LONGO
HARGIS
GREENLEE
GILLEN
DESANTIS
CONOVER
BREEDLOVE
WAYNE
SUMPTER
SCHERER
RUPP
REICHERT
HEREDIA
FALLON
CREEL
COHN
CLEMMONS
CASAS
BICKFORD
BELTON
BACH
WILLIFORD
WHITCOMB
TENNANT
SUTTER
STULL
SESSIONS
MCCALLUM
MANSON
LANGLOIS
KEEL
KEEGAN
EMANUEL
DANGELO
DANCY
DAMRON
CLAPP
CLANTON
BANKSTON
TRINIDAD
OLIVEIRA
MINTZ
MCINNIS
MARTENS
MABE
LASTER
JOLLEY
IRISH
HILDRETH
HEFNER
GLASER
DUCKETT
DEMERS
BROCKMAN
BLAIS
BACK
ALCORN
AGNEW
TOLIVER
TICE
SONG
SEELEY
NAJERA
MUSSER
MCFALL
LAPLANTE
GALVIN
FAJARDO
DOAN
COYNE
COPLEY
CLAWSON
CHEUNG
BARONE
WYNNE
WOODLEY
TREMBLAY
STOLL
SPARROW
SPARKMAN
SCHWEITZER
SASSER
SAMPLES
RONEY
RAMON
LEGG
LAI
JOE
HEIM
FARIAS
CONCEPCION
COLWELL
CHRISTMAN
BRATCHER
ALBA
WINCHESTER
UPSHAW
SOUTHERLAND
SORRELL
SHAY
SELLS
MOUNT
MCCLOSKEY
MARTINDALE
LUTTRELL
LOVELESS
LOVEJOY
LINARES
LATIMER
HOLLY
EMBRY
COOMBS
BRATTON
BOSTICK
BOSS
VENABLE
TUGGLE
TORO
STAGGS
SANDLIN
JEFFERIES
HECKMAN
GRIFFIS
CRAYTON
CLEM
BUTTON
BROWDER
ALLAN
THORTON
STURGILL
SPROUSE
ROYER
ROUSSEAU
RIDENOUR
POGUE
PERALES
PEEPLES
METZLER
MESA
MCCUTCHEON
MCBEE
JAY
HORNSBY
HEFFNER
CORRIGAN
ARMIJO
VUE
ROMEO
PLANTE
PEYTON
PAREDES
MACKLIN
HUSSEY
HODGSON
GRANADOS
FRIAS
CARMAN
BRENT
BECNEL
BATTEN
ALMANZA
TURNEY
TEAL
STURGEON
MEEKER
MCDANIELS
LIMON
KEENEY
KEE
HUTTO
HOLGUIN
GORHAM
FISHMAN
FIERRO
BLANCHETTE
RODRIGUE
REDDY
OSBURN
ODEN
LERMA
KIRKWOOD
KEEFER
HAUGEN
HAMMETT
CHALMERS
CARLOS
BRINKMAN
BAUMGARTNER
ZHANG
VALERIO
TELLEZ
STEFFEN
SHUMATE
SAULS
RIPLEY
KEMPER
JACKS
GUFFEY
EVERS
CRADDOCK
CARVALHO
BLAYLOCK
BANUELOS
BALDERAS
WOODEN
WHEATON
TURNBULL
SHUMAN
POINTER
MOSIER
MCCUE
LIGON
KOZLOWSKI
JOHANSEN
INGLE
HERR
BRIONES
SOUTHERN
SNIPES
RICKMAN
PIPKIN
PEACE
PANTOJA
OROSCO
MONIZ
LAWLESS
KUNKEL
HIBBARD
GALARZA
ENOS
BUSSEY
SETTLE
SCHOTT
SALCIDO
PERREAULT
MCDOUGAL
MCCOOL
HAIGHT
GARRIS
FERRY
EASTON
CONYERS
ATHERTON
WIMBERLY
UTLEY
STEPHEN
SPELLMAN
SMITHSON
SLAGLE
SKIPPER
RITCHEY
RAND
PETIT
OSULLIVAN
OAKS
NUTT
MCVAY
MCCREARY
MAYHEW
KNOLL
JEWETT
HARWOOD
HAILEY
CARDOZA
ASHE
ARRIAGA
ANDRES
ZELLER
WIRTH
WHITMIRE
STAUFFER
SPRING
ROUNTREE
REDDEN
MCCAFFREY
MARTZ
LOVING
LAROSE
LANGDON
HUMES
GASKIN
FABER
DOLL
DEVITO
CASS
ALMOND
WINGFIELD
WINGATE
VILLAREAL
TYNER
SMOTHERS
SEVERSON
RENO
PENNELL
MAUPIN
LEIGHTON
JANSSEN
HASSELL
HALLMAN
HALCOMB
FOLSE
FITZSIMMONS
FAHEY
CRANFORD
BOLEN
BATTLES
BATTAGLIA
WOOLDRIDGE
WEED
TRASK
ROSSER
REGALADO
MCEWEN
KEEFE
FUQUA
ECHEVARRIA
DOMINGO
DANG
CARO
BOYNTON
ANDRUS
WILD
VIERA
VANMETER
TABER
SPRADLIN
SEIBERT
PROVOST
PRENTICE
OLIPHANT
LAPORTE
HWANG
HATCHETT
HASS
GREINER
FREEDMAN
COVERT
CHILTON
BYARS
WIESE
VENEGAS
SWANK
SHRADER
RODERICK
ROBERGE
MULLIS
MORTENSEN
MCCUNE
MARLOWE
KIRCHNER
KECK
ISAACSON
HOSTETLER
HALVERSON
GUNTHER
GRISWOLD
GERARD
FENNER
DURDEN
BLACKWOOD
BERTRAM
AHRENS
SAWYERS
SAVOY
NABORS
MCSWAIN
MACKAY
LOY
LAVENDER
LASH
LABBE
JESSUP
HUBERT
FULLERTON
DONNELL
CRUSE
CRITTENDEN
CORREIA
CENTENO
CAUDLE
CANADY
CALLENDER
ALARCON
AHERN
WINFREY
TRIBBLE
TOM
STYLES
SALLEY
RODEN
MUSGROVE
MINNICK
FORTENBERRY
CARRION
BUNTING
BETHEL
BATISTE
WOO
WHITED
UNDERHILL
STILLWELL
SILVIA
RAUCH
PIPPIN
PERRIN
MESSENGER
MANCINI
LISTER
KINARD
HARTMANN
FLECK
BROADWAY
WILT
TREADWAY
THORNHILL
SPEED
SPALDING
SAM
RAFFERTY
PITRE
PATINO
ORDONEZ
LINKOUS
KELLEHER
HOMAN
HOLIDAY
GALBRAITH
FEENEY
DORRIS
CURTIN
COWARD
CAMARILLO
BUSS
BUNNELL
BOLT
BEELER
AUTRY
ALCALA
WITTE
WENTZ
STIDHAM
SHIVELY
NUNLEY
MEACHAM
MARTINS
LEMKE
LEFEBVRE
KAYE
HYNES
HOROWITZ
HOPPE
HOLCOMBE
ESTRELLA
DUNNE
DERR
COCHRANE
BRITTAIN
BEDARD
BEAUREGARD
TORRENCE
STRUNK
SORIA
SIMONSON
SHUMAKER
SCOGGINS
PACKER
OCONNER
MORIARTY
LEROY
KUNTZ
IVES
HUTCHESON
HORAN
HALES
GARMON
FITTS
DELL
BOHN
ATCHISON
WORTH
WISNIEWSKI
WILL
VANWINKLE
STURM
SALLEE
PROSSER
MOEN
LUNDBERG
KUNZ
KOHL
KEANE
JORGENSON
JAYNES
FUNDERBURK
FREED
FRAME
DURR
CREAMER
COSGROVE
CANDELARIA
BERLIN
BATSON
VANHOOSE
THOMSEN
TEETER
SOMMER
SMYTH
SENA
REDMON
ORELLANA
MANESS
LENNON
HEFLIN
GOULET
FRICK
FORNEY
DOLLAR
BUNKER
ASBURY
AGUIAR
TALBOTT
SOUTHARD
PLEASANT
MOWERY
MEARS
LEMMON
KRIEGER
HICKSON
GRACIA
ELSTON
DUONG
DELGADILLO
DAYTON
DASILVA
CONAWAY
CATRON
BRUTON
BRADBURY
BORDELON
BIVINS
BITTNER
BERGSTROM
BEALS
ABELL
WHELAN
TRAVERS
TEJADA
PULLEY
PINO
NORFLEET
NEALY
MAES
LOPER
HELD
GERALD
GATEWOOD
FRIERSON
FREUND
FINNEGAN
CUPP
COVEY
CATALANO
BOEHM
BADER
YOON
WALSTON
TENNEY
SIPES
ROLLER
RAWLINS
MEDLOCK
MCCASKILL
MCCALLISTER
MARCOTTE
MACLEAN
HUGHEY
HENKE
HARWELL
GLADNEY
GILSON
DEW
CHISM
CASKEY
BRANDENBURG
BAYLOR
VILLASENOR
VEAL
VAN
THATCHER
STEGALL
SHORE
PETRIE
NOWLIN
NAVARRETE
MUHAMMAD
LOMBARD
LOFTIN
LEMASTER
KROLL
KOVACH
KIMBRELL
KIDWELL
HERSHBERGER
FULCHER
ENG
CANTWELL
BUSTOS
BOLAND
BOBBITT
BINKLEY
WESTER
WEIS
VERDIN
TONG
TILLER
SISCO
SHARKEY
SEYMORE
ROSENBAUM
ROHR
QUINONEZ
PINKSTON
NATION
MALLEY
LOGUE
LESSARD
LERNER
LEBRON
KRAUSS
KLINGER
HALSTEAD
HALLER
GETZ
BURROW
BRANT
ALGER
VICTOR
SHORES
SCULLY
POUNDS
PFEIFER
PERRON
NELMS
MUNN
MCMASTER
MCKENNEY
MANNS
KNUDSON
HUTCHENS
HUSKEY
GOEBEL
FLAGG
CUSHMAN
CLICK
CASTELLANO
CARDER
BUMGARNER
BLAINE
BIBLE
WAMPLER
SPINKS
ROBSON
NEEL
MCREYNOLDS
MATHIAS
MAAS
LOERA
KASPER
JOSE
JENSON
FLOREZ
COONS
BUCKINGHAM
BROGAN
BERRYMAN
WILMOTH
WILHITE
THRASH
SHEPHARD
SEIDEL
SCHULZE
ROLDAN
PETTIS
OBRYAN
MAKI
MACKIE
HATLEY
FRAZER
FIORE
FALLS
CHESSER
BUI
BOTTOMS
BISSON
BENEFIELD
ALLMAN
WILKE
TRUDEAU
TIMM
SHIFFLETT
RAU
MUNDY
MILLIKEN
MAYERS
LEAKE
KOHN
HUNTINGTON
HORSLEY
HERMANN
GUERIN
FRYER
FRIZZELL
FORET
FLEMMING
FIFE
CRISWELL
CARBAJAL
BOZEMAN
BOISVERT
ARCHIE
ANTONIO
ANGULO
WALLEN
TAPP
SILVERS
RAMSAY
OSHEA
ORTA
MOLL
MCKEEVER
MCGEHEE
LUCIANO
LINVILLE
KIEFER
KETCHUM
HOWERTON
GROCE
GAYLORD
GASS
FUSCO
CORBITT
BLYTHE
BETZ
BARTELS
AMARAL
AIELLO
YOO
WEDDLE
TROY
SUN
SPERRY
SEILER
RUNYAN
RALEY
OVERBY
OSTEEN
OLDS
MCKEOWN
MAURO
MATNEY
LAUER
LATTIMORE
HINDMAN
HARTWELL
FREDRICKSON
FREDERICKS
ESPINO
CLEGG
CARSWELL
CAMBELL
BURKHOLDER
AUGUST
WOODBURY
WELKER
TOTTEN
THORNBURG
THERIAULT
STITT
STAMM
STACKHOUSE
SIMONE
SCHOLL
SAXON
RIFE
RAZO
QUINLAN
PINKERTON
OLIVO
NESMITH
NALL
MATTOS
LEAK
LAFFERTY
JUSTUS
GIRON
GEER
FIELDER
EAGLE
DRAYTON
DORTCH
CONNERS
CONGER
CHAU
BOATWRIGHT
BILLIOT
BARDEN
ARMENTA
ANTOINE
TIBBETTS
STEADMAN
SLATTERY
SIDES
RINALDI
RAYNOR
RAYFORD
PINCKNEY
PETTIGREW
NICKEL
MILNE
MATTESON
HALSEY
GONSALVES
FELLOWS
DURAND
DESIMONE
COWLEY
COWLES
BRILL
BARHAM
BARELA
BARBA
ASHMORE
WITHROW
VALENTI
TEJEDA
SPRIGGS
SAYRE
SALERNO
PLACE
PELTIER
PEEL
MERRIMAN
MATHESON
LOWMAN
LINDSTROM
HYLAND
HOMER
HA
GIROUX
FRIES
FRASIER
EARLS
DUGAS
DAMON
DABNEY
COLLADO
BRISENO
BAXLEY
ANDRE
WORD
WHYTE
WENGER
VANOVER
VANBUREN
THIEL
SCHINDLER
SCHILLER
RIGBY
POMEROY
PASSMORE
MARBLE
MANZO
MAHAFFEY
LINDGREN
LAFLAMME
GREATHOUSE
FITE
FERRARI
CALABRESE
BAYNE
YAMAMOTO
WICK
TOWNES
THAMES
STEEL
REINHART
PEELER
NARANJO
MONTEZ
MCDADE
MAST
MARKLEY
MARCHAND
LEEPER
KONG
KELLUM
HUDGENS
HENNESSEY
HADDEN
GUESS
GAINEY
COPPOLA
BORREGO
BOLLING
BEANE
AULT
SLATON
POLAND
PAPE
NULL
MULKEY
LIGHTNER
LANGER
HILLARD
GLASGOW
FABIAN
ETHRIDGE
ENRIGHT
DEROSA
BASKIN
ALFRED
WEINBERG
TURMAN
TINKER
SOMERVILLE
PARDO
NOLL
LASHLEY
INGRAHAM
HILLER
HENDON
GLAZE
FLORA
COTHRAN
COOKSEY
CONTE
CARRICO
APPLE
ABNER
WOOLEY
SWOPE
SUMMERLIN
STURGIS
STURDIVANT
STOTT
SPURGEON
SPILLMAN
SPEIGHT
ROUSSEL
POPP
NUTTER
MCKEON
MAZZA
MAGNUSON
LANNING
KOZAK
JANKOWSKI
HEYWARD
FORSTER
CORWIN
CALLAGHAN
BAYS
WORTHAM
USHER
THERIOT
SAYERS
SABO
RUPERT
POLING
NATHAN
LOYA
LIEBERMAN
LEVI
LAROCHE
LABELLE
HOWES
HARR
GARAY
FOGARTY
EVERSON
DURKIN
DOMINQUEZ
CHAVES
CHAMBLISS
ALFONSO
WITCHER
WILBER
VIEIRA
VANDIVER
TERRILL
STOKER
SCHREINER
NESTOR
MOORMAN
LIDDELL
LEW
LAWHORN
KRUG
IRONS
HYLTON
HOLLENBECK
HERRIN
HEMBREE
HAIR
GOOLSBY
GOODIN
GILMER
FOLTZ
DINKINS
DAUGHTRY
CABAN
BRIM
BRILEY
BILODEAU
BEAR
WYANT
VERGARA
TALLENT
SWEARINGEN
STROUP
SHERRY
SCRIBNER
ROGER
QUILLEN
PITMAN
MONACO
MCCANTS
MAXFIELD
MARTINSON
LANDON
HOLTZ
FLOURNOY
BROOKINS
BRODY
BAUMGARDNER
ANGELO
STRAUB
SILLS
ROYBAL
ROUNDTREE
OSWALT
MONEY
MCGRIFF
MCDOUGALL
MCCLEARY
MAGGARD
GRAGG
GOODING
GODINEZ
DOOLITTLE
DONATO
COWELL
CASSELL
BRACKEN
APPEL
AHMAD
ZAMBRANO
REUTER
PEREA
OLIVE
NAKAMURA
MONAGHAN
MICKENS
MCCLINTON
MCCLARY
MARLER
KISH
JUDKINS
GILBREATH
FREESE
FLANIGAN
FELTS
ERDMANN
DODDS
CHEW
BROWNELL
BRAZIL
BOATRIGHT
BARRETO
SLAYTON
SANDBERG
SALDIVAR
PETTWAY
ODUM
NARVAEZ
MOULTRIE
MONTEMAYOR
MERRELL
LEES
KEYSER
HOKE
HARDAWAY
HANNAN
GILBERTSON
FOGG
DUMONT
DEBERRY
COGGINS
CARRERA
BUXTON
BUCHER
BROADNAX
BEESON
ARAUJO
APPLETON
AMUNDSON
AGUAYO
ACKLEY
YOCUM
WORSHAM
SHIVERS
SHELLY
SANCHES
SACCO
ROBEY
RHODEN
PENDER
OCHS
MCCURRY
MADERA
LUONG
LUIS
KNOTTS
JACKMAN
HEINRICH
HARGRAVE
GAULT
FOREST
COMEAUX
CHITWOOD
CHILD
CARAWAY
BOETTCHER
BERNHARDT
BARRIENTOS
ZINK
WICKHAM
WHITEMAN
THORP
STILLMAN
SETTLES
SCHOONOVER
ROQUE
RIDDELL
REY
PILCHER
PHIFER
NOVOTNY
MAPLE
MACLEOD
HARDEE
HAASE
GRIDER
FREDRICK
EARNEST
DOUCETTE
CLAUSEN
CHRISTMAS
BEVINS
BEAMON
BADILLO
TOLLEY
TINDALL
SOULE
SNOOK
SEBASTIAN
SEALE
PITCHER
PINKNEY
PELLEGRINO
NOWELL
NEMETH
NAIL
MONDRAGON
MCLANE
LUNDGREN
INGALLS
HUDSPETH
HIXSON
GEARHART
FURLONG
DOWNES
DIONNE
DIBBLE
DEYOUNG
CORNEJO
CAMARA
BROOKSHIRE
BOYETTE
WOLCOTT
TRACEY
SURRATT
SELLARS
SEGAL
SALYER
REEVE
RAUSCH
PHILIPS
LABONTE
HARO
GOWER
FREELAND
FAWCETT
EADS
DRIGGERS
DONLEY
COLLETT
CAGE
BROMLEY
BOATMAN
BALLINGER
BALDRIDGE
VOLZ
TROMBLEY
STONGE
SILAS
SHANAHAN
RIVARD
RHYNE
PEDROZA
MATIAS
MALLARD
JAMIESON
HEDGEPETH
HARTNETT
ESTEVEZ
ESKRIDGE
DENMAN
CHIU
CHINN
CATLETT
CARMACK
BUIE
BOOK
BECHTEL
BEARDSLEY
BARD
BALLOU
WINDSOR
ULMER
STORM
SKEEN
ROBLEDO
RINCON
REITZ
PIAZZA
PEARL
MUNGER
MOTEN
MCMICHAEL
LOFTUS
LEDET
KERSEY
GROFF
FOWLKES
FOLK
CRUMPTON
COLLETTE
CLOUSE
BETTIS
VILLAGOMEZ
TIMMERMAN
STROM
SAUL
SANTORO
RODDY
PHILLIP
PENROD
MUSSELMAN
MACPHERSON
LEBOEUF
HARLESS
HADDAD
GUIDO
GOLDING
FULKERSON
FANNIN
DULANEY
DOWDELL
DEANE
COTTLE
CEJA
CATE
BOSLEY
BENGE
ALBRITTON
VOIGT
TROWBRIDGE
SOILEAU
SEELY
ROME
ROHDE
PEARSALL
PAULK
ORTH
NASON
MOTA
MCMULLIN
MARQUARDT
MADIGAN
HOAG
GILLUM
GAYLE
GABBARD
FENWICK
FENDER
ECK
DANFORTH
CUSHING
CRESS
CREED
CAZARES
CASANOVA
BEY
BETTENCOURT
BARRINGER
BABER
STANSBERRY
SCHRAMM
RUTTER
RIVERO
RACE
OQUENDO
NECAISE
MOUTON
MONTENEGRO
MILEY
MCGOUGH
MARRA
MACMILLAN
LOCK
LAMONTAGNE
JASSO
JAIME
HORST
HETRICK
HEILMAN
GAYTAN
GALL
FRIED
FORTNEY
EDEN
DINGLE
DESJARDINS
DABBS
BURBANK
BRIGHAM
BRELAND
BEAMAN
BANNER
ARRIOLA
YARBOROUGH
WALLIN
TREAT
TOSCANO
STOWERS
REISS
PICHARDO
ORTON
MITCHEL
MICHELS
MCNAMEE
MCCRORY
LEATHERMAN
KELL
KEISTER
JEROME
HORNING
HARGETT
GUAY
FRIDAY
FERRO
DEBOER
DAGOSTINO
CLEMENTE
CHRIST
CARPER
BOWLER
BLANKS
BEAUDRY
WILLIE
TOWLE
TAFOYA
STRICKLIN
STRADER
SOPER
SONNIER
SIGMON
SCHENK
SADDLER
RODMAN
PEDIGO
MENDES
LUNN
LOHR
LAHR
KINGSBURY
JARMAN
HUME
HOLLIMAN
HOFMANN
HAWORTH
HARRELSON
HAMBRICK
FLICK
EDMUNDS
DACOSTA
CROSSMAN
COLSTON
CHAPLIN
CARRELL
BUDD
WEILER
WAITS
VIOLA
VALENTINO
TRANTHAM
TARR
STRAIGHT
SOLORIO
ROEBUCK
POWE
PLANK
PETTUS
PALM
PAGANO
MINK
LUKER
LEATHERS
JOSLIN
HARTZELL
GAMBRELL
FEARS
DEUTSCH
CEPEDA
CARTY
CAPUTO
BREWINGTON
BEDELL
BALLEW
APPLEWHITE
WARNOCK
WALZ
URENA
TUDOR
REEL
PIGG
PARTON
MICKELSON
MEAGHER
MCLELLAN
MCCULLEY
MANDEL
LEECH
LAVALLEE
KRAEMER
KLING
KIPP
KINGSTON
KEHOE
HOCHSTETLER
HARRIMAN
GREGOIRE
GRABOWSKI
GOSSELIN
GAMMON
FANCHER
EDENS
DESAI
BUTT
BRANNAN
ARMENDARIZ
WOOLSEY
WHITEHOUSE
WHETSTONE
USSERY
TOWNE
TOWER
TESTA
TALLMAN
STUDER
STRAIT
STEINMETZ
SORRELLS
SAUCEDA
ROLFE
RAE
PADDOCK
MITCHEM
MCGINN
MCCREA
LUCK
LOVATO
LING
HAZEN
GILPIN
GAYNOR
FIKE
DEVOE
DELRIO
CURIEL
BURKHARDT
BRISTOL
BODE
BACKUS
ALTON
ZINN
WATANABE
WACHTER
VANPELT
TURNAGE
SHANER
SCHRODER
SATO
RIORDAN
QUIMBY
PORTIS
NATALE
MCKOY
MCCOWN
MARKER
LUCIO
KILMER
KARL
HOTCHKISS
HESSE
HALBERT
GWINN
GODSEY
DESMOND
DELISLE
CHRISMAN
CANTER
BROOK
ARBOGAST
ANGELL
ACREE
YANCY
WOOLLEY
WESSON
WEATHERSPOON
TRAINOR
STOCKMAN
SPILLER
SIPE
ROOKS
REAVIS
PROPST
PORRAS
NEILSON
MULLENS
LOUCKS
LLEWELLYN
LAMONT
KUMAR
KOESTER
KLINGENSMITH
KIRSCH
KESTER
HONAKER
HODSON
HENNESSY
HELMICK
GARRITY
GARIBAY
FEE
DRAIN
CASAREZ
CALLIS
BOTELLO
BAY
AYCOCK
AVANT
ANGLE
WINGARD
WAYMAN
TULLY
THEISEN
SZYMANSKI
STANSBURY
SEGOVIA
RUDY
RAINWATER
PREECE
PIRTLE
PADRON
MINCEY
MCKELVEY
MATHES
MARTY
LARRABEE
KORNEGAY
KLUG
JUDY
INGERSOLL
HECHT
GERMAIN
EGGERS
DYKSTRA
DENIS
DEERING
DECOTEAU
DEASON
DEARING
COFIELD
CARRIGAN
BRUSH
BONHAM
BAHR
AUCOIN
APPLEBY
ALMONTE
YAGER
WOMBLE
WIMMER
WEIMER
VANDERPOOL
STANCIL
SPRINKLE
ROMINE
REMINGTON
PFAFF
PECKHAM
OLIVERA
MERAZ
MAZE
LATHROP
KOEHN
JONAS
HAZELTON
HALVORSON
HALLOCK
HADDOCK
DUCHARME
DEHAVEN
COLTON
CARUTHERS
BREHM
BOSWORTH
BOST
BLOW
BIAS
BEEMAN
BASILE
BANE
AIKENS
ZACHARY
WOLD
WALTHER
TABB
SUBER
STRAWN
STOCKS
STOCKER
SHIREY
SCHLOSSER
SALVADOR
RIEDEL
REMBERT
REIMER
PYLES
PICKLE
PEELE
MERRIWEATHER
LETOURNEAU
LATTA
KIDDER
HIXON
HILLIS
HIGHT
HERBST
HENRIQUEZ
HAYGOOD
HAMILL
GABEL
FRITTS
EUBANK
DUTY
DAWES
CORRELL
COFFEE
CHA
BUSHEY
BUCHHOLZ
BROTHERTON
BRIDGE
BOTTS
BARNWELL
AUGER
ATCHLEY
WESTPHAL
VEILLEUX
ULLOA
TRUMAN
STUTZMAN
SHRIVER
RYALS
PRIOR
PILKINGTON
NEWPORT
MOYERS
MIRACLE
MARRS
MANGRUM
MADDUX
LOCKARD
LAING
KUHL
HARNEY
HAMMOCK
HAMLETT
FELKER
DOERR
DEPRIEST
CARRASQUILLO
CAROTHERS
BOGLE
BLOOD
BISCHOFF
BERGEN
ALBANESE
WYCKOFF
VERMILLION
VANSICKLE
THIBAULT
TETREAULT
STICKNEY
SHOEMAKE
RUGGIERO
RAWSON
RACINE
PHILPOT
PASCHAL
MCELHANEY
MATHISON
LEGRAND
LAPIERRE
KWAN
KREMER
JILES
HILBERT
GEYER
FAIRCLOTH
EHLERS
EGBERT
DESROSIERS
DALRYMPLE
COTTEN
CASHMAN
CADENA
BREEDING
BOARDMAN
ALCARAZ
AHN
WYRICK
THERRIEN
TANKERSLEY
STRICKLER
PURYEAR
PLOURDE
PATTISON
PARDUE
MILAN
MCGINTY
MCEVOY
LANDRETH
KUHNS
KOON
HEWETT
GIDDENS
EVERETTE
EMERICK
EADES
DEANGELIS
COSME
CEBALLOS
BIRDSONG
BENHAM
BEMIS
ARMOUR
ANGUIANO
ANGELES
WELBORN
TSOSIE
STORMS
SHOUP
SESSOMS
SAMANIEGO
ROOD
ROJO
RHINEHART
RABY
NORTHCUTT
MYER
MUNGUIA
MOREHOUSE
MORE
MCDEVITT
MATEO
MALLETT
LOZADA
LEMOINE
KUEHN
HALLETT
GRIM
GILLARD
GAYLOR
GARMAN
GALLAHER
FEASTER
FARIS
DARROW
DARDAR
CONEY
CARREON
BYRON
BRAITHWAITE
BOYLAN
BOYETT
BORN
BIXLER
BIGHAM
BENFORD
BARRAGAN
BARNUM
ZUBER
WYCHE
WESTCOTT
VINING
STOLTZFUS
SIMONDS
SHUPE
SABIN
RUBLE
RITTENHOUSE
RICHMAN
PERRONE
MULHOLLAND
MILLAN
MEISTER
MATHEW
LOMELI
KITE
JEMISON
HULETT
HOLLER
HICKERSON
HEROLD
HAZELWOOD
GRIFFEN
GAUSE
FORDE
EISENBERG
DILWORTH
CHARRON
CHAISSON
BRODIE
BRISTOW
BREUNIG
BRACE
BOUTWELL
BENTZ
BELK
BAYLESS
BATCHELDER
BARAN
BAEZA
ZIMMERMANN
WEATHERSBY
VOLK
TOOLE
THEIS
TEDESCO
SHINE
SEARLE
SCHENCK
SATTERWHITE
SANDY
RUELAS
ROYCE
RANKINS
PARTIDA
NESBIT
MOREL
MENCHACA
LEVASSEUR
KAYLOR
JOHNSTONE
HULSE
HOLLAR
HERSEY
HARRIGAN
HARBISON
GUYER
GISH
GIESE
GERLACH
GELLER
GEISLER
FALCONE
ERNEST
ELWELL
DOUCET
DEESE
DARR
CORDER
CHAFIN
BYLER
BUSSELL
BURDETT
BRASHER
BOWE
BELLINGER
BASTIAN
BARNER
ALLEYNE
WILBORN
WEIL
WEGNER
WALES
TATRO
SPITZER
SMITHERS
SCHOEN
RESENDEZ
PETE
PARISI
OVERMAN
OBRIAN
MUDD
MOY
MCLAREN
MAHLER
MAGGIO
LINDNER
LALONDE
LACASSE
LABOY
KILLION
KAHL
JESSEN
JAMERSON
HOUK
HENSHAW
GUSTIN
GROOM
GRABER
DURST
DUENAS
DAVEY
CUNDIFF
CONLON
COLUNGA
COAKLEY
CHILES
CAPERS
BUELL
BRICKER
BISSONNETTE
BIRMINGHAM
BARTZ
BAGBY
ZAYAS
VOLPE
TREECE
TOOMBS
THOM
TERRAZAS
SWINNEY
SKILES
SILVEIRA
SHOUSE
SENN
RAMBO
RAMAGE
NEZ
MOUA
MARLIN
MALIK
LANGHAM
KYLES
HOLSTON
HOAGLAND
HERD
HECTOR
FELLER
EMORY
DENISON
CORLISS
CARRAWAY
BURFORD
BICKEL
AMBRIZ
ABERCROMBIE
YAMADA
WINNER
WEIDNER
WADDLE
VERDUZCO
THURMOND
SWINDLE
SCHROCK
SANABRIA
ROSENBERGER
PROBST
PEABODY
OLINGER
NEIGHBORS
NAZARIO
MCCAFFERTY
MCBROOM
MCABEE
MAZUR
MATHERNE
MAPES
LEVERETT
KILLINGSWORTH
HEISLER
GRIEGO
GRANDE
GOSNELL
FRANKEL
FRANKE
FERRANTE
FENN
ELMER
EHRLICH
CHRISTOPHERSO
CHICK
CHASSE
CHANCELLOR
CATON
BRUNELLE
BLY
BLOOMFIELD
BABBITT
AZEVEDO
ABRAMSON
ABLES
ABEYTA
YOUMANS
WOZNIAK
WAINWRIGHT
SUMMER
STOWELL
SMITHERMAN
SITES
SAMUELSON
RUNGE
RULE
ROTHMAN
ROSENFELD
QUAN
PEAKE
OXFORD
OWINGS
OLMOS
MUNRO
MOREIRA
LEATHERWOOD
LARKINS
KRANTZ
KOVACS
KIZER
KINDRED
KARNES
JAFFE
HUBBELL
HOSEY
HAUCK
HAROLD
GOODELL
FAVORS
ERDMAN
DVORAK
DOANE
CURETON
COFER
BUEHLER
BIERMAN
BERNDT
BANTA
ANNIS
ABRAM
ABDULLAH
WARWICK
WALTZ
TURCOTTE
TRINH
TORREY
STITH
SEGER
SACHS
QUESADA
PINDER
PEPPERS
PASCUAL
PASCHALL
PARKHURST
OZUNA
OSTER
NICHOLLS
MORTIMER
LHEUREUX
LAVALLEY
KIMURA
JABLONSKI
HAUN
GOURLEY
GILLIGAN
FIX
DERBY
CROY
COTTO
CARGILL
BURWELL
BURGETT
BUCKMAN
BRETT
BOOHER
ADORNO
WRENN
WHITTEMORE
URIAS
SZABO
SAYLES
SAIZ
RUTLAND
RAEL
PLANT
PHARR
PENNEY
PELKEY
OGRADY
NICKELL
MUSICK
MOATS
MATHER
MASSA
LAURENT
KIRSCHNER
KIEFFER
KELLAR
HENDERSHOT
GOTT
GODOY
GADSON
FURTADO
FIEDLER
ERSKINE
EDISON
DUTCHER
DEVER
DAGGETT
CHEVALIER
CHAO
BRAKE
BALLESTEROS
AMERSON
ALEJANDRO
WINGO
WALDON
TROTT
SPIKES
SILVEY
SHOWERS
SCHLEGEL
RUE
RITZ
PEPIN
PELAYO
PARSLEY
PALERMO
MOOREHEAD
MCHALE
LETT
KOCHER
KILBURN
IGLESIAS
HUMBLE
HULBERT
HUCKABY
HIX
HAVEN
HARTFORD
HARDIMAN
GURNEY
GRIGG
GRASSO
GOINGS
FILLMORE
FARBER
DEPEW
DANDREA
DAME
COWEN
COVARRUBIAS
CORY
BURRUS
BRACY
ARDOIN
THOMPKINS
SUZUKI
STANDLEY
RUSSEL
RADCLIFFE
POHL
PERSAUD
PERCY
PARENTEAU
PABON
NEWSON
NEWHOUSE
NAPOLITANO
MULCAHY
MAYA
MALAVE
KEIM
HOOTEN
HERNANDES
HEFFERNAN
HEARNE
GREENLEAF
GLICK
FUHRMAN
FETTER
FARIA
DISHMAN
DICKENSON
CRITES
CRISS
CLAPPER
CHENAULT
CASTOR
CASTO
BUGG
BOVE
BONNEY
BLESSING
ARD
ANDERTON
ALLGOOD
ALDERSON
WOODMAN
WISDOM
WARRICK
TOOMEY
TOOLEY
TARRANT
SUMMERVILLE
STEBBINS
SOKOL
SINK
SEARLES
SCHUTZ
SCHUMANN
SCHEER
REMILLARD
RAPER
PROULX
PALMORE
MONROY
MIGUEL
MESSIER
MELO
MELANSON
MASHBURN
MANZANO
LUSSIER
LOVELY
LIEN
JENKS
HUNEYCUTT
HARTWIG
GRIMSLEY
FULK
FIELDING
FIDLER
ENGSTROM
ELDRED
DANTZLER
CRANDELL
CHING
CALDER
BRUMLEY
BRETON
BRANN
BRAMLETT
BOYKINS
BIANCO
BANCROFT
ALMARAZ
ALCANTAR
WHITMER
WHITENER
WELTON
VINEYARD
SU
RAHN
PAQUIN
MIZELL
MIX
MCMILLIN
MCKEAN
MARSTON
MACIEL
LUNDQUIST
LOUIE
LIGGINS
LAMPKIN
KRANZ
KOSKI
KIRKHAM
JIMINEZ
HAZZARD
HARROD
GRAZIANO
GRAMMER
GENDRON
GARRIDO
FORDHAM
ENGLERT
ELWOOD
DRYDEN
DEMOSS
DELUNA
CRABB
COMEAU
CLAUDIO
BRUMMETT
BLUME
BENALLY
WESSEL
VANBUSKIRK
THORSON
STUMPF
STOCKWELL
ROCCO
REAMS
RADTKE
RACKLEY
PELTON
NIEMI
NEWLAND
NELSEN
MORRISSETTE
MIRAMONTES
MCGINLEY
MCCLUSKEY
MARLEY
MARCHANT
LUEVANO
LAMPE
LAIL
JEFFCOAT
INFANTE
HU
HINMAN
GAONA
ERB
EADY
DESMARAIS
DECOSTA
DANSBY
CISCO
CHOE
BRECKENRIDGE
BOSTWICK
BORG
BIANCHI
BEER
ALBERTS
ADRIAN
WILKIE
WHORTON
VARGO
TAIT
SYLVIA
SOUCY
SCHUMAN
OUSLEY
MUMFORD
LUM
LIPPERT
LEATH
LAVERGNE
LALIBERTE
KIRKSEY
KENNER
JOHNSEN
IZZO
HILES
GULLETT
GREENWELL
GASPAR
GALBREATH
GAITAN
ERICSON
DUCK
DELAPAZ
CROOM
COTTINGHAM
CLIFT
BUSHNELL
BOOZER
BICE
BERNARDO
BEASON
ARROWOOD
WARING
VOORHEES
TRUAX
SHREVE
SHOCKEY
SCHATZ
SANDIFER
RUBINO
ROZIER
ROSEBERRY
ROLL
PLAYER
PIEPER
PEDEN
NESTER
NAVE
MURPHEY
MALINOWSKI
MACGREGOR
LIANG
LAFRANCE
KUNKLE
KIRKMAN
JORGE
HIPP
HASTY
HADDIX
GERVAIS
GERDES
GARFIELD
GAMACHE
FOUTS
FITZWATER
DILLINGHAM
DEMING
DEANDA
CEDENO
CANNADY
BURSON
BOULDIN
ARCENEAUX
WOODHOUSE
WHITFORD
WESCOTT
WELTY
WEIGEL
TORGERSON
TOMS
SURBER
SUNDERLAND
STERNER
SETZER
SALVATORE
RIOJAS
PUMPHREY
PUGA
PEDRO
PATCH
METTS
MCGARRY
MCCANDLESS
MAGILL
LUPO
LOVELAND
LLAMAS
LECLERC
KOONS
KAHLER
HUSS
HOLBERT
HEINTZ
HAUPT
GRIMMETT
GASKILL
FLOWER
ELLINGSON
DORR
DINGESS
DEWEESE
DESILVA
CROSSLEY
CORDEIRO
CONVERSE
CONDE
CHEEKS
CALDERA
CAIRNS
BURMEISTER
BURKHALTER
BRAWNER
BOTT
YOUNGS
VIERRA
VALLADARES
TIFFANY
SHRUM
SHROPSHIRE
SEVILLA
RUSK
ROOF
RODARTE
PEDRAZA
NINO
MONTANA
MERINO
MCMINN
MARKLE
MAPP
LUCIA
LAJOIE
KOERNER
KITTRELL
KATO
HYDER
HOLLIFIELD
HEISER
HAZLETT
GREENWALD
FANT
ELDREDGE
DREHER
DELAFUENTE
CRAVENS
CLAYPOOL
BEECHER
ARONSON
ALANIS
WORTHEN
WOJCIK
WINGER
WHITACRE
WELLINGTON
VALVERDE
VALDIVIA
TROUPE
THROWER
SWINDELL
SUTTLES
SUH
STROMAN
SPIRES
SLATE
SHEALY
SARVER
SARTIN
SADOWSKI
RONDEAU
ROLON
RICK
REX
RASCON
PRIDDY
PINE
PAULINO
NOLTE
MUNROE
MOLLOY
MELLON
MCIVER
LYKINS
LOGGINS
LILLIE
LENOIR
KLOTZ
KEMPF
JONE
HUPP
HOLLOWELL
HOLLANDER
HAYNIE
HASSAN
HARKNESS
HARKER
GOTTLIEB
FRITH
EDDINS
DRISKELL
DOGGETT
DENSMORE
CHARETTE
CASSADY
CARROL
BYRUM
BURCHAM
BUGGS
BENN
WHITTED
WARRINGTON
VANDUSEN
VAILLANCOURT
STEGER
SPELL
SIEBERT
SCOFIELD
QUIRK
PURSER
PLUMB
ORCUTT
NORTHERN
NORDSTROM
MOSELY
MICHALSKI
MCPHAIL
MCDAVID
MCCRAW
MARTINI
MARCHESE
MANNINO
LEO
LEFEVRE
LARGENT
LANZA
KRESS
ISHAM
HUNSAKER
HOCH
HILDEBRANDT
GUARINO
GRIJALVA
GRAYBILL
FICK
EWELL
EWALD
DEANGELO
CUSICK
CRUMLEY
COSTON
CATHCART
CARRUTHERS
BULLINGTON
BRIAN
BOWES
BLAIN
BLACKFORD
BARBOZA
YINGLING
WOODLAND
WERT
WEILAND
VARGA
SILVERSTEIN
SIEVERS
SHUSTER
SHUMWAY
SCUDDER
RUNNELS
RUMSEY
RENFROE
PROVENCHER
POLLEY
MOHLER
MIDDLEBROOKS
KUTZ
KOSTER
KORN
GROW
GROTH
GLIDDEN
FAZIO
DEEN
CORN
COPPER
CHIPMAN
CHENOWETH
CHAMPLIN
CEDILLO
CARRERO
CARMODY
BUCKLES
BRIEN
BOUTIN
BOSCH
BILL
BERKOWITZ
ALTAMIRANO
WILFONG
WIEGAND
WAITES
TRUESDALE
TOUSSAINT
TOBEY
TEDDER
STEELMAN
SIROIS
SCHNELL
ROBICHAUD
RIDGE
RICHBURG
PRAY
PLUMLEY
PIZARRO
PIERCY
ORTEGO
OBERG
NEACE
MUSIC
MICKEY
MERTZ
MCNEW
MATTA
LAWYER
LAPP
LAIR
KIBLER
JESSIE
HOWLETT
HOLLISTER
HOFER
HATTEN
HAGLER
GERMANY
FALGOUST
ENGELHARDT
EBERLE
EASTWOOD
DOMBROWSKI
DINSMORE
DAYE
COOL
CASARES
CAPONE
BRAUD
BALCH
AUTREY
WENDEL
TYNDALL
TOY
STROBEL
STOLTZ
SPINELLI
SERRATO
ROCHESTER
REBER
REAL
RATHBONE
PALOMINO
NOAH
NICKELS
MAYLE
MATHERS
MACH
LOEFFLER
LITTRELL
LEVINSON
LEONG
LEMIRE
LEJEUNE
LAZO
LASLEY
KOLLER
KENNARD
JESTER
HOELSCHER
HINTZ
HAGERMAN
GREAVES
FORE
EUDY
ENGLER
CORRALES
CORDES
BRUNET
BIDWELL
BENNET
BARE
TYRRELL
THARPE
SWINTON
STRIBLING
STEVEN
SOUTHWORTH
SISNEROS
SHANE
SAVOIE
SAMONS
RUVALCABA
ROSCOE
RIES
RAMER
OMARA
MOSQUEDA
MILLAR
MCPEAK
MACOMBER
LUCKEY
LITTON
LEHR
LAVIN
HUBBS
HOARD
HIBBS
HAGANS
FUTRELL
EXUM
EVENSON
DICKS
CULLER
CHOU
CARBAUGH
CALLEN
BRASHEAR
BLOOMER
BLAKENEY
BIGLER
ADDINGTON
WOODFORD
WITTER
UNRUH
TOLENTINO
SUMRALL
STGERMAIN
SMOCK
SHERER
SALEM
ROCHELLE
RAYNER
POOLER
OQUINN
NERO
MILANO
MCGLOTHLIN
MARS
LINDEN
KOWAL
KERRIGAN
IBRAHIM
HARVELL
HANRAHAN
GOODALL
GEIST
FUSSELL
FUNG
FEREBEE
FEDERICO
ELEY
EGGERT
DORSETT
DINGMAN
DESTEFANO
COLUCCI
CLEMMER
CAESAR
BURNELL
BRUMBAUGH
BODDIE
BERRYHILL
AVELAR
ALCANTARA
ABBEY
WINDER
WINCHELL
VANDENBERG
TROTMAN
THURBER
THIBEAULT
STLOUIS
STILWELL
SPERLING
SHATTUCK
SARMIENTO
RUPPERT
RUMPH
RENAUD
RANDAZZO
RADEMACHER
QUILES
PEARMAN
PALOMO
MERCURIO
LOWREY
LINDEMAN
LAWLOR
LAROSA
LANDER
LABRECQUE
KIMBER
HOVIS
HOLIFIELD
HENNINGER
HAWKES
HARTFIELD
HANN
HAGUE
GENOVESE
GARRICK
FUDGE
FRINK
EDDINGS
DINH
DEAR
CUTTER
CRIBBS
CONSTANT
CALVILLO
BUNTON
BRODEUR
BOLDING
BLANDING
AGOSTO
ZAHN
WIENER
TRUSSELL
TEW
TELLO
TEIXEIRA
STEPHAN
SPECK
SHARMA
SHANKLIN
SEALY
SCANLAN
SANTAMARIA
ROUNDY
ROBICHAUX
RINGER
RIGNEY
PREVOST
POLSON
PHILIP
PASS
NORD
MOXLEY
MOHAMMED
MEDFORD
MCCASLIN
MCARDLE
MACARTHUR
LEWIN
LASHER
KETCHAM
KEISER
HEINE
HACKWORTH
GROSE
GRIZZLE
GRASS
GILLMAN
GARTNER
GARTH
FRAZEE
FLEURY
FAST
EDSON
EDMONSON
DERRY
DECK
CRONK
CONANT
BURRESS
BURGIN
BROOM
BROCKINGTON
BOLICK
BOGER
BIRCHFIELD
BILLINGTON
BAILY
BAHENA
ARMBRUSTER
ANSON
YOHO
WILCHER
TINNEY
TIMBERLAKE
THOMA
THIELEN
SUTPHIN
STULTZ
SIKORA
SERRA
SCHULMAN
SCHEFFLER
SANTILLAN
ROBIN
REGO
PRECIADO
PINKHAM
MONDAY
MICKLE
LUU
LOMAS
LIZOTTE
LENT
LENARD
KELLERMAN
KEIL
JUAN
JOHANSON
HERNADEZ
HARTSFIELD
HANG
HABER
GORSKI
FARKAS
EBERHARDT
DUQUETTE
DELANO
CROPPER
COZART
COCKERHAM
CHAMBLEE
CARTAGENA
CAHOON
BUZZELL
BRISTER
BREWTON
BLACKSHEAR
BENFIELD
ASTON
ASHBURN
ARRUDA
WETMORE
WEISE
VACCARO
TUCCI
SUDDUTH
STROMBERG
STOOPS
SHOWALTER
SHEARS
RUNION
ROWDEN
ROSENBLUM
RIFFLE
RENFROW
PERES
OBRYANT
NICOLAS
LEFTWICH
LARK
LANDEROS
KISTLER
KILLOUGH
KERLEY
KASTNER
HOGGARD
HARTUNG
GUERTIN
GOVAN
GATLING
GAILEY
FULLMER
FULFORD
FLATT
ESQUIBEL
ENDICOTT
EDMISTON
EDELSTEIN
DUFRESNE
DRESSLER
DICKMAN
CHEE
BUSSE
BONNETT
BOGART
BERARD
BARRINGTON
ARENA
ANTON
YOSHIDA
VELARDE
VEACH
VANHOUTEN
VACHON
TOLSON
TOLMAN
TENNYSON
STITES
SOLER
SHUTT
RUGGLES
RHONE
PEGUES
ONG
NEESE
MURO
MONCRIEF
MEFFORD
MCPHEE
MCMORRIS
MCEACHERN
MCCLURG
MANSOUR
MAI
MADER
LEIJA
LECOMPTE
LAFOUNTAIN
LABRIE
JAQUEZ
HEALD
HASH
HARTLE
GAINER
FRISBY
FARINA
EIDSON
EDGERTON
DYKE
DURRETT
DUHON
CUOMO
COBOS
CERVANTEZ
BYBEE
BROCKWAY
BOROWSKI
BINION
BEERY
ARGUELLO
AMARO
ACTON
YUEN
WINTON
WIGFALL
WEEKLEY
VIDRINE
VANNOY
TARDIFF
SHOOP
SHILLING
SCHICK
SAND
SAFFORD
PRENDERGAST
PILGRIM
PELLERIN
OSUNA
NISSEN
NALLEY
MORITZ
MOLLER
MESSNER
MESSICK
MERRY
MERRIFIELD
MCGUINNESS
MATHERLY
MARCANO
MAHONE
LEMOS
LEBRUN
JARA
HOFFER
HEWLETT
HERREN
HECKER
HAWS
HAUG
HACK
GWIN
GOBER
GILLIARD
FREDETTE
FAVELA
ECHEVERRIA
DOWNER
DONOFRIO
DESROCHERS
DEE
CROZIER
CORSON
CLYDE
BECHTOLD
ARGUETA
APARICIO
ZAMUDIO
WILLETTE
WESTOVER
WESTERMAN
UTTER
TROYER
THIES
TAPLEY
SLAVIN
SHIRK
SANDLER
ROOP
RIMMER
RAYMER
RANGE
RADCLIFF
OTTEN
MOORER
MILLET
MCKIBBEN
MCCUTCHEN
MCAVOY
MCADOO
MAYORGA
MASTIN
MARTINEAU
MAREK
MADORE
LEFLORE
KROEGER
KENNON
JIMERSON
JAVIER
HOSTETTER
HORNBACK
HENDLEY
HANCE
GUARDADO
GRANADO
GOWEN
GOODALE
FLINN
FLEETWOOD
FITZ
DURKEE
DUPREY
DIPIETRO
DILLEY
CLYBURN
BRAWLEY
BECKLEY
ARANA
WEATHERBY
VOLLMER
VICTORIA
VESTAL
TUNNELL
TRIGG
TINGLE
TAKAHASHI
SWEATT
STORER
SNAPP
SHIVER
ROOKER
RED
RATHBUN
POISSON
PERRINE
PERRI
PASTOR
PARMER
PARKE
PARE
PAPA
PALMIERI
NOTTINGHAM
MIDKIFF
MECHAM
MCCOMAS
MCALPINE
LOVELADY
LILLARD
LALLY
KNOPP
KILE
KIGER
HAILE
GUPTA
GOLDSBERRY
GILREATH
FULKS
FRIESEN
FRANZEN
FLACK
FINDLAY
FERLAND
DREYER
DORE
DENNARD
DECKARD
DEBOSE
CRIM
COULOMBE
CORK
CHANCEY
CANTOR
BRANTON
BISSELL
BARNS
WOOLARD
WITHAM
WASSERMAN
WALDO
SPIEGEL
SHOFFNER
SCHOLZ
RUCH
ROSSMAN
READY
PETRY
PALACIO
PAEZ
NEARY
MORTENSON
MILLSAP
MIELE
MICK
MENKE
MCKIM
MCANALLY
MARTINES
MANOR
MALCOM
LEMLEY
LAROCHELLE
KLAUS
KLATT
KAUFMANN
KAPP
HELMER
HEDGE
HALLORAN
GLISSON
FRECHETTE
FONTANA
ENOCH
EAGAN
DRUM
DISTEFANO
DANLEY
CREEKMORE
CHARTIER
CHAFFEE
CARILLO
BURG
BOLINGER
BERKLEY
BENZ
BASSO
BASH
BARRIER
ZELAYA
WOODRING
WITKOWSKI
WILMOT
WILKENS
WIELAND
VIRGIL
VERDUGO
URQUHART
TSAI
TIMMS
SWIGER
SWAIM
SUSSMAN
SCARLETT
PIRES
MOLNAR
MCATEE
MAURICE
LOWDER
LOOS
LINKER
LANDES
KINGERY
KEELEY
HUFFORD
HIGA
HENDREN
HAMMACK
HAMANN
GILLAM
GERHARDT
FELL
EUGENE
EDELMAN
EBY
DELK
DEANS
CURL
CONSTANTINE
CLEAVER
CLAAR
CASIANO
CARRUTH
CARLYLE
BUMP
BROPHY
BOLANOS
BIBBS
BESSETTE
BEGGS
BAUGHER
BARTEL
AVERILL
ANDRESEN
AMIN
ALDEN
ADAMES
WILDMAN
VIA
VALENTE
TURNBOW
TSE
SWINK
SUBLETT
STROH
STRINGFELLOW
RIDGWAY
PUGLIESE
POTEAT
PANG
OHARE
NEUBAUER
MURCHISON
MOHAMED
MINGO
LUCKY
LEMMONS
KWON
KELLAM
KEAN
JARMON
HYDEN
HUDAK
HOLLINGER
HENKEL
HEMINGWAY
HASSON
HANSEL
HALTER
HAIRE
GOODNIGHT
GINSBERG
GILLISPIE
FOGEL
FLORY
ETTER
ELLEDGE
ECKMAN
DEAS
CURRIN
CRAFTON
COOMER
COLTER
CLAXTON
BULTER
BRADDOCK
BOWYER
BLIZZARD
BINNS
BING
BELLOWS
BASKERVILLE
BARROS
ANSLEY
WOOLF
WIGHT
WALDMAN
WADLEY
TULL
TRULL
TESCH
STRUCK
STOUFFER
STADLER
SLAY
SHUBERT
SEDILLO
SANTACRUZ
REINKE
RALEIGH
POYNTER
NERI
NEALE
NATIVIDAD
MOWRY
MORALEZ
MONGER
MITCHUM
MERRYMAN
MANION
MACDOUGALL
LUX
LITCHFIELD
LEY
LEVITT
LEPAGE
LASALLE
LAINE
KHOURY
KAVANAGH
KARNS
IVIE
HUEBNER
HODGKINS
HALPIN
GARICA
EVERSOLE
DUTRA
DUNAGAN
DUFFEY
DILLMAN
DILLION
DEVILLE
DEARBORN
DAMATO
COURSON
COULSON
BURDINE
BRYCE
BOUSQUET
BONIN
BISH
ATENCIO
WESTBROOKS
WAGES
VACA
TYE
TONER
TOMAS
TILLIS
SWETT
SURFACE
STRUBLE
STANFILL
SON
SOLORZANO
SLUSHER
SIPPLE
SIM
SILVAS
SHULTS
SCHEXNAYDER
SAEZ
RODAS
RAGER
PULVER
PLAZA
PENTON
PANIAGUA
MENESES
MCFARLIN
MCAULEY
MATZ
MALOY
MAGRUDER
LOHMAN
LANDA
LACOMBE
JAIMES
HOM
HOLZER
HOLST
HEIL
HACKLER
GRUNDY
GREGOR
GILKEY
FARNHAM
DURFEE
DUNTON
DUNSTON
DUDA
DEWS
DANA
CRAVER
CORRIVEAU
CONWELL
COLELLA
CHAMBLESS
BREMER
BOUTTE
BOURASSA
BLAISDELL
BACKMAN
BABINEAUX
AUDETTE
ALLEMAN
TOWNER
TAVERAS
TARANGO
SULLINS
SUITER
STALLARD
SOLBERG
SCHLUETER
POULOS
PIMENTAL
OWSLEY
OLIVIER
OKELLEY
NATIONS
MOFFATT
METCALFE
MEEKINS
MEDELLIN
MCGLYNN
MCCOWAN
MARRIOTT
MARABLE
LENNOX
LAMOUREUX
KOSS
KERBY
KARP
JASON
ISENBERG
HOWZE
HOCKENBERRY
HIGHSMITH
HARBOUR
HALLMARK
GUSMAN
GREELEY
GIDDINGS
GAUDET
GALLUP
FLEENOR
EICHER
EDINGTON
DIMAGGIO
DEMENT
DEMELLO
DECASTRO
CRUISE
BUSHMAN
BRUNDAGE
BROOKER
BROOKE
BOURG
BOARD
BLACKSTOCK
BERGMANN
BEATON
BANISTER
ARGO
APPLING
WORTMAN
WATTERSON
VILLALPANDO
TILLOTSON
TIGHE
SUNDBERG
STERNBERG
STAMEY
SPEAKS
SHIPE
SEEGER
SCARBERRY
SATTLER
SAIN
ROTHSTEIN
POTEET
PLOWMAN
PETTIFORD
PENLAND
PEACH
PARTAIN
PANKEY
OYLER
OGLETREE
OGBURN
MOTON
MILLION
MERKEL
MASK
MARKUS
LUCIER
LAZARUS
LAVELLE
LAKEY
KRATZ
KINSER
KERSHAW
JOSEPHSON
JESSE
IMHOFF
IBANEZ
HENDRY
HAMMON
FRISBIE
FRIEDRICH
FRAWLEY
FRAGA
FORESTER
ESKEW
EMMERT
DRENNAN
DOYON
DOMINICK
DANDRIDGE
CUMMING
CAWLEY
CARVAJAL
BRACEY
BELISLE
BATEY
AHNER
WYSOCKI
WEISER
VELIZ
TINCHER
SHERLOCK
SANTO
SANSONE
SANKEY
SANDSTROM
SALE
ROHRER
RISNER
PRIDEMORE
PFEFFER
PERSINGER
PEERY
OUBRE
ORANGE
NOWICKI
MUSGRAVE
MURDOCH
MULLINAX
MCCARY
MATHIEU
LIVENGOOD
LEONARDO
KYSER
KLINK
KIMES
KELLNER
KAVANAUGH
KASTEN
IMES
HOEY
HINSHAW
HALLEY
HAKE
GURULE
GRUBE
GRILLO
GETER
GATTO
GARVER
GARRETSON
FARWELL
EILAND
DUNFORD
DECARLO
CORSO
CORE
COLMAN
COLLARD
CLEGHORN
CHASTEEN
CAVENDER
CARLILE
CALVO
BYERLY
BROGDON
BROADWATER
BREAULT
BONO
BERGIN
BEHR
BALLENGER
AMICK
YAN
VICE
TAMEZ
STIFFLER
STEINKE
SIMMON
SHANKLE
SCHALLER
SALMONS
SACKETT
SAAD
RIDEOUT
READER
RATCLIFFE
RAO
RANSON
RANDELL
PLASCENCIA
PETTERSON
OLSZEWSKI
OLNEY
OLGUIN
NILSSON
NEVELS
MORELLI
MONTIEL
MONGE
MICHELL
MICHAELSON
MERTENS
MCCHESNEY
MCALPIN
MATHEWSON
LOWER
LOUDERMILK
LINEBERRY
LIGGETT
LAMP
KINLAW
KIGHT
JUST
JOST
HEREFORD
HARDEMAN
HALPERN
HALLIDAY
HAFER
GAUL
FRIEL
FREITAG
FRANCES
FORSBERG
EVANGELISTA
DOERING
DICARLO
DENDY
DELP
DEGUZMAN
DAMERON
CURTISS
COUSIN
COSPER
CHARLEY
CAUTHEN
CAO
CAMPER
BRADBERRY
BOUTON
BONNELL
BIXBY
BIEBER
BEVERIDGE
BELLE
BEDWELL
BARHORST
BANNON
BALTAZAR
BAIER
AYOTTE
ATTAWAY
ARENAS
ALEX
ABREGO
WATFORD
VALLEY
TURGEON
TUNSTALL
THAXTON
THAI
TENORIO
STOTTS
STHILAIRE
SPIKER
SHEDD
SENG
SEABOLT
SCALF
SALYERS
RUHL
ROWLETT
ROBINETT
PFISTER
PERLMAN
PEPE
PARKMAN
PARADISE
OLIN
NUNNALLY
NORVELL
NAPPER
MODLIN
MCKELLAR
MCCLEAN
MASCARENAS
MANCHESTER
LEIBOWITZ
LEDEZMA
KUHLMAN
KOBAYASHI
HUNLEY
HOLMQUIST
HINKLEY
HAZARD
HARTSELL
GRIBBLE
GRAVELY
FIFIELD
ELIASON
DOCTOR
DOAK
CROSSLAND
COVER
CLAIR
CARLETON
BUTTERS
BRIDGEMAN
BOJORQUEZ
BOGGESS
BANKER
AUTEN
JAMES  | http://www.census.gov/genealogy/names/dist.male.first
JOHN
ROBERT
MICHAEL
WILLIAM
DAVID
RICHARD
CHARLES
JOSEPH
THOMAS
CHRISTOPHER
DANIEL
PAUL
MARK
DONALD
GEORGE
KENNETH
STEVEN
EDWARD
BRIAN
RONALD
ANTHONY
KEVIN
JASON
MATTHEW
GARY
TIMOTHY
JOSE
LARRY
JEFFREY
FRANK
SCOTT
ERIC
STEPHEN
ANDREW
RAYMOND
GREGORY
JOSHUA
JERRY
DENNIS
WALTER
PATRICK
PETER
HAROLD
DOUGLAS
HENRY
CARL
ARTHUR
RYAN
ROGER
JOE
JUAN
JACK
ALBERT
JONATHAN
JUSTIN
TERRY
GERALD
KEITH
SAMUEL
WILLIE
RALPH
LAWRENCE
NICHOLAS
ROY
BENJAMIN
BRUCE
BRANDON
ADAM
HARRY
FRED
WAYNE
BILLY
STEVE
LOUIS
JEREMY
AARON
RANDY
HOWARD
EUGENE
CARLOS
RUSSELL
BOBBY
VICTOR
MARTIN
ERNEST
PHILLIP
TODD
JESSE
CRAIG
ALAN
SHAWN
CLARENCE
SEAN
PHILIP
CHRIS
JOHNNY
EARL
JIMMY
ANTONIO
DANNY
BRYAN
TONY
LUIS
MIKE
STANLEY
LEONARD
NATHAN
DALE
MANUEL
RODNEY
CURTIS
NORMAN
ALLEN
MARVIN
VINCENT
GLENN
JEFFERY
TRAVIS
JEFF
CHAD
JACOB
LEE
MELVIN
ALFRED
KYLE
FRANCIS
BRADLEY
JESUS
HERBERT
FREDERICK
RAY
JOEL
EDWIN
DON
EDDIE
RICKY
TROY
RANDALL
BARRY
ALEXANDER
BERNARD
MARIO
LEROY
FRANCISCO
MARCUS
MICHEAL
THEODORE
CLIFFORD
MIGUEL
OSCAR
JAY
JIM
TOM
CALVIN
ALEX
JON
RONNIE
BILL
LLOYD
TOMMY
LEON
DEREK
WARREN
DARRELL
JEROME
FLOYD
LEO
ALVIN
TIM
WESLEY
GORDON
DEAN
GREG
JORGE
DUSTIN
PEDRO
DERRICK
DAN
LEWIS
ZACHARY
COREY
HERMAN
MAURICE
VERNON
ROBERTO
CLYDE
GLEN
HECTOR
SHANE
RICARDO
SAM
RICK
LESTER
BRENT
RAMON
CHARLIE
TYLER
GILBERT
GENE
MARC
REGINALD
RUBEN
BRETT
ANGEL
NATHANIEL
RAFAEL
LESLIE
EDGAR
MILTON
RAUL
BEN
CHESTER
CECIL
DUANE
FRANKLIN
ANDRE
ELMER
BRAD
GABRIEL
RON
MITCHELL
ROLAND
ARNOLD
HARVEY
JARED
ADRIAN
KARL
CORY
CLAUDE
ERIK
DARRYL
JAMIE
NEIL
JESSIE
CHRISTIAN
JAVIER
FERNANDO
CLINTON
TED
MATHEW
TYRONE
DARREN
LONNIE
LANCE
CODY
JULIO
KELLY
KURT
ALLAN
NELSON
GUY
CLAYTON
HUGH
MAX
DWAYNE
DWIGHT
ARMANDO
FELIX
JIMMIE
EVERETT
JORDAN
IAN
WALLACE
KEN
BOB
JAIME
CASEY
ALFREDO
ALBERTO
DAVE
IVAN
JOHNNIE
SIDNEY
BYRON
JULIAN
ISAAC
MORRIS
CLIFTON
WILLARD
DARYL
ROSS
VIRGIL
ANDY
MARSHALL
SALVADOR
PERRY
KIRK
SERGIO
MARION
TRACY
SETH
KENT
TERRANCE
RENE
EDUARDO
TERRENCE
ENRIQUE
FREDDIE
WADE
AUSTIN
STUART
FREDRICK
ARTURO
ALEJANDRO
JACKIE
JOEY
NICK
LUTHER
WENDELL
JEREMIAH
EVAN
JULIUS
DANA
DONNIE
OTIS
SHANNON
TREVOR
OLIVER
LUKE
HOMER
GERARD
DOUG
KENNY
HUBERT
ANGELO
SHAUN
LYLE
MATT
LYNN
ALFONSO
ORLANDO
REX
CARLTON
ERNESTO
CAMERON
NEAL
PABLO
LORENZO
OMAR
WILBUR
BLAKE
GRANT
HORACE
RODERICK
KERRY
ABRAHAM
WILLIS
RICKEY
JEAN
IRA
ANDRES
CESAR
JOHNATHAN
MALCOLM
RUDOLPH
DAMON
KELVIN
RUDY
PRESTON
ALTON
ARCHIE
MARCO
WM
PETE
RANDOLPH
GARRY
GEOFFREY
JONATHON
FELIPE
BENNIE
GERARDO
ED
DOMINIC
ROBIN
LOREN
DELBERT
COLIN
GUILLERMO
EARNEST
LUCAS
BENNY
NOEL
SPENCER
RODOLFO
MYRON
EDMUND
GARRETT
SALVATORE
CEDRIC
LOWELL
GREGG
SHERMAN
WILSON
DEVIN
SYLVESTER
KIM
ROOSEVELT
ISRAEL
JERMAINE
FORREST
WILBERT
LELAND
SIMON
GUADALUPE
CLARK
IRVING
CARROLL
BRYANT
OWEN
RUFUS
WOODROW
SAMMY
KRISTOPHER
MACK
LEVI
MARCOS
GUSTAVO
JAKE
LIONEL
MARTY
TAYLOR
ELLIS
DALLAS
GILBERTO
CLINT
NICOLAS
LAURENCE
ISMAEL
ORVILLE
DREW
JODY
ERVIN
DEWEY
AL
WILFRED
JOSH
HUGO
IGNACIO
CALEB
TOMAS
SHELDON
ERICK
FRANKIE
STEWART
DOYLE
DARREL
ROGELIO
TERENCE
SANTIAGO
ALONZO
ELIAS
BERT
ELBERT
RAMIRO
CONRAD
PAT
NOAH
GRADY
PHIL
CORNELIUS
LAMAR
ROLANDO
CLAY
PERCY
DEXTER
BRADFORD
MERLE
DARIN
AMOS
TERRELL
MOSES
IRVIN
SAUL
ROMAN
DARNELL
RANDAL
TOMMIE
TIMMY
DARRIN
WINSTON
BRENDAN
TOBY
VAN
ABEL
DOMINICK
BOYD
COURTNEY
JAN
EMILIO
ELIJAH
CARY
DOMINGO
SANTOS
AUBREY
EMMETT
MARLON
EMANUEL
JERALD
EDMOND
EMIL
DEWAYNE
WILL
OTTO
TEDDY
REYNALDO
BRET
MORGAN
JESS
TRENT
HUMBERTO
EMMANUEL
STEPHAN
LOUIE
VICENTE
LAMONT
STACY
GARLAND
MILES
MICAH
EFRAIN
BILLIE
LOGAN
HEATH
RODGER
HARLEY
DEMETRIUS
ETHAN
ELDON
ROCKY
PIERRE
JUNIOR
FREDDY
ELI
BRYCE
ANTOINE
ROBBIE
KENDALL
ROYCE
STERLING
MICKEY
CHASE
GROVER
ELTON
CLEVELAND
DYLAN
CHUCK
DAMIAN
REUBEN
STAN
AUGUST
LEONARDO
JASPER
RUSSEL
ERWIN
BENITO
HANS
MONTE
BLAINE
ERNIE
CURT
QUENTIN
AGUSTIN
MURRAY
JAMAL
DEVON
ADOLFO
HARRISON
TYSON
BURTON
BRADY
ELLIOTT
WILFREDO
BART
JARROD
VANCE
DENIS
DAMIEN
JOAQUIN
HARLAN
DESMOND
ELLIOT
DARWIN
ASHLEY
GREGORIO
BUDDY
XAVIER
KERMIT
ROSCOE
ESTEBAN
ANTON
SOLOMON
SCOTTY
NORBERT
ELVIN
WILLIAMS
NOLAN
CAREY
ROD
QUINTON
HAL
BRAIN
ROB
ELWOOD
KENDRICK
DARIUS
MOISES
SON
MARLIN
FIDEL
THADDEUS
CLIFF
MARCEL
ALI
JACKSON
RAPHAEL
BRYON
ARMAND
ALVARO
JEFFRY
DANE
JOESPH
THURMAN
NED
SAMMIE
RUSTY
MICHEL
MONTY
RORY
FABIAN
REGGIE
MASON
GRAHAM
KRIS
ISAIAH
VAUGHN
GUS
AVERY
LOYD
DIEGO
ALEXIS
ADOLPH
NORRIS
MILLARD
ROCCO
GONZALO
DERICK
RODRIGO
GERRY
STACEY
CARMEN
WILEY
RIGOBERTO
ALPHONSO
TY
SHELBY
RICKIE
NOE
VERN
BOBBIE
REED
JEFFERSON
ELVIS
BERNARDO
MAURICIO
HIRAM
DONOVAN
BASIL
RILEY
OLLIE
NICKOLAS
MAYNARD
SCOT
VINCE
QUINCY
EDDY
SEBASTIAN
FEDERICO
ULYSSES
HERIBERTO
DONNELL
COLE
DENNY
DAVIS
GAVIN
EMERY
WARD
ROMEO
JAYSON
DION
DANTE
CLEMENT
COY
ODELL
MAXWELL
JARVIS
BRUNO
ISSAC
MARY
DUDLEY
BROCK
SANFORD
COLBY
CARMELO
BARNEY
NESTOR
HOLLIS
STEFAN
DONNY
ART
LINWOOD
BEAU
WELDON
GALEN
ISIDRO
TRUMAN
DELMAR
JOHNATHON
SILAS
FREDERIC
DICK
KIRBY
IRWIN
CRUZ
MERLIN
MERRILL
CHARLEY
MARCELINO
LANE
HARRIS
CLEO
CARLO
TRENTON
KURTIS
HUNTER
AURELIO
WINFRED
VITO
COLLIN
DENVER
CARTER
LEONEL
EMORY
PASQUALE
MOHAMMAD
MARIANO
DANIAL
BLAIR
LANDON
DIRK
BRANDEN
ADAN
NUMBERS
CLAIR
BUFORD
GERMAN
BERNIE
WILMER
JOAN
EMERSON
ZACHERY
FLETCHER
JACQUES
ERROL
DALTON
MONROE
JOSUE
DOMINIQUE
EDWARDO
BOOKER
WILFORD
SONNY
SHELTON
CARSON
THERON
RAYMUNDO
DAREN
TRISTAN
HOUSTON
ROBBY
LINCOLN
JAME
GENARO
GALE
BENNETT
OCTAVIO
CORNELL
LAVERNE
HUNG
ARRON
ANTONY
HERSCHEL
ALVA
GIOVANNI
GARTH
CYRUS
CYRIL
RONNY
STEVIE
LON
FREEMAN
ERIN
DUNCAN
KENNITH
CARMINE
AUGUSTINE
YOUNG
ERICH
CHADWICK
WILBURN
RUSS
REID
MYLES
ANDERSON
MORTON
JONAS
FOREST
MITCHEL
MERVIN
ZANE
RICH
JAMEL
LAZARO
ALPHONSE
RANDELL
MAJOR
JOHNIE
JARRETT
BROOKS
ARIEL
ABDUL
DUSTY
LUCIANO
LINDSEY
TRACEY
SEYMOUR
SCOTTIE
EUGENIO
MOHAMMED
SANDY
VALENTIN
CHANCE
ARNULFO
LUCIEN
FERDINAND
THAD
EZRA
SYDNEY
ALDO
RUBIN
ROYAL
MITCH
EARLE
ABE
WYATT
MARQUIS
LANNY
KAREEM
JAMAR
BORIS
ISIAH
EMILE
ELMO
ARON
LEOPOLDO
EVERETTE
JOSEF
GAIL
ELOY
DORIAN
RODRICK
REINALDO
LUCIO
JERROD
WESTON
HERSHEL
BARTON
PARKER
LEMUEL
LAVERN
BURT
JULES
GIL
ELISEO
AHMAD
NIGEL
EFREN
ANTWAN
ALDEN
MARGARITO
COLEMAN
REFUGIO
DINO
OSVALDO
LES
DEANDRE
NORMAND
KIETH
IVORY
ANDREA
TREY
NORBERTO
NAPOLEON
JEROLD
FRITZ
ROSENDO
MILFORD
SANG
DEON
CHRISTOPER
ALFONZO
LYMAN
JOSIAH
BRANT
WILTON
RICO
JAMAAL
DEWITT
CAROL
BRENTON
YONG
OLIN
FOSTER
FAUSTINO
CLAUDIO
JUDSON
GINO
EDGARDO
BERRY
ALEC
TANNER
JARRED
DONN
TRINIDAD
TAD
SHIRLEY
PRINCE
PORFIRIO
ODIS
MARIA
LENARD
CHAUNCEY
CHANG
TOD
MEL
MARCELO
KORY
AUGUSTUS
KEVEN
HILARIO
BUD
SAL
ROSARIO
ORVAL
MAURO
DANNIE
ZACHARIAH
OLEN
ANIBAL
MILO
JED
FRANCES
THANH
DILLON
AMADO
NEWTON
CONNIE
LENNY
TORY
RICHIE
LUPE
HORACIO
BRICE
MOHAMED
DELMER
DARIO
REYES
DEE
MAC
JONAH
JERROLD
ROBT
HANK
SUNG
RUPERT
ROLLAND
KENTON
DAMION
CHI
ANTONE
WALDO
FREDRIC
BRADLY
QUINN
KIP
BURL
WALKER
TYREE
JEFFEREY
AHMED
WILLY
STANFORD
OREN
NOBLE
MOSHE
MIKEL
ENOCH
BRENDON
QUINTIN
JAMISON
FLORENCIO
DARRICK
TOBIAS
MINH
HASSAN
GIUSEPPE
DEMARCUS
CLETUS
TYRELL
LYNDON
KEENAN
WERNER
THEO
GERALDO
LOU
COLUMBUS
CHET
BERTRAM
MARKUS
HUEY
HILTON
DWAIN
DONTE
TYRON
OMER
ISAIAS
HIPOLITO
FERMIN
CHUNG
ADALBERTO
VALENTINE
JAMEY
BO
BARRETT
WHITNEY
TEODORO
MCKINLEY
MAXIMO
GARFIELD
SOL
RALEIGH
LAWERENCE
ABRAM
RASHAD
KING
EMMITT
DARON
CHONG
SAMUAL
PARIS
OTHA
MIQUEL
LACY
EUSEBIO
DONG
DOMENIC
DARRON
BUSTER
ANTONIA
WILBER
RENATO
JC
HOYT
HAYWOOD
EZEKIEL
CHAS
FLORENTINO
ELROY
CLEMENTE
ARDEN
NEVILLE
KELLEY
EDISON
DESHAWN
CARROL
SHAYNE
NATHANIAL
JORDON
DANILO
CLAUD
VAL
SHERWOOD
RAYMON
RAYFORD
CRISTOBAL
AMBROSE
TITUS
HYMAN
FELTON
EZEQUIEL
ERASMO
STANTON
LONNY
LEN
IKE
MILAN
LINO
JAROD
HERB
ANDREAS
WALTON
RHETT
PALMER
JUDE
DOUGLASS
CORDELL
OSWALDO
ELLSWORTH
VIRGILIO
TONEY
NATHANAEL
DEL
BRITT
BENEDICT
MOSE
HONG
LEIGH
JOHNSON
ISREAL
GAYLE
GARRET
FAUSTO
ASA
ARLEN
ZACK
WARNER
MODESTO
FRANCESCO
MANUAL
JAE
GAYLORD
GASTON
FILIBERTO
DEANGELO
MICHALE
GRANVILLE
WES
MALIK
ZACKARY
TUAN
NICKY
ELDRIDGE
CRISTOPHER
CORTEZ
ANTIONE
MALCOM
LONG
KOREY
JOSPEH
COLTON
WAYLON
VON
HOSEA
SHAD
SANTO
RUDOLF
ROLF
REY
RENALDO
MARCELLUS
LUCIUS
LESLEY
KRISTOFER
BOYCE
BENTON
MAN
KASEY
JEWELL
HAYDEN
HARLAND
ARNOLDO
RUEBEN
LEANDRO
KRAIG
JERRELL
JEROMY
HOBERT
CEDRICK
ARLIE
WINFORD
WALLY
PATRICIA
LUIGI
KENETH
JACINTO
GRAIG
FRANKLYN
EDMUNDO
SID
PORTER
LEIF
LAUREN
JERAMY
ELISHA
BUCK
WILLIAN
VINCENZO
SHON
MICHAL
LYNWOOD
LINDSAY
JEWEL
JERE
HAI
ELDEN
DORSEY
DARELL
BRODERICK
ALONSO
MARY  | http://www.census.gov/genealogy/names/dist.female.first
PATRICIA
LINDA
BARBARA
ELIZABETH
JENNIFER
MARIA
SUSAN
MARGARET
DOROTHY
LISA
NANCY
KAREN
BETTY
HELEN
SANDRA
DONNA
CAROL
RUTH
SHARON
MICHELLE
LAURA
SARAH
KIMBERLY
DEBORAH
JESSICA
SHIRLEY
CYNTHIA
ANGELA
MELISSA
BRENDA
AMY
ANNA
REBECCA
VIRGINIA
KATHLEEN
PAMELA
MARTHA
DEBRA
AMANDA
STEPHANIE
CAROLYN
CHRISTINE
MARIE
JANET
CATHERINE
FRANCES
ANN
JOYCE
DIANE
ALICE
JULIE
HEATHER
TERESA
DORIS
GLORIA
EVELYN
JEAN
CHERYL
MILDRED
KATHERINE
JOAN
ASHLEY
JUDITH
ROSE
JANICE
KELLY
NICOLE
JUDY
CHRISTINA
KATHY
THERESA
BEVERLY
DENISE
TAMMY
IRENE
JANE
LORI
RACHEL
MARILYN
ANDREA
KATHRYN
LOUISE
SARA
ANNE
JACQUELINE
WANDA
BONNIE
JULIA
RUBY
LOIS
TINA
PHYLLIS
NORMA
PAULA
DIANA
ANNIE
LILLIAN
EMILY
ROBIN
PEGGY
CRYSTAL
GLADYS
RITA
DAWN
CONNIE
FLORENCE
TRACY
EDNA
TIFFANY
CARMEN
ROSA
CINDY
GRACE
WENDY
VICTORIA
EDITH
KIM
SHERRY
SYLVIA
JOSEPHINE
THELMA
SHANNON
SHEILA
ETHEL
ELLEN
ELAINE
MARJORIE
CARRIE
CHARLOTTE
MONICA
ESTHER
PAULINE
EMMA
JUANITA
ANITA
RHONDA
HAZEL
AMBER
EVA
DEBBIE
APRIL
LESLIE
CLARA
LUCILLE
JAMIE
JOANNE
ELEANOR
VALERIE
DANIELLE
MEGAN
ALICIA
SUZANNE
MICHELE
GAIL
BERTHA
DARLENE
VERONICA
JILL
ERIN
GERALDINE
LAUREN
CATHY
JOANN
LORRAINE
LYNN
SALLY
REGINA
ERICA
BEATRICE
DOLORES
BERNICE
AUDREY
YVONNE
ANNETTE
JUNE
SAMANTHA
MARION
DANA
STACY
ANA
RENEE
IDA
VIVIAN
ROBERTA
HOLLY
BRITTANY
MELANIE
LORETTA
YOLANDA
JEANETTE
LAURIE
KATIE
KRISTEN
VANESSA
ALMA
SUE
ELSIE
BETH
JEANNE
VICKI
CARLA
TARA
ROSEMARY
EILEEN
TERRI
GERTRUDE
LUCY
TONYA
ELLA
STACEY
WILMA
GINA
KRISTIN
JESSIE
NATALIE
AGNES
VERA
WILLIE
CHARLENE
BESSIE
DELORES
MELINDA
PEARL
ARLENE
MAUREEN
COLLEEN
ALLISON
TAMARA
JOY
GEORGIA
CONSTANCE
LILLIE
CLAUDIA
JACKIE
MARCIA
TANYA
NELLIE
MINNIE
MARLENE
HEIDI
GLENDA
LYDIA
VIOLA
COURTNEY
MARIAN
STELLA
CAROLINE
DORA
JO
VICKIE
MATTIE
TERRY
MAXINE
IRMA
MABEL
MARSHA
MYRTLE
LENA
CHRISTY
DEANNA
PATSY
HILDA
GWENDOLYN
JENNIE
NORA
MARGIE
NINA
CASSANDRA
LEAH
PENNY
KAY
PRISCILLA
NAOMI
CAROLE
BRANDY
OLGA
BILLIE
DIANNE
TRACEY
LEONA
JENNY
FELICIA
SONIA
MIRIAM
VELMA
BECKY
BOBBIE
VIOLET
KRISTINA
TONI
MISTY
MAE
SHELLY
DAISY
RAMONA
SHERRI
ERIKA
KATRINA
CLAIRE
LINDSEY
LINDSAY
GENEVA
GUADALUPE
BELINDA
MARGARITA
SHERYL
CORA
FAYE
ADA
NATASHA
SABRINA
ISABEL
MARGUERITE
HATTIE
HARRIET
MOLLY
CECILIA
KRISTI
BRANDI
BLANCHE
SANDY
ROSIE
JOANNA
IRIS
EUNICE
ANGIE
INEZ
LYNDA
MADELINE
AMELIA
ALBERTA
GENEVIEVE
MONIQUE
JODI
JANIE
MAGGIE
KAYLA
SONYA
JAN
LEE
KRISTINE
CANDACE
FANNIE
MARYANN
OPAL
ALISON
YVETTE
MELODY
LUZ
SUSIE
OLIVIA
FLORA
SHELLEY
KRISTY
MAMIE
LULA
LOLA
VERNA
BEULAH
ANTOINETTE
CANDICE
JUANA
JEANNETTE
PAM
KELLI
HANNAH
WHITNEY
BRIDGET
KARLA
CELIA
LATOYA
PATTY
SHELIA
GAYLE
DELLA
VICKY
LYNNE
SHERI
MARIANNE
KARA
JACQUELYN
ERMA
BLANCA
MYRA
LETICIA
PAT
KRISTA
ROXANNE
ANGELICA
JOHNNIE
ROBYN
FRANCIS
ADRIENNE
ROSALIE
ALEXANDRA
BROOKE
BETHANY
SADIE
BERNADETTE
TRACI
JODY
KENDRA
JASMINE
NICHOLE
RACHAEL
CHELSEA
MABLE
ERNESTINE
MURIEL
MARCELLA
ELENA
KRYSTAL
ANGELINA
NADINE
KARI
ESTELLE
DIANNA
PAULETTE
LORA
MONA
DOREEN
ROSEMARIE
ANGEL
DESIREE
ANTONIA
HOPE
GINGER
JANIS
BETSY
CHRISTIE
FREDA
MERCEDES
MEREDITH
LYNETTE
TERI
CRISTINA
EULA
LEIGH
MEGHAN
SOPHIA
ELOISE
ROCHELLE
GRETCHEN
CECELIA
RAQUEL
HENRIETTA
ALYSSA
JANA
KELLEY
GWEN
KERRY
JENNA
TRICIA
LAVERNE
OLIVE
ALEXIS
TASHA
SILVIA
ELVIRA
CASEY
DELIA
SOPHIE
KATE
PATTI
LORENA
KELLIE
SONJA
LILA
LANA
DARLA
MAY
MINDY
ESSIE
MANDY
LORENE
ELSA
JOSEFINA
JEANNIE
MIRANDA
DIXIE
LUCIA
MARTA
FAITH
LELA
JOHANNA
SHARI
CAMILLE
TAMI
SHAWNA
ELISA
EBONY
MELBA
ORA
NETTIE
TABITHA
OLLIE
JAIME
WINIFRED
KRISTIE
MARINA
ALISHA
AIMEE
RENA
MYRNA
MARLA
TAMMIE
LATASHA
BONITA
PATRICE
RONDA
SHERRIE
ADDIE
FRANCINE
DELORIS
STACIE
ADRIANA
CHERI
SHELBY
ABIGAIL
CELESTE
JEWEL
CARA
ADELE
REBEKAH
LUCINDA
DORTHY
CHRIS
EFFIE
TRINA
REBA
SHAWN
SALLIE
AURORA
LENORA
ETTA
LOTTIE
KERRI
TRISHA
NIKKI
ESTELLA
FRANCISCA
JOSIE
TRACIE
MARISSA
KARIN
BRITTNEY
JANELLE
LOURDES
LAUREL
HELENE
FERN
ELVA
CORINNE
KELSEY
INA
BETTIE
ELISABETH
AIDA
CAITLIN
INGRID
IVA
EUGENIA
CHRISTA
GOLDIE
CASSIE
MAUDE
JENIFER
THERESE
FRANKIE
DENA
LORNA
JANETTE
LATONYA
CANDY
MORGAN
CONSUELO
TAMIKA
ROSETTA
DEBORA
CHERIE
POLLY
DINA
JEWELL
FAY
JILLIAN
DOROTHEA
NELL
TRUDY
ESPERANZA
PATRICA
KIMBERLEY
SHANNA
HELENA
CAROLINA
CLEO
STEFANIE
ROSARIO
OLA
JANINE
MOLLIE
LUPE
ALISA
LOU
MARIBEL
SUSANNE
BETTE
SUSANA
ELISE
CECILE
ISABELLE
LESLEY
JOCELYN
PAIGE
JONI
RACHELLE
LEOLA
DAPHNE
ALTA
ESTER
PETRA
GRACIELA
IMOGENE
JOLENE
KEISHA
LACEY
GLENNA
GABRIELA
KERI
URSULA
LIZZIE
KIRSTEN
SHANA
ADELINE
MAYRA
JAYNE
JACLYN
GRACIE
SONDRA
CARMELA
MARISA
ROSALIND
CHARITY
TONIA
BEATRIZ
MARISOL
CLARICE
JEANINE
SHEENA
ANGELINE
FRIEDA
LILY
ROBBIE
SHAUNA
MILLIE
CLAUDETTE
CATHLEEN
ANGELIA
GABRIELLE
AUTUMN
KATHARINE
SUMMER
JODIE
STACI
LEA
CHRISTI
JIMMIE
JUSTINE
ELMA
LUELLA
MARGRET
DOMINIQUE
SOCORRO
RENE
MARTINA
MARGO
MAVIS
CALLIE
BOBBI
MARITZA
LUCILE
LEANNE
JEANNINE
DEANA
AILEEN
LORIE
LADONNA
WILLA
MANUELA
GALE
SELMA
DOLLY
SYBIL
ABBY
LARA
DALE
IVY
DEE
WINNIE
MARCY
LUISA
JERI
MAGDALENA
OFELIA
MEAGAN
AUDRA
MATILDA
LEILA
CORNELIA
BIANCA
SIMONE
BETTYE
RANDI
VIRGIE
LATISHA
BARBRA
GEORGINA
ELIZA
LEANN
BRIDGETTE
RHODA
HALEY
ADELA
NOLA
BERNADINE
FLOSSIE
ILA
GRETA
RUTHIE
NELDA
MINERVA
LILLY
TERRIE
LETHA
HILARY
ESTELA
VALARIE
BRIANNA
ROSALYN
EARLINE
CATALINA
AVA
MIA
CLARISSA
LIDIA
CORRINE
ALEXANDRIA
CONCEPCION
TIA
SHARRON
RAE
DONA
ERICKA
JAMI
ELNORA
CHANDRA
LENORE
NEVA
MARYLOU
MELISA
TABATHA
SERENA
AVIS
ALLIE
SOFIA
JEANIE
ODESSA
NANNIE
HARRIETT
LORAINE
PENELOPE
MILAGROS
EMILIA
BENITA
ALLYSON
ASHLEE
TANIA
TOMMIE
ESMERALDA
KARINA
EVE
PEARLIE
ZELMA
MALINDA
NOREEN
TAMEKA
SAUNDRA
HILLARY
AMIE
ALTHEA
ROSALINDA
JORDAN
LILIA
ALANA
GAY
CLARE
ALEJANDRA
ELINOR
MICHAEL
LORRIE
JERRI
DARCY
EARNESTINE
CARMELLA
TAYLOR
NOEMI
MARCIE
LIZA
ANNABELLE
LOUISA
EARLENE
MALLORY
CARLENE
NITA
SELENA
TANISHA
KATY
JULIANNE
JOHN
LAKISHA
EDWINA
MARICELA
MARGERY
KENYA
DOLLIE
ROXIE
ROSLYN
KATHRINE
NANETTE
CHARMAINE
LAVONNE
ILENE
KRIS
TAMMI
SUZETTE
CORINE
KAYE
JERRY
MERLE
CHRYSTAL
LINA
DEANNE
LILIAN
JULIANA
ALINE
LUANN
KASEY
MARYANNE
EVANGELINE
COLETTE
MELVA
LAWANDA
YESENIA
NADIA
MADGE
KATHIE
EDDIE
OPHELIA
VALERIA
NONA
MITZI
MARI
GEORGETTE
CLAUDINE
FRAN
ALISSA
ROSEANN
LAKEISHA
SUSANNA
REVA
DEIDRE
CHASITY
SHEREE
CARLY
JAMES
ELVIA
ALYCE
DEIRDRE
GENA
BRIANA
ARACELI
KATELYN
ROSANNE
WENDI
TESSA
BERTA
MARVA
IMELDA
MARIETTA
MARCI
LEONOR
ARLINE
SASHA
MADELYN
JANNA
JULIETTE
DEENA
AURELIA
JOSEFA
AUGUSTA
LILIANA
YOUNG
CHRISTIAN
LESSIE
AMALIA
SAVANNAH
ANASTASIA
VILMA
NATALIA
ROSELLA
LYNNETTE
CORINA
ALFREDA
LEANNA
CAREY
AMPARO
COLEEN
TAMRA
AISHA
WILDA
KARYN
CHERRY
QUEEN
MAURA
MAI
EVANGELINA
ROSANNA
HALLIE
ERNA
ENID
MARIANA
LACY
JULIET
JACKLYN
FREIDA
MADELEINE
MARA
HESTER
CATHRYN
LELIA
CASANDRA
BRIDGETT
ANGELITA
JANNIE
DIONNE
ANNMARIE
KATINA
BERYL
PHOEBE
MILLICENT
KATHERYN
DIANN
CARISSA
MARYELLEN
LIZ
LAURI
HELGA
GILDA
ADRIAN
RHEA
MARQUITA
HOLLIE
TISHA
TAMERA
ANGELIQUE
FRANCESCA
BRITNEY
KAITLIN
LOLITA
FLORINE
ROWENA
REYNA
TWILA
FANNY
JANELL
INES
CONCETTA
BERTIE
ALBA
BRIGITTE
ALYSON
VONDA
PANSY
ELBA
NOELLE
LETITIA
KITTY
DEANN
BRANDIE
LOUELLA
LETA
FELECIA
SHARLENE
LESA
BEVERLEY
ROBERT
ISABELLA
HERMINIA
TERRA
CELINA
TORI
OCTAVIA
JADE
DENICE
GERMAINE
SIERRA
MICHELL
CORTNEY
NELLY
DORETHA
SYDNEY
DEIDRA
MONIKA
LASHONDA
JUDI
CHELSEY
ANTIONETTE
MARGOT
BOBBY
ADELAIDE
NAN
LEEANN
ELISHA
DESSIE
LIBBY
KATHI
GAYLA
LATANYA
MINA
MELLISA
KIMBERLEE
JASMIN
RENAE
ZELDA
ELDA
MA
JUSTINA
GUSSIE
EMILIE
CAMILLA
ABBIE
ROCIO
KAITLYN
JESSE
EDYTHE
ASHLEIGH
SELINA
LAKESHA
GERI
ALLENE
PAMALA
MICHAELA
DAYNA
CARYN
ROSALIA
SUN
JACQULINE
REBECA
MARYBETH
KRYSTLE
IOLA
DOTTIE
BENNIE
BELLE
AUBREY
GRISELDA
ERNESTINA
ELIDA
ADRIANNE
DEMETRIA
DELMA
CHONG
JAQUELINE
DESTINY
ARLEEN
VIRGINA
RETHA
FATIMA
TILLIE
ELEANORE
CARI
TREVA
BIRDIE
WILHELMINA
ROSALEE
MAURINE
LATRICE
YONG
JENA
TARYN
ELIA
DEBBY
MAUDIE
JEANNA
DELILAH
CATRINA
SHONDA
HORTENCIA
THEODORA
TERESITA
ROBBIN
DANETTE
MARYJANE
FREDDIE
DELPHINE
BRIANNE
NILDA
DANNA
CINDI
BESS
IONA
HANNA
ARIEL
WINONA
VIDA
ROSITA
MARIANNA
WILLIAM
RACHEAL
GUILLERMINA
ELOISA
CELESTINE
CAREN
MALISSA
LONA
CHANTEL
SHELLIE
MARISELA
LEORA
AGATHA
SOLEDAD
MIGDALIA
IVETTE
CHRISTEN
ATHENA
JANEL
CHLOE
VEDA
PATTIE
TESSIE
TERA
MARILYNN
LUCRETIA
KARRIE
DINAH
DANIELA
ALECIA
ADELINA
VERNICE
SHIELA
PORTIA
MERRY
LASHAWN
DEVON
DARA
TAWANA
OMA
VERDA
CHRISTIN
ALENE
ZELLA
SANDI
RAFAELA
MAYA
KIRA
CANDIDA
ALVINA
SUZAN
SHAYLA
LYN
LETTIE
ALVA
SAMATHA
ORALIA
MATILDE
MADONNA
LARISSA
VESTA
RENITA
INDIA
DELOIS
SHANDA
PHILLIS
LORRI
ERLINDA
CRUZ
CATHRINE
BARB
ZOE
ISABELL
IONE
GISELA
CHARLIE
VALENCIA
ROXANNA
MAYME
KISHA
ELLIE
MELLISSA
DORRIS
DALIA
BELLA
ANNETTA
ZOILA
RETA
REINA
LAURETTA
KYLIE
CHRISTAL
PILAR
CHARLA
ELISSA
TIFFANI
TANA
PAULINA
LEOTA
BREANNA
JAYME
CARMEL
VERNELL
TOMASA
MANDI
DOMINGA
SANTA
MELODIE
LURA
ALEXA
TAMELA
RYAN
MIRNA
KERRIE
VENUS
NOEL
FELICITA
CRISTY
CARMELITA
BERNIECE
ANNEMARIE
TIARA
ROSEANNE
MISSY
CORI
ROXANA
PRICILLA
KRISTAL
JUNG
ELYSE
HAYDEE
ALETHA
BETTINA
MARGE
GILLIAN
FILOMENA
CHARLES
ZENAIDA
HARRIETTE
CARIDAD
VADA
UNA
ARETHA
PEARLINE
MARJORY
MARCELA
FLOR
EVETTE
ELOUISE
ALINA
TRINIDAD
DAVID
DAMARIS
CATHARINE
CARROLL
BELVA
NAKIA
MARLENA
LUANNE
LORINE
KARON
DORENE
DANITA
BRENNA
TATIANA
SAMMIE
LOUANN
LOREN
JULIANNA
ANDRIA
PHILOMENA
LUCILA
LEONORA
DOVIE
ROMONA
MIMI
JACQUELIN
GAYE
TONJA
MISTI
JOE
GENE
CHASTITY
STACIA
ROXANN
MICAELA
NIKITA
MEI
VELDA
MARLYS
JOHNNA
AURA
LAVERN
IVONNE
HAYLEY
NICKI
MAJORIE
HERLINDA
GEORGE
ALPHA
YADIRA
PERLA
GREGORIA
DANIEL
ANTONETTE
SHELLI
MOZELLE
MARIAH
JOELLE
CORDELIA
JOSETTE
CHIQUITA
TRISTA
LOUIS
LAQUITA
GEORGIANA
CANDI
SHANON
LONNIE
HILDEGARD
CECIL
VALENTINA
STEPHANY
MAGDA
KAROL
GERRY
GABRIELLA
TIANA
ROMA
RICHELLE
RAY
PRINCESS
OLETA
JACQUE
IDELLA
ALAINA
SUZANNA
JOVITA
BLAIR
TOSHA
RAVEN
NEREIDA
MARLYN
KYLA
JOSEPH
DELFINA
TENA
STEPHENIE
SABINA
NATHALIE
MARCELLE
GERTIE
DARLEEN
THEA
SHARONDA
SHANTEL
BELEN
VENESSA
ROSALINA
ONA
GENOVEVA
COREY
CLEMENTINE
ROSALBA
RENATE
RENATA
MI
IVORY
GEORGIANNA
FLOY
DORCAS
ARIANA
TYRA
THEDA
MARIAM
JULI
JESICA
DONNIE
VIKKI
VERLA
ROSELYN
MELVINA
JANNETTE
GINNY
DEBRAH
CORRIE
ASIA
VIOLETA
MYRTIS
LATRICIA
COLLETTE
CHARLEEN
ANISSA
VIVIANA
TWYLA
PRECIOUS
NEDRA
LATONIA
LAN
HELLEN
FABIOLA
ANNAMARIE
ADELL
SHARYN
CHANTAL
NIKI
MAUD
LIZETTE
LINDY
KIA
KESHA
JEANA
DANELLE
CHARLINE
CHANEL
CARROL
VALORIE
LIA
DORTHA
CRISTAL
SUNNY
LEONE
LEILANI
GERRI
DEBI
ANDRA
KESHIA
IMA
EULALIA
EASTER
DULCE
NATIVIDAD
LINNIE
KAMI
GEORGIE
CATINA
BROOK
ALDA
WINNIFRED
SHARLA
RUTHANN
MEAGHAN
MAGDALENE
LISSETTE
ADELAIDA
VENITA
TRENA
SHIRLENE
SHAMEKA
ELIZEBETH
DIAN
SHANTA
MICKEY
LATOSHA
CARLOTTA
WINDY
SOON
ROSINA
MARIANN
LEISA
JONNIE
DAWNA
CATHIE
BILLY
ASTRID
SIDNEY
LAUREEN
JANEEN
HOLLI
FAWN
VICKEY
TERESSA
SHANTE
RUBYE
MARCELINA
CHANDA
CARY
TERESE
SCARLETT
MARTY
MARNIE
LULU
LISETTE
JENIFFER
ELENOR
DORINDA
DONITA
CARMAN
BERNITA
ALTAGRACIA
ALETA
ADRIANNA
ZORAIDA
RONNIE
NICOLA
LYNDSEY
KENDALL
JANINA
CHRISSY
AMI
STARLA
PHYLIS
PHUONG
KYRA
CHARISSE
BLANCH
SANJUANITA
RONA
NANCI
MARILEE
MARANDA
CORY
BRIGETTE
SANJUANA
MARITA
KASSANDRA
JOYCELYN
IRA
FELIPA
CHELSIE
BONNY
MIREYA
LORENZA
KYONG
ILEANA
CANDELARIA
TONY
TOBY
SHERIE
OK
MARK
LUCIE
LEATRICE
LAKESHIA
GERDA
EDIE
BAMBI
MARYLIN
LAVON
HORTENSE
GARNET
EVIE
TRESSA
SHAYNA
LAVINA
KYUNG
JEANETTA
SHERRILL
SHARA
PHYLISS
MITTIE
ANABEL
ALESIA
THUY
TAWANDA
RICHARD
JOANIE
TIFFANIE
LASHANDA
KARISSA
ENRIQUETA
DARIA
DANIELLA
CORINNA
ALANNA
ABBEY
ROXANE
ROSEANNA
MAGNOLIA
LIDA
KYLE
JOELLEN
ERA
CORAL
CARLEEN
TRESA
PEGGIE
NOVELLA
NILA
MAYBELLE
JENELLE
CARINA
NOVA
MELINA
MARQUERITE
MARGARETTE
JOSEPHINA
EVONNE
DEVIN
CINTHIA
ALBINA
TOYA
TAWNYA
SHERITA
SANTOS
MYRIAM
LIZABETH
LISE
KEELY
JENNI
GISELLE
CHERYLE
ARDITH
ARDIS
ALESHA
ADRIANE
SHAINA
LINNEA
KAROLYN
HONG
FLORIDA
FELISHA
DORI
DARCI
ARTIE
ARMIDA
ZOLA
XIOMARA
VERGIE
SHAMIKA
NENA
NANNETTE
MAXIE
LOVIE
JEANE
JAIMIE
INGE
FARRAH
ELAINA
CAITLYN
STARR
FELICITAS
CHERLY
CARYL
YOLONDA
YASMIN
TEENA
PRUDENCE
PENNIE
NYDIA
MACKENZIE
ORPHA
MARVEL
LIZBETH
LAURETTE
JERRIE
HERMELINDA
CAROLEE
TIERRA
MIRIAN
META
MELONY
KORI
JENNETTE
JAMILA
ENA
ANH
YOSHIKO
SUSANNAH
SALINA
RHIANNON
JOLEEN
CRISTINE
ASHTON
ARACELY
TOMEKA
SHALONDA
MARTI
LACIE
KALA
JADA
ILSE
HAILEY
BRITTANI
ZONA
SYBLE
SHERRYL
RANDY
NIDIA
MARLO
KANDICE
KANDI
DEB
DEAN
AMERICA
ALYCIA
TOMMY
RONNA
NORENE
MERCY
JOSE
INGEBORG
GIOVANNA
GEMMA
CHRISTEL
AUDRY
ZORA
VITA
VAN
TRISH
STEPHAINE
SHIRLEE
SHANIKA
MELONIE
MAZIE
JAZMIN
INGA
HOA
HETTIE
GERALYN
FONDA
ESTRELLA
ADELLA
SU
SARITA
RINA
MILISSA
MARIBETH
GOLDA
EVON
ETHELYN
ENEDINA
CHERISE
CHANA
VELVA
TAWANNA
SADE
MIRTA
LI
KARIE
JACINTA
ELNA
DAVINA
CIERRA
ASHLIE
ALBERTHA
TANESHA
STEPHANI
NELLE
MINDI
LU
LORINDA
LARUE
FLORENE
DEMETRA
DEDRA
CIARA
CHANTELLE
ASHLY
SUZY
ROSALVA
NOELIA
LYDA
LEATHA
KRYSTYNA
KRISTAN
KARRI
DARLINE
DARCIE
CINDA
CHEYENNE
CHERRIE
AWILDA
ALMEDA
ROLANDA
LANETTE
JERILYN
GISELE
EVALYN
CYNDI
CLETA
CARIN
ZINA
ZENA
VELIA
TANIKA
PAUL
CHARISSA
THOMAS
TALIA
MARGARETE
LAVONDA
KAYLEE
KATHLENE
JONNA
IRENA
ILONA
IDALIA
CANDIS
CANDANCE
BRANDEE
ANITRA
ALIDA
SIGRID
NICOLETTE
MARYJO
LINETTE
HEDWIG
CHRISTIANA
CASSIDY
ALEXIA
TRESSIE
MODESTA
LUPITA
LITA
GLADIS
EVELIA
DAVIDA
CHERRI
CECILY
ASHELY
ANNABEL
AGUSTINA
WANITA
SHIRLY
ROSAURA
HULDA
EUN
BAILEY
YETTA
VERONA
THOMASINA
SIBYL
SHANNAN
MECHELLE
LUE
LEANDRA
LANI
KYLEE
KANDY
JOLYNN
FERNE
EBONI
CORENE
ALYSIA
ZULA
NADA
MOIRA
LYNDSAY
LORRETTA
JUAN
JAMMIE
HORTENSIA
GAYNELL
CAMERON
ADRIA
VINA
VICENTA
TANGELA
STEPHINE
NORINE
NELLA
LIANA
LESLEE
KIMBERELY
ILIANA
GLORY
FELICA
EMOGENE
ELFRIEDE
EDEN
EARTHA
CARMA
BEA
OCIE
MARRY
LENNIE
KIARA
JACALYN
CARLOTA
ARIELLE
YU
STAR
OTILIA
KIRSTIN
KACEY
JOHNETTA
JOEY
JOETTA
JERALDINE
JAUNITA
ELANA
DORTHEA
CAMI
AMADA
ADELIA
VERNITA
TAMAR
SIOBHAN
RENEA
RASHIDA
OUIDA
ODELL
NILSA
MERYL
KRISTYN
JULIETA
DANICA
BREANNE
AUREA
ANGLEA
SHERRON
ODETTE
MALIA
LORELEI
LIN
LEESA
KENNA
KATHLYN
FIONA
CHARLETTE
SUZIE
SHANTELL
SABRA
RACQUEL
MYONG
MIRA
MARTINE
LUCIENNE
LAVADA
JULIANN
JOHNIE
ELVERA
DELPHIA
CLAIR
CHRISTIANE
CHAROLETTE
CARRI
AUGUSTINE
ASHA
ANGELLA
PAOLA
NINFA
LEDA
LAI
EDA
SUNSHINE
STEFANI
SHANELL
PALMA
MACHELLE
LISSA
KECIA
KATHRYNE
KARLENE
JULISSA
JETTIE
JENNIFFER
HUI
CORRINA
CHRISTOPHER
CAROLANN
ALENA
TESS
ROSARIA
MYRTICE
MARYLEE
LIANE
KENYATTA
JUDIE
JANEY
IN
ELMIRA
ELDORA
DENNA
CRISTI
CATHI
ZAIDA
VONNIE
VIVA
VERNIE
ROSALINE
MARIELA
LUCIANA
LESLI
KARAN
FELICE
DENEEN
ADINA
WYNONA
TARSHA
SHERON
SHASTA
SHANITA
SHANI
SHANDRA
RANDA
PINKIE
PARIS
NELIDA
MARILOU
LYLA
LAURENE
LACI
JOI
JANENE
DOROTHA
DANIELE
DANI
CAROLYNN
CARLYN
BERENICE
AYESHA
ANNELIESE
ALETHEA
THERSA
TAMIKO
RUFINA
OLIVA
MOZELL
MARYLYN
MADISON
KRISTIAN
KATHYRN
KASANDRA
KANDACE
JANAE
GABRIEL
DOMENICA
DEBBRA
DANNIELLE
CHUN
BUFFY
BARBIE
ARCELIA
AJA
ZENOBIA
SHAREN
SHAREE
PATRICK
PAGE
MY
LAVINIA
KUM
KACIE
JACKELINE
HUONG
FELISA
EMELIA
ELEANORA
CYTHIA
CRISTIN
CLYDE
CLARIBEL
CARON
ANASTACIA
ZULMA
ZANDRA
YOKO
TENISHA
SUSANN
SHERILYN
SHAY
SHAWANDA
SABINE
ROMANA
MATHILDA
LINSEY
KEIKO
JOANA
ISELA
GRETTA
GEORGETTA
EUGENIE
DUSTY
DESIRAE
DELORA
CORAZON
ANTONINA
ANIKA
WILLENE
TRACEE
TAMATHA
REGAN
NICHELLE
MICKIE
MAEGAN
LUANA
LANITA
KELSIE
EDELMIRA
BREE
AFTON
TEODORA
TAMIE
SHENA
MEG
LINH
KELI
KACI
DANYELLE
BRITT
ARLETTE
ALBERTINE
ADELLE
TIFFINY
STORMY
SIMONA
NUMBERS
NICOLASA
NICHOL
NIA
NAKISHA
MEE
MAIRA
LOREEN
KIZZY
JOHNNY
JAY
FALLON
CHRISTENE
BOBBYE
ANTHONY
YING
VINCENZA
TANJA
RUBIE
RONI
QUEENIE
MARGARETT
KIMBERLI
IRMGARD
IDELL
HILMA
EVELINA
ESTA
EMILEE
DENNISE
DANIA
CARL
CARIE
ANTONIO
WAI
SANG
RISA
RIKKI
PARTICIA
MUI
MASAKO
MARIO
LUVENIA
LOREE
LONI
LIEN
KEVIN
GIGI
FLORENCIA
DORIAN
DENITA
DALLAS
CHI
BILLYE
ALEXANDER
TOMIKA
SHARITA
RANA
NIKOLE
NEOMA
MARGARITE
MADALYN
LUCINA
LAILA
KALI
JENETTE
GABRIELE
EVELYNE
ELENORA
CLEMENTINA
ALEJANDRINA
ZULEMA
VIOLETTE
VANNESSA
THRESA
RETTA
PIA
PATIENCE
NOELLA
NICKIE
JONELL
DELTA
CHUNG
CHAYA
CAMELIA
BETHEL
ANYA
ANDREW
THANH
SUZANN
SPRING
SHU
MILA
LILLA
LAVERNA
KEESHA
KATTIE
GIA
GEORGENE
EVELINE
ESTELL
ELIZBETH
VIVIENNE
VALLIE
TRUDIE
STEPHANE
MICHEL
MAGALY
MADIE
KENYETTA
KARREN
JANETTA
HERMINE
HARMONY
DRUCILLA
DEBBI
CELESTINA
CANDIE
BRITNI
BECKIE
AMINA
ZITA
YUN
YOLANDE
VIVIEN
VERNETTA
TRUDI
SOMMER
PEARLE
PATRINA
OSSIE
NICOLLE
LOYCE
LETTY
LARISA
KATHARINA
JOSELYN
JONELLE
JENELL
IESHA
HEIDE
FLORINDA
FLORENTINA
FLO
ELODIA
DORINE
BRUNILDA
BRIGID
ASHLI
ARDELLA
TWANA
THU
TARAH
SUNG
SHEA
SHAVON
SHANE
SERINA
RAYNA
RAMONITA
NGA
MARGURITE
LUCRECIA
KOURTNEY
KATI
JESUS
JESENIA
DIAMOND
CRISTA
AYANA
ALICA
ALIA
VINNIE
SUELLEN
ROMELIA
RACHELL
PIPER
OLYMPIA
MICHIKO
KATHALEEN
JOLIE
JESSI
JANESSA
HANA
HA
ELEASE
CARLETTA
BRITANY
SHONA
SALOME
ROSAMOND
REGENA
RAINA
NGOC
NELIA
LOUVENIA
LESIA
LATRINA
LATICIA
LARHONDA
JINA
JACKI
HOLLIS
HOLLEY
EMMY
DEEANN
CORETTA
ARNETTA
VELVET
THALIA
SHANICE
NETA
MIKKI
MICKI
LONNA
LEANA
LASHUNDA
KILEY
JOYE
JACQULYN
IGNACIA
HYUN
HIROKO
HENRY
HENRIETTE
ELAYNE
DELINDA
DARNELL
DAHLIA
COREEN
CONSUELA
CONCHITA
CELINE
BABETTE
AYANNA
ANETTE
ALBERTINA
SKYE
SHAWNEE
SHANEKA
QUIANA
PAMELIA
MIN
MERRI
MERLENE
MARGIT
KIESHA
KIERA
KAYLENE
JODEE
JENISE
ERLENE
EMMIE
ELSE
DARYL
DALILA
DAISEY
CODY
CASIE
BELIA
BABARA
VERSIE
VANESA
SHELBA
SHAWNDA
SAM
NORMAN
NIKIA
NAOMA
MARNA
MARGERET
MADALINE
LAWANA
KINDRA
JUTTA
JAZMINE
JANETT
HANNELORE
GLENDORA
GERTRUD
GARNETT
FREEDA
FREDERICA
FLORANCE
FLAVIA
DENNIS
CARLINE
BEVERLEE
ANJANETTE
VALDA
TRINITY
TAMALA
STEVIE
SHONNA
SHA
SARINA
ONEIDA
MICAH
MERILYN
MARLEEN
LURLINE
LENNA
KATHERIN
JIN
JENI
HAE
GRACIA
GLADY
FARAH
ERIC
ENOLA
EMA
DOMINQUE
DEVONA
DELANA
CECILA
CAPRICE
ALYSHA
ALI
ALETHIA
VENA
THERESIA
TAWNY
SONG
SHAKIRA
SAMARA
SACHIKO
RACHELE
PAMELLA
NICKY
MARNI
MARIEL
MAREN
MALISA
LIGIA
LERA
LATORIA
LARAE
KIMBER
KATHERN
KAREY
JENNEFER
JANETH
HALINA
FREDIA
DELISA
DEBROAH
CIERA
CHIN
ANGELIKA
ANDREE
ALTHA
YEN
VIVAN
TERRESA
TANNA
SUK
SUDIE
SOO
SIGNE
SALENA
RONNI
REBBECCA
MYRTIE
MCKENZIE
MALIKA
MAIDA
LOAN
LEONARDA
KAYLEIGH
FRANCE
ETHYL
ELLYN
DAYLE
CAMMIE
BRITTNI
BIRGIT
AVELINA
ASUNCION
ARIANNA
AKIKO
VENICE
TYESHA
TONIE
TIESHA
TAKISHA
STEFFANIE
SINDY
SANTANA
MEGHANN
MANDA
MACIE
LADY
KELLYE
KELLEE
JOSLYN
JASON
INGER
INDIRA
GLINDA
GLENNIS
FERNANDA
FAUSTINA
ENEIDA
ELICIA
DOT
DIGNA
DELL
ARLETTA
ANDRE
WILLIA
TAMMARA
TABETHA
SHERRELL
SARI
REFUGIO
REBBECA
PAULETTA
NIEVES
NATOSHA
NAKITA
MAMMIE
KENISHA
KAZUKO
KASSIE
GARY
EARLEAN
DAPHINE
CORLISS
CLOTILDE
CAROLYNE
BERNETTA
AUGUSTINA
AUDREA
ANNIS
ANNABELL
YAN
TENNILLE
TAMICA
SELENE
SEAN
ROSANA
REGENIA
QIANA
MARKITA
MACY
LEEANNE
LAURINE
KYM
JESSENIA
JANITA
GEORGINE
GENIE
EMIKO
ELVIE
DEANDRA
DAGMAR
CORIE
COLLEN
CHERISH
ROMAINE
PORSHA
PEARLENE
MICHELINE
MERNA
MARGORIE
MARGARETTA
LORE
KENNETH
JENINE
HERMINA
FREDERICKA
ELKE
DRUSILLA
DORATHY
DIONE
DESIRE
CELENA
BRIGIDA
ANGELES
ALLEGRA
THEO
TAMEKIA
SYNTHIA
STEPHEN
SOOK
SLYVIA
ROSANN
REATHA
RAYE
MARQUETTA
MARGART
LING
LAYLA
KYMBERLY
KIANA
KAYLEEN
KATLYN
KARMEN
JOELLA
IRINA
EMELDA
ELENI
DETRA
CLEMMIE
CHERYLL
CHANTELL
CATHEY
ARNITA
ARLA
ANGLE
ANGELIC
ALYSE
ZOFIA
THOMASINE
TENNIE
SON
SHERLY
SHERLEY
SHARYL
REMEDIOS
PETRINA
NICKOLE
MYUNG
MYRLE
MOZELLA
LOUANNE
LISHA
LATIA
LANE
KRYSTA
JULIENNE
JOEL
JEANENE
JACQUALINE
ISAURA
GWENDA
EARLEEN
DONALD
CLEOPATRA
CARLIE
AUDIE
ANTONIETTA
ALISE
ALEX
VERDELL
VAL
TYLER
TOMOKO
THAO
TALISHA
STEVEN
SO
SHEMIKA
SHAUN
SCARLET
SAVANNA
SANTINA
ROSIA
RAEANN
ODILIA
NANA
MINNA
MAGAN
LYNELLE
LE
KARMA
JOEANN
IVANA
INELL
ILANA
HYE
HONEY
HEE
GUDRUN
FRANK
DREAMA
CRISSY
CHANTE
CARMELINA
ARVILLA
ARTHUR
ANNAMAE
ALVERA
ALEIDA
AARON
YEE
YANIRA
VANDA
TIANNA
TAM
STEFANIA
SHIRA
PERRY
NICOL
NANCIE
MONSERRATE
MINH
MELYNDA
MELANY
MATTHEW
LOVELLA
LAURE
KIRBY
KACY
JACQUELYNN
HYON
GERTHA
FRANCISCO
ELIANA
CHRISTENA
CHRISTEEN
CHARISE
CATERINA
CARLEY
CANDYCE
ARLENA
AMMIE
YANG
WILLETTE
VANITA
TUYET
TINY
SYREETA
SILVA
SCOTT
RONALD
PENNEY
NYLA
MICHAL
MAURICE
MARYAM
MARYA
MAGEN
LUDIE
LOMA
LIVIA
LANELL
KIMBERLIE
JULEE
DONETTA
DIEDRA
DENISHA
DEANE
DAWNE
CLARINE
CHERRYL
BRONWYN
BRANDON
ALLA
VALERY
TONDA
SUEANN
SORAYA
SHOSHANA
SHELA
SHARLEEN
SHANELLE
NERISSA
MICHEAL
MERIDITH
MELLIE
MAYE
MAPLE
MAGARET
LUIS
LILI
LEONILA
LEONIE
LEEANNA
LAVONIA
LAVERA
KRISTEL
KATHEY
KATHE
JUSTIN
JULIAN
JIMMY
JANN
ILDA
HILDRED
HILDEGARDE
GENIA
FUMIKO
EVELIN
ERMELINDA
ELLY
DUNG
DOLORIS
DIONNA
DANAE
BERNEICE
ANNICE
ALIX
VERENA
VERDIE
TRISTAN
SHAWNNA
SHAWANA
SHAUNNA
ROZELLA
RANDEE
RANAE
MILAGRO
LYNELL
LUISE
LOUIE
LOIDA
LISBETH
KARLEEN
JUNITA
JONA
ISIS
HYACINTH
HEDY
GWENN
ETHELENE
ERLINE
EDWARD
DONYA
DOMONIQUE
DELICIA
DANNETTE
CICELY
BRANDA
BLYTHE
BETHANN
ASHLYN
ANNALEE
ALLINE
YUKO
VELLA
TRANG
TOWANDA
TESHA
SHERLYN
NARCISA
MIGUELINA
MERI
MAYBELL
MARLANA
MARGUERITA
MADLYN
LUNA
LORY
LORIANN
LIBERTY
LEONORE
LEIGHANN
LAURICE
LATESHA
LARONDA
KATRICE
KASIE
KARL
KALEY
JADWIGA
GLENNIE
GEARLDINE
FRANCINA
EPIFANIA
DYAN
DORIE
DIEDRE
DENESE
DEMETRICE
DELENA
DARBY
CRISTIE
CLEORA
CATARINA
CARISA
BERNIE
BARBERA
ALMETA
TRULA
TEREASA
SOLANGE
SHEILAH
SHAVONNE
SANORA
ROCHELL
MATHILDE
MARGARETA
MAIA
LYNSEY
LAWANNA
LAUNA
KENA
KEENA
KATIA
JAMEY
GLYNDA
GAYLENE
ELVINA
ELANOR
DANUTA
DANIKA
CRISTEN
CORDIE
COLETTA
CLARITA
CARMON
BRYNN
AZUCENA
AUNDREA
ANGELE
YI
WALTER
VERLIE
VERLENE
TAMESHA
SILVANA
SEBRINA
SAMIRA
REDA
RAYLENE
PENNI
PANDORA
NORAH
NOMA
MIREILLE
MELISSIA
MARYALICE
LARAINE
KIMBERY
KARYL
KARINE
KAM
JOLANDA
JOHANA
JESUSA
JALEESA
JAE
JACQUELYNE
IRISH
ILUMINADA
HILARIA
HANH
GENNIE
FRANCIE
FLORETTA
EXIE
EDDA
DREMA
DELPHA
BEV
BARBAR
ASSUNTA
ARDELL
ANNALISA
ALISIA
YUKIKO
YOLANDO
WONDA
WEI
WALTRAUD
VETA
TEQUILA
TEMEKA
TAMEIKA
SHIRLEEN
SHENITA
PIEDAD
OZELLA
MIRTHA
MARILU
KIMIKO
JULIANE
JENICE
JEN
JANAY
JACQUILINE
HILDE
FE
FAE
EVAN
EUGENE
ELOIS
ECHO
DEVORAH
CHAU
BRINDA
BETSEY
ARMINDA
ARACELIS
APRYL
ANNETT
ALISHIA
VEOLA
USHA
TOSHIKO
THEOLA
TASHIA
TALITHA
SHERY
RUDY
RENETTA
REIKO
RASHEEDA
OMEGA
OBDULIA
MIKA
MELAINE
MEGGAN
MARTIN
MARLEN
MARGET
MARCELINE
MANA
MAGDALEN
LIBRADA
LEZLIE
LEXIE
LATASHIA
LASANDRA
KELLE
ISIDRA
ISA
INOCENCIA
GWYN
FRANCOISE
ERMINIA
ERINN
DIMPLE
DEVORA
CRISELDA
ARMANDA
ARIE
ARIANE
ANGELO
ANGELENA
ALLEN
ALIZA
ADRIENE
ADALINE
XOCHITL
TWANNA
TRAN
TOMIKO
TAMISHA
TAISHA
SUSY
SIU
RUTHA
ROXY
RHONA
RAYMOND
OTHA
NORIKO
NATASHIA
MERRIE
MELVIN
MARINDA
MARIKO
MARGERT
LORIS
LIZZETTE
LEISHA
KAILA
KA
JOANNIE
JERRICA
JENE
JANNET
JANEE
JACINDA
HERTA
ELENORE
DORETTA
DELAINE
DANIELL
CLAUDIE
CHINA
BRITTA
APOLONIA
AMBERLY
ALEASE
YURI
YUK
WEN
WANETA
UTE
TOMI
SHARRI
SANDIE
ROSELLE
REYNALDA
RAGUEL
PHYLICIA
PATRIA
OLIMPIA
ODELIA
MITZIE
MITCHELL
MISS
MINDA
MIGNON
MICA
MENDY
MARIVEL
MAILE
LYNETTA
LAVETTE
LAURYN
LATRISHA
LAKIESHA
KIERSTEN
KARY
JOSPHINE
JOLYN
JETTA
JANISE
JACQUIE
IVELISSE
GLYNIS
GIANNA
GAYNELLE
EMERALD
DEMETRIUS
DANYELL
DANILLE
DACIA
CORALEE
CHER
CEOLA
BRETT
BELL
ARIANNE
ALESHIA
YUNG
WILLIEMAE
TROY
TRINH
THORA
TAI
SVETLANA
SHERIKA
SHEMEKA
SHAUNDA
ROSELINE
RICKI
MELDA
MALLIE
LAVONNA
LATINA
LARRY
LAQUANDA
LALA
LACHELLE
KLARA
KANDIS
JOHNA
JEANMARIE
JAYE
HANG
GRAYCE
GERTUDE
EMERITA
EBONIE
CLORINDA
CHING
CHERY
CAROLA
BREANN
BLOSSOM
BERNARDINE
BECKI
ARLETHA
ARGELIA
ARA
ALITA
YULANDA
YON
YESSENIA
TOBI
TASIA
SYLVIE
SHIRL
SHIRELY
SHERIDAN
SHELLA
SHANTELLE
SACHA
ROYCE
REBECKA
REAGAN
PROVIDENCIA
PAULENE
MISHA
MIKI
MARLINE
MARICA
LORITA
LATOYIA
LASONYA
KERSTIN
KENDA
KEITHA
KATHRIN
JAYMIE
JACK
GRICELDA
GINETTE
ERYN
ELINA
ELFRIEDA
DANYEL
CHEREE
CHANELLE
BARRIE
AVERY
AURORE
ANNAMARIA
ALLEEN
AILENE
AIDE
YASMINE
VASHTI
VALENTINE
TREASA
TORY
TIFFANEY
SHERYLL
SHARIE
SHANAE
SAU
RAISA
PA
NEDA
MITSUKO
MIRELLA
MILDA
MARYANNA
MARAGRET
MABELLE
LUETTA
LORINA
LETISHA
LATARSHA
LANELLE
LAJUANA
KRISSY
KARLY
KARENA
JON
JESSIKA
JERICA
JEANELLE
JANUARY
JALISA
JACELYN
IZOLA
IVEY
GREGORY
EUNA
ETHA
DREW
DOMITILA
DOMINICA
DAINA
CREOLA
CARLI
CAMIE
BUNNY
BRITTNY
ASHANTI
ANISHA
ALEEN
ADAH
YASUKO
WINTER
VIKI
VALRIE
TONA
TINISHA
THI
TERISA
TATUM
TANEKA
SIMONNE
SHALANDA
SERITA
RESSIE
REFUGIA
PAZ
OLENE
NA
MERRILL
MARGHERITA
MANDIE
MAN
MAIRE
LYNDIA
LUCI
LORRIANE
LORETA
LEONIA
LAVONA
LASHAWNDA
LAKIA
KYOKO
KRYSTINA
KRYSTEN
KENIA
KELSI
JUDE
JEANICE
ISOBEL
GEORGIANN
GENNY
FELICIDAD
EILENE
DEON
DELOISE
DEEDEE
DANNIE
CONCEPTION
CLORA
CHERILYN
CHANG
CALANDRA
BERRY
ARMANDINA
ANISA
ULA
TIMOTHY
TIERA
THERESSA
STEPHANIA
SIMA
SHYLA
SHONTA
SHERA
SHAQUITA
SHALA
SAMMY
ROSSANA
NOHEMI
NERY
MORIAH
MELITA
MELIDA
MELANI
MARYLYNN
MARISHA
MARIETTE
MALORIE
MADELENE
LUDIVINA
LORIA
LORETTE
LORALEE
LIANNE
LEON
LAVENIA
LAURINDA
LASHON
KIT
KIMI
KEILA
KATELYNN
KAI
JONE
JOANE
JI
JAYNA
JANELLA
JA
HUE
HERTHA
FRANCENE
ELINORE
DESPINA
DELSIE
DEEDRA
CLEMENCIA
CARRY
CAROLIN
CARLOS
BULAH
BRITTANIE
BOK
BLONDELL
BIBI
BEAULAH
BEATA
ANNITA
AGRIPINA
VIRGEN
VALENE
UN
TWANDA
TOMMYE
TOI
TARRA
TARI
TAMMERA
SHAKIA
SADYE
RUTHANNE
ROCHEL
RIVKA
PURA
NENITA
NATISHA
MING
MERRILEE
MELODEE
MARVIS
LUCILLA
LEENA
LAVETA
LARITA
LANIE
KEREN
ILEEN
GEORGEANN
GENNA
GENESIS
FRIDA
EWA
EUFEMIA
EMELY
ELA
EDYTH
DEONNA
DEADRA
DARLENA
CHANELL
CHAN
CATHERN
CASSONDRA
CASSAUNDRA
BERNARDA
BERNA
ARLINDA
ANAMARIA
ALBERT
WESLEY
VERTIE
VALERI
TORRI
TATYANA
STASIA
SHERISE
SHERILL
SEASON
SCOTTIE
SANDA
RUTHE
ROSY
ROBERTO
ROBBI
RANEE
QUYEN
PEARLY
PALMIRA
ONITA
NISHA
NIESHA
NIDA
NEVADA
NAM
MERLYN
MAYOLA
MARYLOUISE
MARYLAND
MARX
MARTH
MARGENE
MADELAINE
LONDA
LEONTINE
LEOMA
LEIA
LAWRENCE
LAURALEE
LANORA
LAKITA
KIYOKO
KETURAH
KATELIN
KAREEN
JONIE
JOHNETTE
JENEE
JEANETT
IZETTA
HIEDI
HEIKE
HASSIE
HAROLD
GIUSEPPINA
GEORGANN
FIDELA
FERNANDE
ELWANDA
ELLAMAE
ELIZ
DUSTI
DOTTY
CYNDY
CORALIE
CELESTA
ARGENTINA
ALVERTA
XENIA
WAVA
VANETTA
TORRIE
TASHINA
TANDY
TAMBRA
TAMA
STEPANIE
SHILA
SHAUNTA
SHARAN
SHANIQUA
SHAE
SETSUKO
SERAFINA
SANDEE
ROSAMARIA
PRISCILA
OLINDA
NADENE
MUOI
MICHELINA
MERCEDEZ
MARYROSE
MARIN
MARCENE
MAO
MAGALI
MAFALDA
LOGAN
LINN
LANNIE
KAYCE
KAROLINE
KAMILAH
KAMALA
JUSTA
JOLINE
JENNINE
JACQUETTA
IRAIDA
GERALD
GEORGEANNA
FRANCHESCA
FAIRY
EMELINE
ELANE
EHTEL
EARLIE
DULCIE
DALENE
CRIS
CLASSIE
CHERE
CHARIS
CAROYLN
CARMINA
CARITA
BRIAN
BETHANIE
AYAKO
ARICA
AN
ALYSA
ALESSANDRA
AKILAH
ADRIEN
ZETTA
YOULANDA
YELENA
YAHAIRA
XUAN
WENDOLYN
VICTOR
TIJUANA
TERRELL
TERINA
TERESIA
SUZI
SUNDAY
SHERELL
SHAVONDA
SHAUNTE
SHARDA
SHAKITA
SENA
RYANN
RUBI
RIVA
REGINIA
REA
RACHAL
PARTHENIA
PAMULA
MONNIE
MONET
MICHAELE
MELIA
MARINE
MALKA
MAISHA
LISANDRA
LEO
LEKISHA
LEAN
LAURENCE
LAKENDRA
KRYSTIN
KORTNEY
KIZZIE
KITTIE
KERA
KENDAL
KEMBERLY
KANISHA
JULENE
JULE
JOSHUA
JOHANNE
JEFFREY
JAMEE
HAN
HALLEY
GIDGET
GALINA
FREDRICKA
FLETA
FATIMAH
EUSEBIA
ELZA
ELEONORE
DORTHEY
DORIA
DONELLA
DINORAH
DELORSE
CLARETHA
CHRISTINIA
CHARLYN
BONG
BELKIS
AZZIE
ANDERA
AIKO
ADENA
YER
YAJAIRA
WAN
VANIA
ULRIKE
TOSHIA
TIFANY
STEFANY
SHIZUE
SHENIKA
SHAWANNA
SHAROLYN
SHARILYN
SHAQUANA
SHANTAY
SEE
ROZANNE
ROSELEE
RICKIE
REMONA
REANNA
RAELENE
QUINN
PHUNG
PETRONILA
NATACHA
NANCEY
MYRL
MIYOKO
MIESHA
MERIDETH
MARVELLA
MARQUITTA
MARHTA
MARCHELLE
LIZETH
LIBBIE
LAHOMA
LADAWN
KINA
KATHELEEN
KATHARYN
KARISA
KALEIGH
JUNIE
JULIEANN
JOHNSIE
JANEAN
JAIMEE
JACKQUELINE
HISAKO
HERMA
HELAINE
GWYNETH
GLENN
GITA
EUSTOLIA
EMELINA
ELIN
EDRIS
DONNETTE
DONNETTA
DIERDRE
DENAE
DARCEL
CLAUDE
CLARISA
CINDERELLA
CHIA
CHARLESETTA
CHARITA
CELSA
CASSY
CASSI
CARLEE
BRUNA
BRITTANEY
BRANDE
BILLI
BAO
ANTONETTA
ANGLA
ANGELYN
ANALISA
ALANE
WENONA
WENDIE
VERONIQUE
VANNESA
TOBIE
TEMPIE
SUMIKO
SULEMA
SPARKLE
SOMER
SHEBA
SHAYNE
SHARICE
SHANEL
SHALON
SAGE
ROY
ROSIO
ROSELIA
RENAY
REMA
REENA
PORSCHE
PING
PEG
OZIE
ORETHA
ORALEE
ODA
NU
NGAN
NAKESHA
MILLY
MARYBELLE
MARLIN
MARIS
MARGRETT
MARAGARET
MANIE
LURLENE
LILLIA
LIESELOTTE
LAVELLE
LASHAUNDA
LAKEESHA
KEITH
KAYCEE
KALYN
JOYA
JOETTE
JENAE
JANIECE
ILLA
GRISEL
GLAYDS
GENEVIE
GALA
FREDDA
FRED
ELMER
ELEONOR
DEBERA
DEANDREA
DAN
CORRINNE
CORDIA
CONTESSA
COLENE
CLEOTILDE
CHARLOTT
CHANTAY
CECILLE
BEATRIS
AZALEE
ARLEAN
ARDATH
ANJELICA
ANJA
ALFREDIA
ALEISHA
ADAM
ZADA
YUONNE
XIAO
WILLODEAN
WHITLEY
VENNIE
VANNA
TYISHA
TOVA
TORIE
TONISHA
TILDA
TIEN
TEMPLE
SIRENA
SHERRIL
SHANTI
SHAN
SENAIDA
SAMELLA
ROBBYN
RENDA
REITA
PHEBE
PAULITA
NOBUKO
NGUYET
NEOMI
MOON
MIKAELA
MELANIA
MAXIMINA
MARG
MAISIE
LYNNA
LILLI
LAYNE
LASHAUN
LAKENYA
LAEL
KIRSTIE
KATHLINE
KASHA
KARLYN
KARIMA
JOVAN
JOSEFINE
JENNELL
JACQUI
JACKELYN
HYO
HIEN
GRAZYNA
FLORRIE
FLORIA
ELEONORA
DWANA
DORLA
DONG
DELMY
DEJA
DEDE
DANN
CRYSTA
CLELIA
CLARIS
CLARENCE
CHIEKO
CHERLYN
CHERELLE
CHARMAIN
CHARA
CAMMY
BEE
ARNETTE
ARDELLE
ANNIKA
AMIEE
AMEE
ALLENA
YVONE
YUKI
YOSHIE
YEVETTE
YAEL
WILLETTA
VONCILE
VENETTA
TULA
TONETTE
TIMIKA
TEMIKA
TELMA
TEISHA
TAREN
TA
STACEE
SHIN
SHAWNTA
SATURNINA
RICARDA
POK
PASTY
ONIE
NUBIA
MORA
MIKE
MARIELLE
MARIELLA
MARIANELA
MARDELL
MANY
LUANNA
LOISE
LISABETH
LINDSY
LILLIANA
LILLIAM
LELAH
LEIGHA
LEANORA
LANG
KRISTEEN
KHALILAH
KEELEY
KANDRA
JUNKO
JOAQUINA
JERLENE
JANI
JAMIKA
JAME
HSIU
HERMILA
GOLDEN
GENEVIVE
EVIA
EUGENA
EMMALINE
ELFREDA
ELENE
DONETTE
DELCIE
DEEANNA
DARCEY
CUC
CLARINDA
CIRA
CHAE
CELINDA
CATHERYN
CATHERIN
CASIMIRA
CARMELIA
CAMELLIA
BREANA
BOBETTE
BERNARDINA
BEBE
BASILIA
ARLYNE
AMAL
ALAYNA
ZONIA
ZENIA
YURIKO
YAEKO
WYNELL
WILLOW
WILLENA
VERNIA
TU
TRAVIS
TORA
TERRILYN
TERICA
TENESHA
TAWNA
TAJUANA
TAINA
STEPHNIE
SONA
SOL
SINA
SHONDRA
SHIZUKO
SHERLENE
SHERICE
SHARIKA
ROSSIE
ROSENA
RORY
RIMA
RIA
RHEBA
RENNA
PETER
NATALYA
NANCEE
MELODI
MEDA
MAXIMA
MATHA
MARKETTA
MARICRUZ
MARCELENE
MALVINA
LUBA
LOUETTA
LEIDA
LECIA
LAURAN
LASHAWNA
LAINE
KHADIJAH
KATERINE
KASI
KALLIE
JULIETTA
JESUSITA
JESTINE
JESSIA
JEREMY
JEFFIE
JANYCE
ISADORA
GEORGIANNE
FIDELIA
EVITA
EURA
EULAH
ESTEFANA
ELSY
ELIZABET
ELADIA
DODIE
DION
DIA
DENISSE
DELORAS
DELILA
DAYSI
DAKOTA
CURTIS
CRYSTLE
CONCHA
COLBY
CLARETTA
CHU
CHRISTIA
CHARLSIE
CHARLENA
CARYLON
BETTYANN
ASLEY
ASHLEA
AMIRA
AI
AGUEDA
AGNUS
YUETTE
VINITA
VICTORINA
TYNISHA
TREENA
TOCCARA
TISH
THOMASENA
TEGAN
SOILA
SHILOH
SHENNA
SHARMAINE
SHANTAE
SHANDI
SEPTEMBER
SARAN
SARAI
SANA
SAMUEL
SALLEY
ROSETTE
ROLANDE
REGINE
OTELIA
OSCAR
OLEVIA
NICHOLLE
NECOLE
NAIDA
MYRTA
MYESHA
MITSUE
MINTA
MERTIE
MARGY
MAHALIA
MADALENE
LOVE
LOURA
LOREAN
LEWIS
LESHA
LEONIDA
LENITA
LAVONE
LASHELL
LASHANDRA
LAMONICA
KIMBRA
KATHERINA
KARRY
KANESHA
JULIO
JONG
JENEVA
JAQUELYN
HWA
GILMA
GHISLAINE
GERTRUDIS
FRANSISCA
FERMINA
ETTIE
ETSUKO
ELLIS
ELLAN
ELIDIA
EDRA
DORETHEA
DOREATHA
DENYSE
DENNY
DEETTA
DAINE
CYRSTAL
CORRIN
CAYLA
CARLITA
CAMILA
BURMA
BULA
BUENA
BLAKE
BARABARA
AVRIL
AUSTIN
ALAINE
ZANA
WILHEMINA
WANETTA
VIRGIL
VI
VERONIKA
VERNON
VERLINE
VASILIKI
TONITA
TISA
TEOFILA
TAYNA
TAUNYA
TANDRA
TAKAKO
SUNNI
SUANNE
SIXTA
SHARELL
SEEMA
RUSSELL
ROSENDA
ROBENA
RAYMONDE
PEI
PAMILA
OZELL
NEIDA
NEELY
MISTIE
MICHA
MERISSA
MAURITA
MARYLN
MARYETTA
MARSHALL
MARCELL
MALENA
MAKEDA
MADDIE
LOVETTA
LOURIE
LORRINE
LORILEE
LESTER
LAURENA
LASHAY
LARRAINE
LAREE
LACRESHA
KRISTLE
KRISHNA
KEVA
KEIRA
KAROLE
JOIE
JINNY
JEANNETTA
JAMA
HEIDY
GILBERTE
GEMA
FAVIOLA
EVELYNN
ENDA
ELLI
ELLENA
DIVINA
DAGNY
COLLENE
CODI
CINDIE
CHASSIDY
CHASIDY
CATRICE
CATHERINA
CASSEY
CAROLL
CARLENA
CANDRA
CALISTA
BRYANNA
BRITTENY
BEULA
BARI
AUDRIE
AUDRIA
ARDELIA
ANNELLE
ANGILA
ALONA
ALLYN
LI  |  http://en.wikipedia.org/wiki/List_of_most_common_surnames#China
WANG
ZHANG
LIU
CHEN
YANG
HUANG
ZHAO
ZHOU
WU
XU
SUN
ZHU
MA
HU
GUO
LIN
HE
GAO
LIANG'''
stopwords_names = stopwords_names.replace('\n',',')
stopwords_names = stopwords_names.replace(' ','')
stopwords_names = stopwords_names.replace('|',',')
stopwords_names = stopwords_names.split(',')
stopwords_names

['SMITH',
 'Surnamesfrom1990census>.002%.www.census.gov.genealogy/names/dist.all.last',
 'JOHNSON',
 'WILLIAMS',
 'JONES',
 'BROWN',
 'DAVIS',
 'MILLER',
 'WILSON',
 'MOORE',
 'TAYLOR',
 'ANDERSON',
 'THOMAS',
 'JACKSON',
 'WHITE',
 'HARRIS',
 'MARTIN',
 'THOMPSON',
 'GARCIA',
 'MARTINEZ',
 'ROBINSON',
 'CLARK',
 'RODRIGUEZ',
 'LEWIS',
 'LEE',
 'WALKER',
 'HALL',
 'ALLEN',
 'YOUNG',
 'HERNANDEZ',
 'KING',
 'WRIGHT',
 'LOPEZ',
 'HILL',
 'SCOTT',
 'GREEN',
 'ADAMS',
 'BAKER',
 'GONZALEZ',
 'NELSON',
 'CARTER',
 'MITCHELL',
 'PEREZ',
 'ROBERTS',
 'TURNER',
 'PHILLIPS',
 'CAMPBELL',
 'PARKER',
 'EVANS',
 'EDWARDS',
 'COLLINS',
 'STEWART',
 'SANCHEZ',
 'MORRIS',
 'ROGERS',
 'REED',
 'COOK',
 'MORGAN',
 'BELL',
 'MURPHY',
 'BAILEY',
 'RIVERA',
 'COOPER',
 'RICHARDSON',
 'COX',
 'HOWARD',
 'WARD',
 'TORRES',
 'PETERSON',
 'GRAY',
 'RAMIREZ',
 'JAMES',
 'WATSON',
 'BROOKS',
 'KELLY',
 'SANDERS',
 'PRICE',
 'BENNETT',
 'WOOD',
 'BARNES',
 'ROSS',
 'HENDERSON',
 'COLEMAN',
 'JENKINS',
 'PERRY',


In [10]:
stopwords =  stopwords_audit + stopwords_cuurencies + stopwords_date_num + stopwords_names + stopwords_geographic + stopwords_generic_long + stopwords_generic
stopwords

['ERNST',
 'YOUNG',
 'DELOITTE',
 'TOUCHE',
 'KPMG',
 'PRICEWATERHOUSECOOPERS',
 'PRICEWATERHOUSE',
 'COOPERS',
 'AFGHANI',
 'Afghanistan',
 'ARIARY',
 'Madagascar',
 'BAHT',
 'Thailand',
 'BALBOA',
 'Panama',
 'BIRR',
 'Ethiopia',
 'BOLIVAR',
 'Venezuela',
 'BOLIVIANO',
 'Bolivia',
 'CEDI',
 'Ghana',
 'COLON',
 'CostaRica',
 'CÓRDOBA',
 'Nicaragua',
 'DALASI',
 'Gambia',
 'DENAR',
 'Macedonia(FormerYug.Rep.)',
 'DINAR',
 'Algeria',
 'DIRHAM',
 'Morocco',
 'DOBRA',
 'SãoTomandPríncipe',
 'DONG',
 'Vietnam',
 'DRAM',
 'Armenia',
 'ESCUDO',
 'CapeVerde',
 'EURO',
 'Belgium',
 'FLORIN',
 'Aruba',
 'FORINT',
 'Hungary',
 'GOURDE',
 'Haiti',
 'GUARANI',
 'Paraguay',
 'GULDEN',
 'NetherlandsAntilles',
 'HRYVNIA',
 'Ukraine',
 'KINA',
 'PapuaNewGuinea',
 'KIP',
 'Laos',
 'KONVERTIBILNAMARKA',
 'Bosnia-Herzegovina',
 'KORUNA',
 'CzechRepublic',
 'KRONA',
 'Sweden',
 'KRONE',
 'Denmark',
 'KROON',
 'Estonia',
 'KUNA',
 'Croatia',
 'KWACHA',
 'Zambia',
 'KWANZA',
 'Angola',
 'KYAT',
 'Myanmar',


In [11]:
positive_words = '''a+
abound
abounds
abundance
abundant
accessable
accessible
acclaim
acclaimed
acclamation
accolade
accolades
accommodative
accomodative
accomplish
accomplished
accomplishment
accomplishments
accurate
accurately
achievable
achievement
achievements
achievible
acumen
adaptable
adaptive
adequate
adjustable
admirable
admirably
admiration
admire
admirer
admiring
admiringly
adorable
adore
adored
adorer
adoring
adoringly
adroit
adroitly
adulate
adulation
adulatory
advanced
advantage
advantageous
advantageously
advantages
adventuresome
adventurous
advocate
advocated
advocates
affability
affable
affably
affectation
affection
affectionate
affinity
affirm
affirmation
affirmative
affluence
affluent
afford
affordable
affordably
afordable
agile
agilely
agility
agreeable
agreeableness
agreeably
all-around
alluring
alluringly
altruistic
altruistically
amaze
amazed
amazement
amazes
amazing
amazingly
ambitious
ambitiously
ameliorate
amenable
amenity
amiability
amiabily
amiable
amicability
amicable
amicably
amity
ample
amply
amuse
amusing
amusingly
angel
angelic
apotheosis
appeal
appealing
applaud
appreciable
appreciate
appreciated
appreciates
appreciative
appreciatively
appropriate
approval
approve
ardent
ardently
ardor
articulate
aspiration
aspirations
aspire
assurance
assurances
assure
assuredly
assuring
astonish
astonished
astonishing
astonishingly
astonishment
astound
astounded
astounding
astoundingly
astutely
attentive
attraction
attractive
attractively
attune
audible
audibly
auspicious
authentic
authoritative
autonomous
available
aver
avid
avidly
award
awarded
awards
awe
awed
awesome
awesomely
awesomeness
awestruck
awsome
backbone
balanced
bargain
beauteous
beautiful
beautifullly
beautifully
beautify
beauty
beckon
beckoned
beckoning
beckons
believable
believeable
beloved
benefactor
beneficent
beneficial
beneficially
beneficiary
benefit
benefits
benevolence
benevolent
benifits
best
best-known
best-performing
best-selling
better
better-known
better-than-expected
beutifully
blameless
bless
blessing
bliss
blissful
blissfully
blithe
blockbuster
bloom
blossom
bolster
bonny
bonus
bonuses
boom
booming
boost
boundless
bountiful
brainiest
brainy
brand-new
brave
bravery
bravo
breakthrough
breakthroughs
breathlessness
breathtaking
breathtakingly
breeze
bright
brighten
brighter
brightest
brilliance
brilliances
brilliant
brilliantly
brisk
brotherly
bullish
buoyant
cajole
calm
calming
calmness
capability
capable
capably
captivate
captivating
carefree
cashback
cashbacks
catchy
celebrate
celebrated
celebration
celebratory
champ
champion
charisma
charismatic
charitable
charm
charming
charmingly
chaste
cheaper
cheapest
cheer
cheerful
cheery
cherish
cherished
cherub
chic
chivalrous
chivalry
civility
civilize
clarity
classic
classy
clean
cleaner
cleanest
cleanliness
cleanly
clear
clear-cut
cleared
clearer
clearly
clears
clever
cleverly
cohere
coherence
coherent
cohesive
colorful
comely
comfort
comfortable
comfortably
comforting
comfy
commend
commendable
commendably
commitment
commodious
compact
compactly
compassion
compassionate
compatible
competitive
complement
complementary
complemented
complements
compliant
compliment
complimentary
comprehensive
conciliate
conciliatory
concise
confidence
confident
congenial
congratulate
congratulation
congratulations
congratulatory
conscientious
considerate
consistent
consistently
constructive
consummate
contentment
continuity
contrasty
contribution
convenience
convenient
conveniently
convience
convienient
convient
convincing
convincingly
cool
coolest
cooperative
cooperatively
cornerstone
correct
correctly
cost-effective
cost-saving
counter-attack
counter-attacks
courage
courageous
courageously
courageousness
courteous
courtly
covenant
cozy
creative
credence
credible
crisp
crisper
cure
cure-all
cushy
cute
cuteness
danke
danken
daring
daringly
darling
dashing
dauntless
dawn
dazzle
dazzled
dazzling
dead-cheap
dead-on
decency
decent
decisive
decisiveness
dedicated
defeat
defeated
defeating
defeats
defender
deference
deft
deginified
delectable
delicacy
delicate
delicious
delight
delighted
delightful
delightfully
delightfulness
dependable
dependably
deservedly
deserving
desirable
desiring
desirous
destiny
detachable
devout
dexterous
dexterously
dextrous
dignified
dignify
dignity
diligence
diligent
diligently
diplomatic
dirt-cheap
distinction
distinctive
distinguished
diversified
divine
divinely
dominate
dominated
dominates
dote
dotingly
doubtless
dreamland
dumbfounded
dumbfounding
dummy-proof
durable
dynamic
eager
eagerly
eagerness
earnest
earnestly
earnestness
ease
eased
eases
easier
easiest
easiness
easing
easy
easy-to-use
easygoing
ebullience
ebullient
ebulliently
ecenomical
economical
ecstasies
ecstasy
ecstatic
ecstatically
edify
educated
effective
effectively
effectiveness
effectual
efficacious
efficient
efficiently
effortless
effortlessly
effusion
effusive
effusively
effusiveness
elan
elate
elated
elatedly
elation
electrify
elegance
elegant
elegantly
elevate
elite
eloquence
eloquent
eloquently
embolden
eminence
eminent
empathize
empathy
empower
empowerment
enchant
enchanted
enchanting
enchantingly
encourage
encouragement
encouraging
encouragingly
endear
endearing
endorse
endorsed
endorsement
endorses
endorsing
energetic
energize
energy-efficient
energy-saving
engaging
engrossing
enhance
enhanced
enhancement
enhances
enjoy
enjoyable
enjoyably
enjoyed
enjoying
enjoyment
enjoys
enlighten
enlightenment
enliven
ennoble
enough
enrapt
enrapture
enraptured
enrich
enrichment
enterprising
entertain
entertaining
entertains
enthral
enthrall
enthralled
enthuse
enthusiasm
enthusiast
enthusiastic
enthusiastically
entice
enticed
enticing
enticingly
entranced
entrancing
entrust
enviable
enviably
envious
enviously
enviousness
envy
equitable
ergonomical
err-free
erudite
ethical
eulogize
euphoria
euphoric
euphorically
evaluative
evenly
eventful
everlasting
evocative
exalt
exaltation
exalted
exaltedly
exalting
exaltingly
examplar
examplary
excallent
exceed
exceeded
exceeding
exceedingly
exceeds
excel
exceled
excelent
excellant
excelled
excellence
excellency
excellent
excellently
excels
exceptional
exceptionally
excite
excited
excitedly
excitedness
excitement
excites
exciting
excitingly
exellent
exemplar
exemplary
exhilarate
exhilarating
exhilaratingly
exhilaration
exonerate
expansive
expeditiously
expertly
exquisite
exquisitely
extol
extoll
extraordinarily
extraordinary
exuberance
exuberant
exuberantly
exult
exultant
exultation
exultingly
eye-catch
eye-catching
eyecatch
eyecatching
fabulous
fabulously
facilitate
fair
fairly
fairness
faith
faithful
faithfully
faithfulness
fame
famed
famous
famously
fancier
fancinating
fancy
fanfare
fans
fantastic
fantastically
fascinate
fascinating
fascinatingly
fascination
fashionable
fashionably
fast
fast-growing
fast-paced
faster
fastest
fastest-growing
faultless
fav
fave
favor
favorable
favored
favorite
favorited
favour
fearless
fearlessly
feasible
feasibly
feat
feature-rich
fecilitous
feisty
felicitate
felicitous
felicity
fertile
fervent
fervently
fervid
fervidly
fervor
festive
fidelity
fiery
fine
fine-looking
finely
finer
finest
firmer
first-class
first-in-class
first-rate
flashy
flatter
flattering
flatteringly
flawless
flawlessly
flexibility
flexible
flourish
flourishing
fluent
flutter
fond
fondly
fondness
foolproof
foremost
foresight
formidable
fortitude
fortuitous
fortuitously
fortunate
fortunately
fortune
fragrant
free
freed
freedom
freedoms
fresh
fresher
freshest
friendliness
friendly
frolic
frugal
fruitful
ftw
fulfillment
fun
futurestic
futuristic
gaiety
gaily
gain
gained
gainful
gainfully
gaining
gains
gallant
gallantly
galore
geekier
geeky
gem
gems
generosity
generous
generously
genial
genius
gentle
gentlest
genuine
gifted
glad
gladden
gladly
gladness
glamorous
glee
gleeful
gleefully
glimmer
glimmering
glisten
glistening
glitter
glitz
glorify
glorious
gloriously
glory
glow
glowing
glowingly
god-given
god-send
godlike
godsend
gold
golden
good
goodly
goodness
goodwill
goood
gooood
gorgeous
gorgeously
grace
graceful
gracefully
gracious
graciously
graciousness
grand
grandeur
grateful
gratefully
gratification
gratified
gratifies
gratify
gratifying
gratifyingly
gratitude
great
greatest
greatness
grin
groundbreaking
guarantee
guidance
guiltless
gumption
gush
gusto
gutsy
hail
halcyon
hale
hallmark
hallmarks
hallowed
handier
handily
hands-down
handsome
handsomely
handy
happier
happily
happiness
happy
hard-working
hardier
hardy
harmless
harmonious
harmoniously
harmonize
harmony
headway
heal
healthful
healthy
hearten
heartening
heartfelt
heartily
heartwarming
heaven
heavenly
helped
helpful
helping
hero
heroic
heroically
heroine
heroize
heros
high-quality
high-spirited
hilarious
holy
homage
honest
honesty
honor
honorable
honored
honoring
hooray
hopeful
hospitable
hot
hotcake
hotcakes
hottest
hug
humane
humble
humility
humor
humorous
humorously
humour
humourous
ideal
idealize
ideally
idol
idolize
idolized
idyllic
illuminate
illuminati
illuminating
illumine
illustrious
ilu
imaculate
imaginative
immaculate
immaculately
immense
impartial
impartiality
impartially
impassioned
impeccable
impeccably
important
impress
impressed
impresses
impressive
impressively
impressiveness
improve
improved
improvement
improvements
improves
improving
incredible
incredibly
indebted
individualized
indulgence
indulgent
industrious
inestimable
inestimably
inexpensive
infallibility
infallible
infallibly
influential
ingenious
ingeniously
ingenuity
ingenuous
ingenuously
innocuous
innovation
innovative
inpressed
insightful
insightfully
inspiration
inspirational
inspire
inspiring
instantly
instructive
instrumental
integral
integrated
intelligence
intelligent
intelligible
interesting
interests
intimacy
intimate
intricate
intrigue
intriguing
intriguingly
intuitive
invaluable
invaluablely
inventive
invigorate
invigorating
invincibility
invincible
inviolable
inviolate
invulnerable
irreplaceable
irreproachable
irresistible
irresistibly
issue-free
jaw-droping
jaw-dropping
jollify
jolly
jovial
joy
joyful
joyfully
joyous
joyously
jubilant
jubilantly
jubilate
jubilation
jubiliant
judicious
justly
keen
keenly
keenness
kid-friendly
kindliness
kindly
kindness
knowledgeable
kudos
large-capacity
laud
laudable
laudably
lavish
lavishly
law-abiding
lawful
lawfully
lead
leading
leads
lean
led
legendary
leverage
levity
liberate
liberation
liberty
lifesaver
light-hearted
lighter
likable
like
liked
likes
liking
lionhearted
lively
logical
long-lasting
lovable
lovably
love
loved
loveliness
lovely
lover
loves
loving
low-cost
low-price
low-priced
low-risk
lower-priced
loyal
loyalty
lucid
lucidly
luck
luckier
luckiest
luckiness
lucky
lucrative
luminous
lush
luster
lustrous
luxuriant
luxuriate
luxurious
luxuriously
luxury
lyrical
magic
magical
magnanimous
magnanimously
magnificence
magnificent
magnificently
majestic
majesty
manageable
maneuverable
marvel
marveled
marvelled
marvellous
marvelous
marvelously
marvelousness
marvels
master
masterful
masterfully
masterpiece
masterpieces
masters
mastery
matchless
mature
maturely
maturity
meaningful
memorable
merciful
mercifully
mercy
merit
meritorious
merrily
merriment
merriness
merry
mesmerize
mesmerized
mesmerizes
mesmerizing
mesmerizingly
meticulous
meticulously
mightily
mighty
mind-blowing
miracle
miracles
miraculous
miraculously
miraculousness
modern
modest
modesty
momentous
monumental
monumentally
morality
motivated
multi-purpose
navigable
neat
neatest
neatly
nice
nicely
nicer
nicest
nifty
nimble
noble
nobly
noiseless
non-violence
non-violent
notably
noteworthy
nourish
nourishing
nourishment
novelty
nurturing
oasis
obsession
obsessions
obtainable
openly
openness
optimal
optimism
optimistic
opulent
orderly
originality
outdo
outdone
outperform
outperformed
outperforming
outperforms
outshine
outshone
outsmart
outstanding
outstandingly
outstrip
outwit
ovation
overjoyed
overtake
overtaken
overtakes
overtaking
overtook
overture
pain-free
painless
painlessly
palatial
pamper
pampered
pamperedly
pamperedness
pampers
panoramic
paradise
paramount
pardon
passion
passionate
passionately
patience
patient
patiently
patriot
patriotic
peace
peaceable
peaceful
peacefully
peacekeepers
peach
peerless
pep
pepped
pepping
peppy
peps
perfect
perfection
perfectly
permissible
perseverance
persevere
personages
personalized
phenomenal
phenomenally
picturesque
piety
pinnacle
playful
playfully
pleasant
pleasantly
pleased
pleases
pleasing
pleasingly
pleasurable
pleasurably
pleasure
plentiful
pluses
plush
plusses
poetic
poeticize
poignant
poise
poised
polished
polite
politeness
popular
portable
posh
positive
positively
positives
powerful
powerfully
praise
praiseworthy
praising
pre-eminent
precious
precise
precisely
preeminent
prefer
preferable
preferably
prefered
preferes
preferring
prefers
premier
prestige
prestigious
prettily
pretty
priceless
pride
principled
privilege
privileged
prize
proactive
problem-free
problem-solver
prodigious
prodigiously
prodigy
productive
productively
proficient
proficiently
profound
profoundly
profuse
profusion
progress
progressive
prolific
prominence
prominent
promise
promised
promises
promising
promoter
prompt
promptly
proper
properly
propitious
propitiously
pros
prosper
prosperity
prosperous
prospros
protect
protection
protective
proud
proven
proves
providence
proving
prowess
prudence
prudent
prudently
punctual
pure
purify
purposeful
quaint
qualified
qualify
quicker
quiet
quieter
radiance
radiant
rapid
rapport
rapt
rapture
raptureous
raptureously
rapturous
rapturously
rational
razor-sharp
reachable
readable
readily
ready
reaffirm
reaffirmation
realistic
realizable
reasonable
reasonably
reasoned
reassurance
reassure
receptive
reclaim
recomend
recommend
recommendation
recommendations
recommended
reconcile
reconciliation
record-setting
recover
recovery
rectification
rectify
rectifying
redeem
redeeming
redemption
refine
refined
refinement
reform
reformed
reforming
reforms
refresh
refreshed
refreshing
refund
refunded
regal
regally
regard
rejoice
rejoicing
rejoicingly
rejuvenate
rejuvenated
rejuvenating
relaxed
relent
reliable
reliably
relief
relish
remarkable
remarkably
remedy
remission
remunerate
renaissance
renewed
renown
renowned
replaceable
reputable
reputation
resilient
resolute
resound
resounding
resourceful
resourcefulness
respect
respectable
respectful
respectfully
respite
resplendent
responsibly
responsive
restful
restored
restructure
restructured
restructuring
retractable
revel
revelation
revere
reverence
reverent
reverently
revitalize
revival
revive
revives
revolutionary
revolutionize
revolutionized
revolutionizes
reward
rewarding
rewardingly
rich
richer
richly
richness
right
righten
righteous
righteously
righteousness
rightful
rightfully
rightly
rightness
risk-free
robust
rock-star
rock-stars
rockstar
rockstars
romantic
romantically
romanticize
roomier
roomy
rosy
safe
safely
sagacity
sagely
saint
saintliness
saintly
salutary
salute
sane
satisfactorily
satisfactory
satisfied
satisfies
satisfy
satisfying
satisified
saver
savings
savior
savvy
scenic
seamless
seasoned
secure
securely
selective
self-determination
self-respect
self-satisfaction
self-sufficiency
self-sufficient
sensation
sensational
sensationally
sensations
sensible
sensibly
sensitive
serene
serenity
sexy
sharp
sharper
sharpest
shimmering
shimmeringly
shine
shiny
significant
silent
simpler
simplest
simplified
simplifies
simplify
simplifying
sincere
sincerely
sincerity
skill
skilled
skillful
skillfully
slammin
sleek
slick
smart
smarter
smartest
smartly
smile
smiles
smiling
smilingly
smitten
smooth
smoother
smoothes
smoothest
smoothly
snappy
snazzy
sociable
soft
softer
solace
solicitous
solicitously
solid
solidarity
soothe
soothingly
sophisticated
soulful
soundly
soundness
spacious
sparkle
sparkling
spectacular
spectacularly
speedily
speedy
spellbind
spellbinding
spellbindingly
spellbound
spirited
spiritual
splendid
splendidly
splendor
spontaneous
sporty
spotless
sprightly
stability
stabilize
stable
stainless
standout
state-of-the-art
stately
statuesque
staunch
staunchly
staunchness
steadfast
steadfastly
steadfastness
steadiest
steadiness
steady
stellar
stellarly
stimulate
stimulates
stimulating
stimulative
stirringly
straighten
straightforward
streamlined
striking
strikingly
striving
strong
stronger
strongest
stunned
stunning
stunningly
stupendous
stupendously
sturdier
sturdy
stylish
stylishly
stylized
suave
suavely
sublime
subsidize
subsidized
subsidizes
subsidizing
substantive
succeed
succeeded
succeeding
succeeds
succes
success
successes
successful
successfully
suffice
sufficed
suffices
sufficient
sufficiently
suitable
sumptuous
sumptuously
sumptuousness
super
superb
superbly
superior
superiority
supple
support
supported
supporter
supporting
supportive
supports
supremacy
supreme
supremely
supurb
supurbly
surmount
surpass
surreal
survival
survivor
sustainability
sustainable
swank
swankier
swankiest
swanky
sweeping
sweet
sweeten
sweetheart
sweetly
sweetness
swift
swiftness
talent
talented
talents
tantalize
tantalizing
tantalizingly
tempt
tempting
temptingly
tenacious
tenaciously
tenacity
tender
tenderly
terrific
terrifically
thank
thankful
thinner
thoughtful
thoughtfully
thoughtfulness
thrift
thrifty
thrill
thrilled
thrilling
thrillingly
thrills
thrive
thriving
thumb-up
thumbs-up
tickle
tidy
time-honored
timely
tingle
titillate
titillating
titillatingly
togetherness
tolerable
toll-free
top
top-notch
top-quality
topnotch
tops
tough
tougher
toughest
traction
tranquil
tranquility
transparent
treasure
tremendously
trendy
triumph
triumphal
triumphant
triumphantly
trivially
trophy
trouble-free
trump
trumpet
trust
trusted
trusting
trustingly
trustworthiness
trustworthy
trusty
truthful
truthfully
truthfulness
twinkly
ultra-crisp
unabashed
unabashedly
unaffected
unassailable
unbeatable
unbiased
unbound
uncomplicated
unconditional
undamaged
undaunted
understandable
undisputable
undisputably
undisputed
unencumbered
unequivocal
unequivocally
unfazed
unfettered
unforgettable
unity
unlimited
unmatched
unparalleled
unquestionable
unquestionably
unreal
unrestricted
unrivaled
unselfish
unwavering
upbeat
upgradable
upgradeable
upgraded
upheld
uphold
uplift
uplifting
upliftingly
upliftment
upscale
usable
useable
useful
user-friendly
user-replaceable
valiant
valiantly
valor
valuable
variety
venerate
verifiable
veritable
versatile
versatility
vibrant
vibrantly
victorious
victory
viewable
vigilance
vigilant
virtue
virtuous
virtuously
visionary
vivacious
vivid
vouch
vouchsafe
warm
warmer
warmhearted
warmly
warmth
wealthy
welcome
well
well-backlit
well-balanced
well-behaved
well-being
well-bred
well-connected
well-educated
well-established
well-informed
well-intentioned
well-known
well-made
well-managed
well-mannered
well-positioned
well-received
well-regarded
well-rounded
well-run
well-wishers
wellbeing
whoa
wholeheartedly
wholesome
whooa
whoooa
wieldy
willing
willingly
willingness
win
windfall
winnable
winner
winners
winning
wins
wisdom
wise
wisely
witty
won
wonder
wonderful
wonderfully
wonderous
wonderously
wonders
wondrous
woo
work
workable
worked
works
world-famous
worth
worth-while
worthiness
worthwhile
worthy
wow
wowed
wowing
wows
yay
youthful
zeal
zenith
zest
zippy'''
positive_words = positive_words.replace('\n',',')
positive_words = positive_words.split(',')
positive_words

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [12]:
negative_words = '''2-faced
2-faces
abnormal
abolish
abominable
abominably
abominate
abomination
abort
aborted
aborts
abrade
abrasive
abrupt
abruptly
abscond
absence
absent-minded
absentee
absurd
absurdity
absurdly
absurdness
abuse
abused
abuses
abusive
abysmal
abysmally
abyss
accidental
accost
accursed
accusation
accusations
accuse
accuses
accusing
accusingly
acerbate
acerbic
acerbically
ache
ached
aches
achey
aching
acrid
acridly
acridness
acrimonious
acrimoniously
acrimony
adamant
adamantly
addict
addicted
addicting
addicts
admonish
admonisher
admonishingly
admonishment
admonition
adulterate
adulterated
adulteration
adulterier
adversarial
adversary
adverse
adversity
afflict
affliction
afflictive
affront
afraid
aggravate
aggravating
aggravation
aggression
aggressive
aggressiveness
aggressor
aggrieve
aggrieved
aggrivation
aghast
agonies
agonize
agonizing
agonizingly
agony
aground
ail
ailing
ailment
aimless
alarm
alarmed
alarming
alarmingly
alienate
alienated
alienation
allegation
allegations
allege
allergic
allergies
allergy
aloof
altercation
ambiguity
ambiguous
ambivalence
ambivalent
ambush
amiss
amputate
anarchism
anarchist
anarchistic
anarchy
anemic
anger
angrily
angriness
angry
anguish
animosity
annihilate
annihilation
annoy
annoyance
annoyances
annoyed
annoying
annoyingly
annoys
anomalous
anomaly
antagonism
antagonist
antagonistic
antagonize
anti-
anti-american
anti-israeli
anti-occupation
anti-proliferation
anti-semites
anti-social
anti-us
anti-white
antipathy
antiquated
antithetical
anxieties
anxiety
anxious
anxiously
anxiousness
apathetic
apathetically
apathy
apocalypse
apocalyptic
apologist
apologists
appal
appall
appalled
appalling
appallingly
apprehension
apprehensions
apprehensive
apprehensively
arbitrary
arcane
archaic
arduous
arduously
argumentative
arrogance
arrogant
arrogantly
ashamed
asinine
asininely
asinininity
askance
asperse
aspersion
aspersions
assail
assassin
assassinate
assault
assult
astray
asunder
atrocious
atrocities
atrocity
atrophy
attack
attacks
audacious
audaciously
audaciousness
audacity
audiciously
austere
authoritarian
autocrat
autocratic
avalanche
avarice
avaricious
avariciously
avenge
averse
aversion
aweful
awful
awfully
awfulness
awkward
awkwardness
ax
babble
back-logged
back-wood
back-woods
backache
backaches
backaching
backbite
backbiting
backward
backwardness
backwood
backwoods
bad
badly
baffle
baffled
bafflement
baffling
bait
balk
banal
banalize
bane
banish
banishment
bankrupt
barbarian
barbaric
barbarically
barbarity
barbarous
barbarously
barren
baseless
bash
bashed
bashful
bashing
bastard
bastards
battered
battering
batty
bearish
beastly
bedlam
bedlamite
befoul
beg
beggar
beggarly
begging
beguile
belabor
belated
beleaguer
belie
belittle
belittled
belittling
bellicose
belligerence
belligerent
belligerently
bemoan
bemoaning
bemused
bent
berate
bereave
bereavement
bereft
berserk
beseech
beset
besiege
besmirch
bestial
betray
betrayal
betrayals
betrayer
betraying
betrays
bewail
beware
bewilder
bewildered
bewildering
bewilderingly
bewilderment
bewitch
bias
biased
biases
bicker
bickering
bid-rigging
bigotries
bigotry
bitch
bitchy
biting
bitingly
bitter
bitterly
bitterness
bizarre
blab
blabber
blackmail
blah
blame
blameworthy
bland
blandish
blaspheme
blasphemous
blasphemy
blasted
blatant
blatantly
blather
bleak
bleakly
bleakness
bleed
bleeding
bleeds
blemish
blind
blinding
blindingly
blindside
blister
blistering
bloated
blockage
blockhead
bloodshed
bloodthirsty
bloody
blotchy
blow
blunder
blundering
blunders
blunt
blur
bluring
blurred
blurring
blurry
blurs
blurt
boastful
boggle
bogus
boil
boiling
boisterous
bomb
bombard
bombardment
bombastic
bondage
bonkers
bore
bored
boredom
bores
boring
botch
bother
bothered
bothering
bothers
bothersome
bowdlerize
boycott
braggart
bragger
brainless
brainwash
brash
brashly
brashness
brat
bravado
brazen
brazenly
brazenness
breach
break
break-up
break-ups
breakdown
breaking
breaks
breakup
breakups
bribery
brimstone
bristle
brittle
broke
broken
broken-hearted
brood
browbeat
bruise
bruised
bruises
bruising
brusque
brutal
brutalising
brutalities
brutality
brutalize
brutalizing
brutally
brute
brutish
bs
buckle
bug
bugging
buggy
bugs
bulkier
bulkiness
bulky
bulkyness
bull****
bull----
bullies
bullshit
bullshyt
bully
bullying
bullyingly
bum
bump
bumped
bumping
bumpping
bumps
bumpy
bungle
bungler
bungling
bunk
burden
burdensome
burdensomely
burn
burned
burning
burns
bust
busts
busybody
butcher
butchery
buzzing
byzantine
cackle
calamities
calamitous
calamitously
calamity
callous
calumniate
calumniation
calumnies
calumnious
calumniously
calumny
cancer
cancerous
cannibal
cannibalize
capitulate
capricious
capriciously
capriciousness
capsize
careless
carelessness
caricature
carnage
carp
cartoonish
cash-strapped
castigate
castrated
casualty
cataclysm
cataclysmal
cataclysmic
cataclysmically
catastrophe
catastrophes
catastrophic
catastrophically
catastrophies
caustic
caustically
cautionary
cave
censure
chafe
chaff
chagrin
challenging
chaos
chaotic
chasten
chastise
chastisement
chatter
chatterbox
cheap
cheapen
cheaply
cheat
cheated
cheater
cheating
cheats
checkered
cheerless
cheesy
chide
childish
chill
chilly
chintzy
choke
choleric
choppy
chore
chronic
chunky
clamor
clamorous
clash
cliche
cliched
clique
clog
clogged
clogs
cloud
clouding
cloudy
clueless
clumsy
clunky
coarse
cocky
coerce
coercion
coercive
cold
coldly
collapse
collude
collusion
combative
combust
comical
commiserate
commonplace
commotion
commotions
complacent
complain
complained
complaining
complains
complaint
complaints
complex
complicated
complication
complicit
compulsion
compulsive
concede
conceded
conceit
conceited
concen
concens
concern
concerned
concerns
concession
concessions
condemn
condemnable
condemnation
condemned
condemns
condescend
condescending
condescendingly
condescension
confess
confession
confessions
confined
conflict
conflicted
conflicting
conflicts
confound
confounded
confounding
confront
confrontation
confrontational
confuse
confused
confuses
confusing
confusion
confusions
congested
congestion
cons
conscons
conservative
conspicuous
conspicuously
conspiracies
conspiracy
conspirator
conspiratorial
conspire
consternation
contagious
contaminate
contaminated
contaminates
contaminating
contamination
contempt
contemptible
contemptuous
contemptuously
contend
contention
contentious
contort
contortions
contradict
contradiction
contradictory
contrariness
contravene
contrive
contrived
controversial
controversy
convoluted
corrode
corrosion
corrosions
corrosive
corrupt
corrupted
corrupting
corruption
corrupts
corruptted
costlier
costly
counter-productive
counterproductive
coupists
covetous
coward
cowardly
crabby
crack
cracked
cracks
craftily
craftly
crafty
cramp
cramped
cramping
cranky
crap
crappy
craps
crash
crashed
crashes
crashing
crass
craven
cravenly
craze
crazily
craziness
crazy
creak
creaking
creaks
credulous
creep
creeping
creeps
creepy
crept
crime
criminal
cringe
cringed
cringes
cripple
crippled
cripples
crippling
crisis
critic
critical
criticism
criticisms
criticize
criticized
criticizing
critics
cronyism
crook
crooked
crooks
crowded
crowdedness
crude
cruel
crueler
cruelest
cruelly
cruelness
cruelties
cruelty
crumble
crumbling
crummy
crumple
crumpled
crumples
crush
crushed
crushing
cry
culpable
culprit
cumbersome
cunt
cunts
cuplrit
curse
cursed
curses
curt
cuss
cussed
cutthroat
cynical
cynicism
d*mn
damage
damaged
damages
damaging
damn
damnable
damnably
damnation
damned
damning
damper
danger
dangerous
dangerousness
dark
darken
darkened
darker
darkness
dastard
dastardly
daunt
daunting
dauntingly
dawdle
daze
dazed
dead
deadbeat
deadlock
deadly
deadweight
deaf
dearth
death
debacle
debase
debasement
debaser
debatable
debauch
debaucher
debauchery
debilitate
debilitating
debility
debt
debts
decadence
decadent
decay
decayed
deceit
deceitful
deceitfully
deceitfulness
deceive
deceiver
deceivers
deceiving
deception
deceptive
deceptively
declaim
decline
declines
declining
decrement
decrepit
decrepitude
decry
defamation
defamations
defamatory
defame
defect
defective
defects
defensive
defiance
defiant
defiantly
deficiencies
deficiency
deficient
defile
defiler
deform
deformed
defrauding
defunct
defy
degenerate
degenerately
degeneration
degradation
degrade
degrading
degradingly
dehumanization
dehumanize
deign
deject
dejected
dejectedly
dejection
delay
delayed
delaying
delays
delinquency
delinquent
delirious
delirium
delude
deluded
deluge
delusion
delusional
delusions
demean
demeaning
demise
demolish
demolisher
demon
demonic
demonize
demonized
demonizes
demonizing
demoralize
demoralizing
demoralizingly
denial
denied
denies
denigrate
denounce
dense
dent
dented
dents
denunciate
denunciation
denunciations
deny
denying
deplete
deplorable
deplorably
deplore
deploring
deploringly
deprave
depraved
depravedly
deprecate
depress
depressed
depressing
depressingly
depression
depressions
deprive
deprived
deride
derision
derisive
derisively
derisiveness
derogatory
desecrate
desert
desertion
desiccate
desiccated
desititute
desolate
desolately
desolation
despair
despairing
despairingly
desperate
desperately
desperation
despicable
despicably
despise
despised
despoil
despoiler
despondence
despondency
despondent
despondently
despot
despotic
despotism
destabilisation
destains
destitute
destitution
destroy
destroyer
destruction
destructive
desultory
deter
deteriorate
deteriorating
deterioration
deterrent
detest
detestable
detestably
detested
detesting
detests
detract
detracted
detracting
detraction
detracts
detriment
detrimental
devastate
devastated
devastates
devastating
devastatingly
devastation
deviate
deviation
devil
devilish
devilishly
devilment
devilry
devious
deviously
deviousness
devoid
diabolic
diabolical
diabolically
diametrically
diappointed
diatribe
diatribes
dick
dictator
dictatorial
die
die-hard
died
dies
difficult
difficulties
difficulty
diffidence
dilapidated
dilemma
dilly-dally
dim
dimmer
din
ding
dings
dinky
dire
direly
direness
dirt
dirtbag
dirtbags
dirts
dirty
disable
disabled
disaccord
disadvantage
disadvantaged
disadvantageous
disadvantages
disaffect
disaffected
disaffirm
disagree
disagreeable
disagreeably
disagreed
disagreeing
disagreement
disagrees
disallow
disapointed
disapointing
disapointment
disappoint
disappointed
disappointing
disappointingly
disappointment
disappointments
disappoints
disapprobation
disapproval
disapprove
disapproving
disarm
disarray
disaster
disasterous
disastrous
disastrously
disavow
disavowal
disbelief
disbelieve
disbeliever
disclaim
discombobulate
discomfit
discomfititure
discomfort
discompose
disconcert
disconcerted
disconcerting
disconcertingly
disconsolate
disconsolately
disconsolation
discontent
discontented
discontentedly
discontinued
discontinuity
discontinuous
discord
discordance
discordant
discountenance
discourage
discouragement
discouraging
discouragingly
discourteous
discourteously
discoutinous
discredit
discrepant
discriminate
discrimination
discriminatory
disdain
disdained
disdainful
disdainfully
disfavor
disgrace
disgraced
disgraceful
disgracefully
disgruntle
disgruntled
disgust
disgusted
disgustedly
disgustful
disgustfully
disgusting
disgustingly
dishearten
disheartening
dishearteningly
dishonest
dishonestly
dishonesty
dishonor
dishonorable
dishonorablely
disillusion
disillusioned
disillusionment
disillusions
disinclination
disinclined
disingenuous
disingenuously
disintegrate
disintegrated
disintegrates
disintegration
disinterest
disinterested
dislike
disliked
dislikes
disliking
dislocated
disloyal
disloyalty
dismal
dismally
dismalness
dismay
dismayed
dismaying
dismayingly
dismissive
dismissively
disobedience
disobedient
disobey
disoobedient
disorder
disordered
disorderly
disorganized
disorient
disoriented
disown
disparage
disparaging
disparagingly
dispensable
dispirit
dispirited
dispiritedly
dispiriting
displace
displaced
displease
displeased
displeasing
displeasure
disproportionate
disprove
disputable
dispute
disputed
disquiet
disquieting
disquietingly
disquietude
disregard
disregardful
disreputable
disrepute
disrespect
disrespectable
disrespectablity
disrespectful
disrespectfully
disrespectfulness
disrespecting
disrupt
disruption
disruptive
diss
dissapointed
dissappointed
dissappointing
dissatisfaction
dissatisfactory
dissatisfied
dissatisfies
dissatisfy
dissatisfying
dissed
dissemble
dissembler
dissension
dissent
dissenter
dissention
disservice
disses
dissidence
dissident
dissidents
dissing
dissocial
dissolute
dissolution
dissonance
dissonant
dissonantly
dissuade
dissuasive
distains
distaste
distasteful
distastefully
distort
distorted
distortion
distorts
distract
distracting
distraction
distraught
distraughtly
distraughtness
distress
distressed
distressing
distressingly
distrust
distrustful
distrusting
disturb
disturbance
disturbed
disturbing
disturbingly
disunity
disvalue
divergent
divisive
divisively
divisiveness
dizzing
dizzingly
dizzy
doddering
dodgey
dogged
doggedly
dogmatic
doldrums
domineer
domineering
donside
doom
doomed
doomsday
dope
doubt
doubtful
doubtfully
doubts
douchbag
douchebag
douchebags
downbeat
downcast
downer
downfall
downfallen
downgrade
downhearted
downheartedly
downhill
downside
downsides
downturn
downturns
drab
draconian
draconic
drag
dragged
dragging
dragoon
drags
drain
drained
draining
drains
drastic
drastically
drawback
drawbacks
dread
dreadful
dreadfully
dreadfulness
dreary
dripped
dripping
drippy
drips
drones
droop
droops
drop-out
drop-outs
dropout
dropouts
drought
drowning
drunk
drunkard
drunken
dubious
dubiously
dubitable
dud
dull
dullard
dumb
dumbfound
dump
dumped
dumping
dumps
dunce
dungeon
dungeons
dupe
dust
dusty
dwindling
dying
earsplitting
eccentric
eccentricity
effigy
effrontery
egocentric
egomania
egotism
egotistical
egotistically
egregious
egregiously
election-rigger
elimination
emaciated
emasculate
embarrass
embarrassing
embarrassingly
embarrassment
embattled
embroil
embroiled
embroilment
emergency
emphatic
emphatically
emptiness
encroach
encroachment
endanger
enemies
enemy
enervate
enfeeble
enflame
engulf
enjoin
enmity
enrage
enraged
enraging
enslave
entangle
entanglement
entrap
entrapment
envious
enviously
enviousness
epidemic
equivocal
erase
erode
erodes
erosion
err
errant
erratic
erratically
erroneous
erroneously
error
errors
eruptions
escapade
eschew
estranged
evade
evasion
evasive
evil
evildoer
evils
eviscerate
exacerbate
exagerate
exagerated
exagerates
exaggerate
exaggeration
exasperate
exasperated
exasperating
exasperatingly
exasperation
excessive
excessively
exclusion
excoriate
excruciating
excruciatingly
excuse
excuses
execrate
exhaust
exhausted
exhaustion
exhausts
exhorbitant
exhort
exile
exorbitant
exorbitantance
exorbitantly
expel
expensive
expire
expired
explode
exploit
exploitation
explosive
expropriate
expropriation
expulse
expunge
exterminate
extermination
extinguish
extort
extortion
extraneous
extravagance
extravagant
extravagantly
extremism
extremist
extremists
eyesore
f**k
fabricate
fabrication
facetious
facetiously
fail
failed
failing
fails
failure
failures
faint
fainthearted
faithless
fake
fall
fallacies
fallacious
fallaciously
fallaciousness
fallacy
fallen
falling
fallout
falls
false
falsehood
falsely
falsify
falter
faltered
famine
famished
fanatic
fanatical
fanatically
fanaticism
fanatics
fanciful
far-fetched
farce
farcical
farcical-yet-provocative
farcically
farfetched
fascism
fascist
fastidious
fastidiously
fastuous
fat
fat-cat
fat-cats
fatal
fatalistic
fatalistically
fatally
fatcat
fatcats
fateful
fatefully
fathomless
fatigue
fatigued
fatique
fatty
fatuity
fatuous
fatuously
fault
faults
faulty
fawningly
faze
fear
fearful
fearfully
fears
fearsome
feckless
feeble
feeblely
feebleminded
feign
feint
fell
felon
felonious
ferociously
ferocity
fetid
fever
feverish
fevers
fiasco
fib
fibber
fickle
fiction
fictional
fictitious
fidget
fidgety
fiend
fiendish
fierce
figurehead
filth
filthy
finagle
finicky
fissures
fist
flabbergast
flabbergasted
flagging
flagrant
flagrantly
flair
flairs
flak
flake
flakey
flakieness
flaking
flaky
flare
flares
flareup
flareups
flat-out
flaunt
flaw
flawed
flaws
flee
fleed
fleeing
fleer
flees
fleeting
flicering
flicker
flickering
flickers
flighty
flimflam
flimsy
flirt
flirty
floored
flounder
floundering
flout
fluster
foe
fool
fooled
foolhardy
foolish
foolishly
foolishness
forbid
forbidden
forbidding
forceful
foreboding
forebodingly
forfeit
forged
forgetful
forgetfully
forgetfulness
forlorn
forlornly
forsake
forsaken
forswear
foul
foully
foulness
fractious
fractiously
fracture
fragile
fragmented
frail
frantic
frantically
franticly
fraud
fraudulent
fraught
frazzle
frazzled
freak
freaking
freakish
freakishly
freaks
freeze
freezes
freezing
frenetic
frenetically
frenzied
frenzy
fret
fretful
frets
friction
frictions
fried
friggin
frigging
fright
frighten
frightening
frighteningly
frightful
frightfully
frigid
frost
frown
froze
frozen
fruitless
fruitlessly
frustrate
frustrated
frustrates
frustrating
frustratingly
frustration
frustrations
fuck
fucking
fudge
fugitive
full-blown
fulminate
fumble
fume
fumes
fundamentalism
funky
funnily
funny
furious
furiously
furor
fury
fuss
fussy
fustigate
fusty
futile
futilely
futility
fuzzy
gabble
gaff
gaffe
gainsay
gainsayer
gall
galling
gallingly
galls
gangster
gape
garbage
garish
gasp
gauche
gaudy
gawk
gawky
geezer
genocide
get-rich
ghastly
ghetto
ghosting
gibber
gibberish
gibe
giddy
gimmick
gimmicked
gimmicking
gimmicks
gimmicky
glare
glaringly
glib
glibly
glitch
glitches
gloatingly
gloom
gloomy
glower
glum
glut
gnawing
goad
goading
god-awful
goof
goofy
goon
gossip
graceless
gracelessly
graft
grainy
grapple
grate
grating
gravely
greasy
greed
greedy
grief
grievance
grievances
grieve
grieving
grievous
grievously
grim
grimace
grind
gripe
gripes
grisly
gritty
gross
grossly
grotesque
grouch
grouchy
groundless
grouse
growl
grudge
grudges
grudging
grudgingly
gruesome
gruesomely
gruff
grumble
grumpier
grumpiest
grumpily
grumpish
grumpy
guile
guilt
guiltily
guilty
gullible
gutless
gutter
hack
hacks
haggard
haggle
hairloss
halfhearted
halfheartedly
hallucinate
hallucination
hamper
hampered
handicapped
hang
hangs
haphazard
hapless
harangue
harass
harassed
harasses
harassment
harboring
harbors
hard
hard-hit
hard-line
hard-liner
hardball
harden
hardened
hardheaded
hardhearted
hardliner
hardliners
hardship
hardships
harm
harmed
harmful
harms
harpy
harridan
harried
harrow
harsh
harshly
hasseling
hassle
hassled
hassles
haste
hastily
hasty
hate
hated
hateful
hatefully
hatefulness
hater
haters
hates
hating
hatred
haughtily
haughty
haunt
haunting
havoc
hawkish
haywire
hazard
hazardous
haze
hazy
head-aches
headache
headaches
heartbreaker
heartbreaking
heartbreakingly
heartless
heathen
heavy-handed
heavyhearted
heck
heckle
heckled
heckles
hectic
hedge
hedonistic
heedless
hefty
hegemonism
hegemonistic
hegemony
heinous
hell
hell-bent
hellion
hells
helpless
helplessly
helplessness
heresy
heretic
heretical
hesitant
hestitant
hideous
hideously
hideousness
high-priced
hiliarious
hinder
hindrance
hiss
hissed
hissing
ho-hum
hoard
hoax
hobble
hogs
hollow
hoodium
hoodwink
hooligan
hopeless
hopelessly
hopelessness
horde
horrendous
horrendously
horrible
horrid
horrific
horrified
horrifies
horrify
horrifying
horrifys
hostage
hostile
hostilities
hostility
hotbeds
hothead
hotheaded
hothouse
hubris
huckster
hum
humid
humiliate
humiliating
humiliation
humming
hung
hurt
hurted
hurtful
hurting
hurts
hustler
hype
hypocricy
hypocrisy
hypocrite
hypocrites
hypocritical
hypocritically
hysteria
hysteric
hysterical
hysterically
hysterics
idiocies
idiocy
idiot
idiotic
idiotically
idiots
idle
ignoble
ignominious
ignominiously
ignominy
ignorance
ignorant
ignore
ill-advised
ill-conceived
ill-defined
ill-designed
ill-fated
ill-favored
ill-formed
ill-mannered
ill-natured
ill-sorted
ill-tempered
ill-treated
ill-treatment
ill-usage
ill-used
illegal
illegally
illegitimate
illicit
illiterate
illness
illogic
illogical
illogically
illusion
illusions
illusory
imaginary
imbalance
imbecile
imbroglio
immaterial
immature
imminence
imminently
immobilized
immoderate
immoderately
immodest
immoral
immorality
immorally
immovable
impair
impaired
impasse
impatience
impatient
impatiently
impeach
impedance
impede
impediment
impending
impenitent
imperfect
imperfection
imperfections
imperfectly
imperialist
imperil
imperious
imperiously
impermissible
impersonal
impertinent
impetuous
impetuously
impiety
impinge
impious
implacable
implausible
implausibly
implicate
implication
implode
impolite
impolitely
impolitic
importunate
importune
impose
imposers
imposing
imposition
impossible
impossiblity
impossibly
impotent
impoverish
impoverished
impractical
imprecate
imprecise
imprecisely
imprecision
imprison
imprisonment
improbability
improbable
improbably
improper
improperly
impropriety
imprudence
imprudent
impudence
impudent
impudently
impugn
impulsive
impulsively
impunity
impure
impurity
inability
inaccuracies
inaccuracy
inaccurate
inaccurately
inaction
inactive
inadequacy
inadequate
inadequately
inadverent
inadverently
inadvisable
inadvisably
inane
inanely
inappropriate
inappropriately
inapt
inaptitude
inarticulate
inattentive
inaudible
incapable
incapably
incautious
incendiary
incense
incessant
incessantly
incite
incitement
incivility
inclement
incognizant
incoherence
incoherent
incoherently
incommensurate
incomparable
incomparably
incompatability
incompatibility
incompatible
incompetence
incompetent
incompetently
incomplete
incompliant
incomprehensible
incomprehension
inconceivable
inconceivably
incongruous
incongruously
inconsequent
inconsequential
inconsequentially
inconsequently
inconsiderate
inconsiderately
inconsistence
inconsistencies
inconsistency
inconsistent
inconsolable
inconsolably
inconstant
inconvenience
inconveniently
incorrect
incorrectly
incorrigible
incorrigibly
incredulous
incredulously
inculcate
indecency
indecent
indecently
indecision
indecisive
indecisively
indecorum
indefensible
indelicate
indeterminable
indeterminably
indeterminate
indifference
indifferent
indigent
indignant
indignantly
indignation
indignity
indiscernible
indiscreet
indiscreetly
indiscretion
indiscriminate
indiscriminately
indiscriminating
indistinguishable
indoctrinate
indoctrination
indolent
indulge
ineffective
ineffectively
ineffectiveness
ineffectual
ineffectually
ineffectualness
inefficacious
inefficacy
inefficiency
inefficient
inefficiently
inelegance
inelegant
ineligible
ineloquent
ineloquently
inept
ineptitude
ineptly
inequalities
inequality
inequitable
inequitably
inequities
inescapable
inescapably
inessential
inevitable
inevitably
inexcusable
inexcusably
inexorable
inexorably
inexperience
inexperienced
inexpert
inexpertly
inexpiable
inexplainable
inextricable
inextricably
infamous
infamously
infamy
infected
infection
infections
inferior
inferiority
infernal
infest
infested
infidel
infidels
infiltrator
infiltrators
infirm
inflame
inflammation
inflammatory
inflammed
inflated
inflationary
inflexible
inflict
infraction
infringe
infringement
infringements
infuriate
infuriated
infuriating
infuriatingly
inglorious
ingrate
ingratitude
inhibit
inhibition
inhospitable
inhospitality
inhuman
inhumane
inhumanity
inimical
inimically
iniquitous
iniquity
injudicious
injure
injurious
injury
injustice
injustices
innuendo
inoperable
inopportune
inordinate
inordinately
insane
insanely
insanity
insatiable
insecure
insecurity
insensible
insensitive
insensitively
insensitivity
insidious
insidiously
insignificance
insignificant
insignificantly
insincere
insincerely
insincerity
insinuate
insinuating
insinuation
insociable
insolence
insolent
insolently
insolvent
insouciance
instability
instable
instigate
instigator
instigators
insubordinate
insubstantial
insubstantially
insufferable
insufferably
insufficiency
insufficient
insufficiently
insular
insult
insulted
insulting
insultingly
insults
insupportable
insupportably
insurmountable
insurmountably
insurrection
intefere
inteferes
intense
interfere
interference
interferes
intermittent
interrupt
interruption
interruptions
intimidate
intimidating
intimidatingly
intimidation
intolerable
intolerablely
intolerance
intoxicate
intractable
intransigence
intransigent
intrude
intrusion
intrusive
inundate
inundated
invader
invalid
invalidate
invalidity
invasive
invective
inveigle
invidious
invidiously
invidiousness
invisible
involuntarily
involuntary
irascible
irate
irately
ire
irk
irked
irking
irks
irksome
irksomely
irksomeness
irksomenesses
ironic
ironical
ironically
ironies
irony
irragularity
irrational
irrationalities
irrationality
irrationally
irrationals
irreconcilable
irrecoverable
irrecoverableness
irrecoverablenesses
irrecoverably
irredeemable
irredeemably
irreformable
irregular
irregularity
irrelevance
irrelevant
irreparable
irreplacible
irrepressible
irresolute
irresolvable
irresponsible
irresponsibly
irretating
irretrievable
irreversible
irritable
irritably
irritant
irritate
irritated
irritating
irritation
irritations
isolate
isolated
isolation
issue
issues
itch
itching
itchy
jabber
jaded
jagged
jam
jarring
jaundiced
jealous
jealously
jealousness
jealousy
jeer
jeering
jeeringly
jeers
jeopardize
jeopardy
jerk
jerky
jitter
jitters
jittery
job-killing
jobless
joke
joker
jolt
judder
juddering
judders
jumpy
junk
junky
junkyard
jutter
jutters
kaput
kill
killed
killer
killing
killjoy
kills
knave
knife
knock
knotted
kook
kooky
lack
lackadaisical
lacked
lackey
lackeys
lacking
lackluster
lacks
laconic
lag
lagged
lagging
laggy
lags
laid-off
lambast
lambaste
lame
lame-duck
lament
lamentable
lamentably
languid
languish
languor
languorous
languorously
lanky
lapse
lapsed
lapses
lascivious
last-ditch
latency
laughable
laughably
laughingstock
lawbreaker
lawbreaking
lawless
lawlessness
layoff
layoff-happy
lazy
leak
leakage
leakages
leaking
leaks
leaky
lech
lecher
lecherous
lechery
leech
leer
leery
left-leaning
lemon
lengthy
less-developed
lesser-known
letch
lethal
lethargic
lethargy
lewd
lewdly
lewdness
liability
liable
liar
liars
licentious
licentiously
licentiousness
lie
lied
lier
lies
life-threatening
lifeless
limit
limitation
limitations
limited
limits
limp
listless
litigious
little-known
livid
lividly
loath
loathe
loathing
loathly
loathsome
loathsomely
lone
loneliness
lonely
loner
lonesome
long-time
long-winded
longing
longingly
loophole
loopholes
loose
loot
lorn
lose
loser
losers
loses
losing
loss
losses
lost
loud
louder
lousy
loveless
lovelorn
low-rated
lowly
ludicrous
ludicrously
lugubrious
lukewarm
lull
lumpy
lunatic
lunaticism
lurch
lure
lurid
lurk
lurking
lying
macabre
mad
madden
maddening
maddeningly
madder
madly
madman
madness
maladjusted
maladjustment
malady
malaise
malcontent
malcontented
maledict
malevolence
malevolent
malevolently
malice
malicious
maliciously
maliciousness
malign
malignant
malodorous
maltreatment
mangle
mangled
mangles
mangling
mania
maniac
maniacal
manic
manipulate
manipulation
manipulative
manipulators
mar
marginal
marginally
martyrdom
martyrdom-seeking
mashed
massacre
massacres
matte
mawkish
mawkishly
mawkishness
meager
meaningless
meanness
measly
meddle
meddlesome
mediocre
mediocrity
melancholy
melodramatic
melodramatically
meltdown
menace
menacing
menacingly
mendacious
mendacity
menial
merciless
mercilessly
mess
messed
messes
messing
messy
midget
miff
militancy
mindless
mindlessly
mirage
mire
misalign
misaligned
misaligns
misapprehend
misbecome
misbecoming
misbegotten
misbehave
misbehavior
miscalculate
miscalculation
miscellaneous
mischief
mischievous
mischievously
misconception
misconceptions
miscreant
miscreants
misdirection
miser
miserable
miserableness
miserably
miseries
miserly
misery
misfit
misfortune
misgiving
misgivings
misguidance
misguide
misguided
mishandle
mishap
misinform
misinformed
misinterpret
misjudge
misjudgment
mislead
misleading
misleadingly
mislike
mismanage
mispronounce
mispronounced
mispronounces
misread
misreading
misrepresent
misrepresentation
miss
missed
misses
misstatement
mist
mistake
mistaken
mistakenly
mistakes
mistified
mistress
mistrust
mistrustful
mistrustfully
mists
misunderstand
misunderstanding
misunderstandings
misunderstood
misuse
moan
mobster
mock
mocked
mockeries
mockery
mocking
mockingly
mocks
molest
molestation
monotonous
monotony
monster
monstrosities
monstrosity
monstrous
monstrously
moody
moot
mope
morbid
morbidly
mordant
mordantly
moribund
moron
moronic
morons
mortification
mortified
mortify
mortifying
motionless
motley
mourn
mourner
mournful
mournfully
muddle
muddy
mudslinger
mudslinging
mulish
multi-polarization
mundane
murder
murderer
murderous
murderously
murky
muscle-flexing
mushy
musty
mysterious
mysteriously
mystery
mystify
myth
nag
nagging
naive
naively
narrower
nastily
nastiness
nasty
naughty
nauseate
nauseates
nauseating
nauseatingly
naīve
nebulous
nebulously
needless
needlessly
needy
nefarious
nefariously
negate
negation
negative
negatives
negativity
neglect
neglected
negligence
negligent
nemesis
nepotism
nervous
nervously
nervousness
nettle
nettlesome
neurotic
neurotically
niggle
niggles
nightmare
nightmarish
nightmarishly
nitpick
nitpicking
noise
noises
noisier
noisy
non-confidence
nonexistent
nonresponsive
nonsense
nosey
notoriety
notorious
notoriously
noxious
nuisance
numb
obese
object
objection
objectionable
objections
oblique
obliterate
obliterated
oblivious
obnoxious
obnoxiously
obscene
obscenely
obscenity
obscure
obscured
obscures
obscurity
obsess
obsessive
obsessively
obsessiveness
obsolete
obstacle
obstinate
obstinately
obstruct
obstructed
obstructing
obstruction
obstructs
obtrusive
obtuse
occlude
occluded
occludes
occluding
odd
odder
oddest
oddities
oddity
oddly
odor
offence
offend
offender
offending
offenses
offensive
offensively
offensiveness
officious
ominous
ominously
omission
omit
one-sided
onerous
onerously
onslaught
opinionated
opponent
opportunistic
oppose
opposition
oppositions
oppress
oppression
oppressive
oppressively
oppressiveness
oppressors
ordeal
orphan
ostracize
outbreak
outburst
outbursts
outcast
outcry
outlaw
outmoded
outrage
outraged
outrageous
outrageously
outrageousness
outrages
outsider
over-acted
over-awe
over-balanced
over-hyped
over-priced
over-valuation
overact
overacted
overawe
overbalance
overbalanced
overbearing
overbearingly
overblown
overdo
overdone
overdue
overemphasize
overheat
overkill
overloaded
overlook
overpaid
overpayed
overplay
overpower
overpriced
overrated
overreach
overrun
overshadow
oversight
oversights
oversimplification
oversimplified
oversimplify
oversize
overstate
overstated
overstatement
overstatements
overstates
overtaxed
overthrow
overthrows
overturn
overweight
overwhelm
overwhelmed
overwhelming
overwhelmingly
overwhelms
overzealous
overzealously
overzelous
pain
painful
painfull
painfully
pains
pale
pales
paltry
pan
pandemonium
pander
pandering
panders
panic
panick
panicked
panicking
panicky
paradoxical
paradoxically
paralize
paralyzed
paranoia
paranoid
parasite
pariah
parody
partiality
partisan
partisans
passe
passive
passiveness
pathetic
pathetically
patronize
paucity
pauper
paupers
payback
peculiar
peculiarly
pedantic
peeled
peeve
peeved
peevish
peevishly
penalize
penalty
perfidious
perfidity
perfunctory
peril
perilous
perilously
perish
pernicious
perplex
perplexed
perplexing
perplexity
persecute
persecution
pertinacious
pertinaciously
pertinacity
perturb
perturbed
pervasive
perverse
perversely
perversion
perversity
pervert
perverted
perverts
pessimism
pessimistic
pessimistically
pest
pestilent
petrified
petrify
pettifog
petty
phobia
phobic
phony
picket
picketed
picketing
pickets
picky
pig
pigs
pillage
pillory
pimple
pinch
pique
pitiable
pitiful
pitifully
pitiless
pitilessly
pittance
pity
plagiarize
plague
plasticky
plaything
plea
pleas
plebeian
plight
plot
plotters
ploy
plunder
plunderer
pointless
pointlessly
poison
poisonous
poisonously
pokey
poky
polarisation
polemize
pollute
polluter
polluters
polution
pompous
poor
poorer
poorest
poorly
posturing
pout
poverty
powerless
prate
pratfall
prattle
precarious
precariously
precipitate
precipitous
predatory
predicament
prejudge
prejudice
prejudices
prejudicial
premeditated
preoccupy
preposterous
preposterously
presumptuous
presumptuously
pretence
pretend
pretense
pretentious
pretentiously
prevaricate
pricey
pricier
prick
prickle
prickles
prideful
prik
primitive
prison
prisoner
problem
problematic
problems
procrastinate
procrastinates
procrastination
profane
profanity
prohibit
prohibitive
prohibitively
propaganda
propagandize
proprietary
prosecute
protest
protested
protesting
protests
protracted
provocation
provocative
provoke
pry
pugnacious
pugnaciously
pugnacity
punch
punish
punishable
punitive
punk
puny
puppet
puppets
puzzled
puzzlement
puzzling
quack
qualm
qualms
quandary
quarrel
quarrellous
quarrellously
quarrels
quarrelsome
quash
queer
questionable
quibble
quibbles
quitter
rabid
racism
racist
racists
racy
radical
radicalization
radically
radicals
rage
ragged
raging
rail
raked
rampage
rampant
ramshackle
rancor
randomly
rankle
rant
ranted
ranting
rantingly
rants
rape
raped
raping
rascal
rascals
rash
rattle
rattled
rattles
ravage
raving
reactionary
rebellious
rebuff
rebuke
recalcitrant
recant
recession
recessionary
reckless
recklessly
recklessness
recoil
recourses
redundancy
redundant
refusal
refuse
refused
refuses
refusing
refutation
refute
refuted
refutes
refuting
regress
regression
regressive
regret
regreted
regretful
regretfully
regrets
regrettable
regrettably
regretted
reject
rejected
rejecting
rejection
rejects
relapse
relentless
relentlessly
relentlessness
reluctance
reluctant
reluctantly
remorse
remorseful
remorsefully
remorseless
remorselessly
remorselessness
renounce
renunciation
repel
repetitive
reprehensible
reprehensibly
reprehension
reprehensive
repress
repression
repressive
reprimand
reproach
reproachful
reprove
reprovingly
repudiate
repudiation
repugn
repugnance
repugnant
repugnantly
repulse
repulsed
repulsing
repulsive
repulsively
repulsiveness
resent
resentful
resentment
resignation
resigned
resistance
restless
restlessness
restrict
restricted
restriction
restrictive
resurgent
retaliate
retaliatory
retard
retarded
retardedness
retards
reticent
retract
retreat
retreated
revenge
revengeful
revengefully
revert
revile
reviled
revoke
revolt
revolting
revoltingly
revulsion
revulsive
rhapsodize
rhetoric
rhetorical
ricer
ridicule
ridicules
ridiculous
ridiculously
rife
rift
rifts
rigid
rigidity
rigidness
rile
riled
rip
rip-off
ripoff
ripped
risk
risks
risky
rival
rivalry
roadblocks
rocky
rogue
rollercoaster
rot
rotten
rough
rremediable
rubbish
rude
rue
ruffian
ruffle
ruin
ruined
ruining
ruinous
ruins
rumbling
rumor
rumors
rumours
rumple
run-down
runaway
rupture
rust
rusts
rusty
rut
ruthless
ruthlessly
ruthlessness
ruts
sabotage
sack
sacrificed
sad
sadden
sadly
sadness
sag
sagged
sagging
saggy
sags
salacious
sanctimonious
sap
sarcasm
sarcastic
sarcastically
sardonic
sardonically
sass
satirical
satirize
savage
savaged
savagery
savages
scaly
scam
scams
scandal
scandalize
scandalized
scandalous
scandalously
scandals
scandel
scandels
scant
scapegoat
scar
scarce
scarcely
scarcity
scare
scared
scarier
scariest
scarily
scarred
scars
scary
scathing
scathingly
sceptical
scoff
scoffingly
scold
scolded
scolding
scoldingly
scorching
scorchingly
scorn
scornful
scornfully
scoundrel
scourge
scowl
scramble
scrambled
scrambles
scrambling
scrap
scratch
scratched
scratches
scratchy
scream
screech
screw-up
screwed
screwed-up
screwy
scuff
scuffs
scum
scummy
second-class
second-tier
secretive
sedentary
seedy
seethe
seething
self-coup
self-criticism
self-defeating
self-destructive
self-humiliation
self-interest
self-interested
self-serving
selfinterested
selfish
selfishly
selfishness
semi-retarded
senile
sensationalize
senseless
senselessly
seriousness
sermonize
servitude
set-up
setback
setbacks
sever
severe
severity
sh*t
shabby
shadowy
shady
shake
shaky
shallow
sham
shambles
shame
shameful
shamefully
shamefulness
shameless
shamelessly
shamelessness
shark
sharply
shatter
shemale
shimmer
shimmy
shipwreck
shirk
shirker
shit
shiver
shock
shocked
shocking
shockingly
shoddy
short-lived
shortage
shortchange
shortcoming
shortcomings
shortness
shortsighted
shortsightedness
showdown
shrew
shriek
shrill
shrilly
shrivel
shroud
shrouded
shrug
shun
shunned
sick
sicken
sickening
sickeningly
sickly
sickness
sidetrack
sidetracked
siege
sillily
silly
simplistic
simplistically
sin
sinful
sinfully
sinister
sinisterly
sink
sinking
skeletons
skeptic
skeptical
skeptically
skepticism
sketchy
skimpy
skinny
skittish
skittishly
skulk
slack
slander
slanderer
slanderous
slanderously
slanders
slap
slashing
slaughter
slaughtered
slave
slaves
sleazy
slime
slog
slogged
slogging
slogs
sloooooooooooooow
sloooow
slooow
sloow
sloppily
sloppy
sloth
slothful
slow
slow-moving
slowed
slower
slowest
slowly
sloww
slowww
slowwww
slug
sluggish
slump
slumping
slumpping
slur
slut
sluts
sly
smack
smallish
smash
smear
smell
smelled
smelling
smells
smelly
smelt
smoke
smokescreen
smolder
smoldering
smother
smoulder
smouldering
smudge
smudged
smudges
smudging
smug
smugly
smut
smuttier
smuttiest
smutty
snag
snagged
snagging
snags
snappish
snappishly
snare
snarky
snarl
sneak
sneakily
sneaky
sneer
sneering
sneeringly
snob
snobbish
snobby
snobish
snobs
snub
so-cal
soapy
sob
sober
sobering
solemn
solicitude
somber
sore
sorely
soreness
sorrow
sorrowful
sorrowfully
sorry
sour
sourly
spade
spank
spendy
spew
spewed
spewing
spews
spilling
spinster
spiritless
spite
spiteful
spitefully
spitefulness
splatter
split
splitting
spoil
spoilage
spoilages
spoiled
spoilled
spoils
spook
spookier
spookiest
spookily
spooky
spoon-fed
spoon-feed
spoonfed
sporadic
spotty
spurious
spurn
sputter
squabble
squabbling
squander
squash
squeak
squeaks
squeaky
squeal
squealing
squeals
squirm
stab
stagnant
stagnate
stagnation
staid
stain
stains
stale
stalemate
stall
stalls
stammer
stampede
standstill
stark
starkly
startle
startling
startlingly
starvation
starve
static
steal
stealing
steals
steep
steeply
stench
stereotype
stereotypical
stereotypically
stern
stew
sticky
stiff
stiffness
stifle
stifling
stiflingly
stigma
stigmatize
sting
stinging
stingingly
stingy
stink
stinks
stodgy
stole
stolen
stooge
stooges
stormy
straggle
straggler
strain
strained
straining
strange
strangely
stranger
strangest
strangle
streaky
strenuous
stress
stresses
stressful
stressfully
stricken
strict
strictly
strident
stridently
strife
strike
stringent
stringently
struck
struggle
struggled
struggles
struggling
strut
stubborn
stubbornly
stubbornness
stuck
stuffy
stumble
stumbled
stumbles
stump
stumped
stumps
stun
stunt
stunted
stupid
stupidest
stupidity
stupidly
stupified
stupify
stupor
stutter
stuttered
stuttering
stutters
sty
stymied
sub-par
subdued
subjected
subjection
subjugate
subjugation
submissive
subordinate
subpoena
subpoenas
subservience
subservient
substandard
subtract
subversion
subversive
subversively
subvert
succumb
suck
sucked
sucker
sucks
sucky
sue
sued
sueing
sues
suffer
suffered
sufferer
sufferers
suffering
suffers
suffocate
sugar-coat
sugar-coated
sugarcoated
suicidal
suicide
sulk
sullen
sully
sunder
sunk
sunken
superficial
superficiality
superficially
superfluous
superstition
superstitious
suppress
suppression
surrender
susceptible
suspect
suspicion
suspicions
suspicious
suspiciously
swagger
swamped
sweaty
swelled
swelling
swindle
swipe
swollen
symptom
symptoms
syndrome
taboo
tacky
taint
tainted
tamper
tangle
tangled
tangles
tank
tanked
tanks
tantrum
tardy
tarnish
tarnished
tarnishes
tarnishing
tattered
taunt
taunting
tauntingly
taunts
taut
tawdry
taxing
tease
teasingly
tedious
tediously
temerity
temper
tempest
temptation
tenderness
tense
tension
tentative
tentatively
tenuous
tenuously
tepid
terrible
terribleness
terribly
terror
terror-genic
terrorism
terrorize
testily
testy
tetchily
tetchy
thankless
thicker
thirst
thorny
thoughtless
thoughtlessly
thoughtlessness
thrash
threat
threaten
threatening
threats
threesome
throb
throbbed
throbbing
throbs
throttle
thug
thumb-down
thumbs-down
thwart
time-consuming
timid
timidity
timidly
timidness
tin-y
tingled
tingling
tired
tiresome
tiring
tiringly
toil
toll
top-heavy
topple
torment
tormented
torrent
tortuous
torture
tortured
tortures
torturing
torturous
torturously
totalitarian
touchy
toughness
tout
touted
touts
toxic
traduce
tragedy
tragic
tragically
traitor
traitorous
traitorously
tramp
trample
transgress
transgression
trap
traped
trapped
trash
trashed
trashy
trauma
traumatic
traumatically
traumatize
traumatized
travesties
travesty
treacherous
treacherously
treachery
treason
treasonous
trick
tricked
trickery
tricky
trivial
trivialize
trouble
troubled
troublemaker
troubles
troublesome
troublesomely
troubling
troublingly
truant
tumble
tumbled
tumbles
tumultuous
turbulent
turmoil
twist
twisted
twists
two-faced
two-faces
tyrannical
tyrannically
tyranny
tyrant
ugh
uglier
ugliest
ugliness
ugly
ulterior
ultimatum
ultimatums
ultra-hardline
un-viewable
unable
unacceptable
unacceptablely
unacceptably
unaccessible
unaccustomed
unachievable
unaffordable
unappealing
unattractive
unauthentic
unavailable
unavoidably
unbearable
unbearablely
unbelievable
unbelievably
uncaring
uncertain
uncivil
uncivilized
unclean
unclear
uncollectible
uncomfortable
uncomfortably
uncomfy
uncompetitive
uncompromising
uncompromisingly
unconfirmed
unconstitutional
uncontrolled
unconvincing
unconvincingly
uncooperative
uncouth
uncreative
undecided
undefined
undependability
undependable
undercut
undercuts
undercutting
underdog
underestimate
underlings
undermine
undermined
undermines
undermining
underpaid
underpowered
undersized
undesirable
undetermined
undid
undignified
undissolved
undocumented
undone
undue
unease
uneasily
uneasiness
uneasy
uneconomical
unemployed
unequal
unethical
uneven
uneventful
unexpected
unexpectedly
unexplained
unfairly
unfaithful
unfaithfully
unfamiliar
unfavorable
unfeeling
unfinished
unfit
unforeseen
unforgiving
unfortunate
unfortunately
unfounded
unfriendly
unfulfilled
unfunded
ungovernable
ungrateful
unhappily
unhappiness
unhappy
unhealthy
unhelpful
unilateralism
unimaginable
unimaginably
unimportant
uninformed
uninsured
unintelligible
unintelligile
unipolar
unjust
unjustifiable
unjustifiably
unjustified
unjustly
unkind
unkindly
unknown
unlamentable
unlamentably
unlawful
unlawfully
unlawfulness
unleash
unlicensed
unlikely
unlucky
unmoved
unnatural
unnaturally
unnecessary
unneeded
unnerve
unnerved
unnerving
unnervingly
unnoticed
unobserved
unorthodox
unorthodoxy
unpleasant
unpleasantries
unpopular
unpredictable
unprepared
unproductive
unprofitable
unprove
unproved
unproven
unproves
unproving
unqualified
unravel
unraveled
unreachable
unreadable
unrealistic
unreasonable
unreasonably
unrelenting
unrelentingly
unreliability
unreliable
unresolved
unresponsive
unrest
unruly
unsafe
unsatisfactory
unsavory
unscrupulous
unscrupulously
unsecure
unseemly
unsettle
unsettled
unsettling
unsettlingly
unskilled
unsophisticated
unsound
unspeakable
unspeakablely
unspecified
unstable
unsteadily
unsteadiness
unsteady
unsuccessful
unsuccessfully
unsupported
unsupportive
unsure
unsuspecting
unsustainable
untenable
untested
unthinkable
unthinkably
untimely
untouched
untrue
untrustworthy
untruthful
unusable
unusably
unuseable
unuseably
unusual
unusually
unviewable
unwanted
unwarranted
unwatchable
unwelcome
unwell
unwieldy
unwilling
unwillingly
unwillingness
unwise
unwisely
unworkable
unworthy
unyielding
upbraid
upheaval
uprising
uproar
uproarious
uproariously
uproarous
uproarously
uproot
upset
upseting
upsets
upsetting
upsettingly
urgent
useless
usurp
usurper
utterly
vagrant
vague
vagueness
vain
vainly
vanity
vehement
vehemently
vengeance
vengeful
vengefully
vengefulness
venom
venomous
venomously
vent
vestiges
vex
vexation
vexing
vexingly
vibrate
vibrated
vibrates
vibrating
vibration
vice
vicious
viciously
viciousness
victimize
vile
vileness
vilify
villainous
villainously
villains
villian
villianous
villianously
villify
vindictive
vindictively
vindictiveness
violate
violation
violator
violators
violent
violently
viper
virulence
virulent
virulently
virus
vociferous
vociferously
volatile
volatility
vomit
vomited
vomiting
vomits
vulgar
vulnerable
wack
wail
wallow
wane
waning
wanton
war-like
warily
wariness
warlike
warned
warning
warp
warped
wary
washed-out
waste
wasted
wasteful
wastefulness
wasting
water-down
watered-down
wayward
weak
weaken
weakening
weaker
weakness
weaknesses
weariness
wearisome
weary
wedge
weed
weep
weird
weirdly
wheedle
whimper
whine
whining
whiny
whips
whore
whores
wicked
wickedly
wickedness
wild
wildly
wiles
wilt
wily
wimpy
wince
wobble
wobbled
wobbles
woe
woebegone
woeful
woefully
womanizer
womanizing
worn
worried
worriedly
worrier
worries
worrisome
worry
worrying
worryingly
worse
worsen
worsening
worst
worthless
worthlessly
worthlessness
wound
wounds
wrangle
wrath
wreak
wreaked
wreaks
wreck
wrest
wrestle
wretch
wretched
wretchedly
wretchedness
wrinkle
wrinkled
wrinkles
wrip
wripped
wripping
writhe
wrong
wrongful
wrongly
wrought
yawn
zap
zapped
zaps
zealot
zealous
zealously
zombie'''
negative_words = negative_words.replace('\n',',')
negative_words = negative_words.split(',')
negative_words

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [13]:
url_id = '''https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/
https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/
https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/
https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/
https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/
https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/
https://insights.blackcoffer.com/how-to-protect-future-data-and-its-privacy-blackcoffer/
https://insights.blackcoffer.com/how-machines-ai-automations-and-robo-human-are-effective-in-finance-and-banking/
https://insights.blackcoffer.com/ai-human-robotics-machine-future-planet-blackcoffer-thinking-jobs-workplace/
https://insights.blackcoffer.com/how-ai-will-change-the-world-blackcoffer/
https://insights.blackcoffer.com/future-of-work-how-ai-has-entered-the-workplace/
https://insights.blackcoffer.com/ai-tool-alexa-google-assistant-finance-banking-tool-future/
https://insights.blackcoffer.com/ai-healthcare-revolution-ml-technology-algorithm-google-analytics-industrialrevolution/
https://insights.blackcoffer.com/all-you-need-to-know-about-online-marketing/
https://insights.blackcoffer.com/evolution-of-advertising-industry/
https://insights.blackcoffer.com/how-data-analytics-can-help-your-business-respond-to-the-impact-of-covid-19/
https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
https://insights.blackcoffer.com/environmental-impact-of-the-covid-19-pandemic-lesson-for-the-future/
https://insights.blackcoffer.com/how-data-analytics-and-ai-are-used-to-halt-the-covid-19-pandemic/
https://insights.blackcoffer.com/difference-between-artificial-intelligence-machine-learning-statistics-and-data-mining/
https://insights.blackcoffer.com/how-python-became-the-first-choice-for-data-science/
https://insights.blackcoffer.com/how-google-fit-measure-heart-and-respiratory-rates-using-a-phone/
https://insights.blackcoffer.com/what-is-the-future-of-mobile-apps/
https://insights.blackcoffer.com/impact-of-ai-in-health-and-medicine/
https://insights.blackcoffer.com/telemedicine-what-patients-like-and-dislike-about-it/
https://insights.blackcoffer.com/how-we-forecast-future-technologies/
https://insights.blackcoffer.com/can-robots-tackle-late-life-loneliness/
https://insights.blackcoffer.com/embedding-care-robots-into-society-socio-technical-considerations/
https://insights.blackcoffer.com/management-challenges-for-future-digitalization-of-healthcare-services/
https://insights.blackcoffer.com/are-we-any-closer-to-preventing-a-nuclear-holocaust/
https://insights.blackcoffer.com/will-technology-eliminate-the-need-for-animal-testing-in-drug-development/
https://insights.blackcoffer.com/will-we-ever-understand-the-nature-of-consciousness/
https://insights.blackcoffer.com/will-we-ever-colonize-outer-space/
https://insights.blackcoffer.com/what-is-the-chance-homo-sapiens-will-survive-for-the-next-500-years/
https://insights.blackcoffer.com/why-does-your-business-need-a-chatbot/
https://insights.blackcoffer.com/how-you-lead-a-project-or-a-team-without-any-technical-expertise/
https://insights.blackcoffer.com/can-you-be-great-leader-without-technical-expertise/
https://insights.blackcoffer.com/how-does-artificial-intelligence-affect-the-environment/
https://insights.blackcoffer.com/how-to-overcome-your-fear-of-making-mistakes-2/
https://insights.blackcoffer.com/is-perfection-the-greatest-enemy-of-productivity/
https://insights.blackcoffer.com/global-financial-crisis-2008-causes-effects-and-its-solution/
https://insights.blackcoffer.com/gender-diversity-and-equality-in-the-tech-industry/
https://insights.blackcoffer.com/how-to-overcome-your-fear-of-making-mistakes/
https://insights.blackcoffer.com/how-small-business-can-survive-the-coronavirus-crisis/
https://insights.blackcoffer.com/impacts-of-covid-19-on-vegetable-vendors-and-food-stalls/
https://insights.blackcoffer.com/impacts-of-covid-19-on-vegetable-vendors/
https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-tourism-aviation-industries/
https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-sports-events-around-the-world/
https://insights.blackcoffer.com/changing-landscape-and-emerging-trends-in-the-indian-it-ites-industry/
https://insights.blackcoffer.com/online-gaming-adolescent-online-gaming-effects-demotivated-depression-musculoskeletal-and-psychosomatic-symptoms/
https://insights.blackcoffer.com/human-rights-outlook/
https://insights.blackcoffer.com/how-voice-search-makes-your-business-a-successful-business/
https://insights.blackcoffer.com/how-the-covid-19-crisis-is-redefining-jobs-and-services/
https://insights.blackcoffer.com/how-to-increase-social-media-engagement-for-marketers/
https://insights.blackcoffer.com/impacts-of-covid-19-on-streets-sides-food-stalls/
https://insights.blackcoffer.com/coronavirus-impact-on-energy-markets-2/
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-5/
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-4/
https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work-2/
https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work-3/
https://insights.blackcoffer.com/travel-and-tourism-outlook/
https://insights.blackcoffer.com/gaming-disorder-and-effects-of-gaming-on-health/
https://insights.blackcoffer.com/what-is-the-repercussion-of-the-environment-due-to-the-covid-19-pandemic-situation/
https://insights.blackcoffer.com/what-is-the-repercussion-of-the-environment-due-to-the-covid-19-pandemic-situation-2/
https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-office-space-and-co-working-industries/
https://insights.blackcoffer.com/contribution-of-handicrafts-visual-arts-literature-in-the-indian-economy/
https://insights.blackcoffer.com/how-covid-19-is-impacting-payment-preferences/
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis/
https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding/
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-2/
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-3/
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-3/
https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work/
https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding-2/
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-4/
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-2/
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-3/
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-4/
https://insights.blackcoffer.com/why-scams-like-nirav-modi-happen-with-indian-banks/
https://insights.blackcoffer.com/impact-of-covid-19-on-the-global-economy/
https://insights.blackcoffer.com/impact-of-covid-19coronavirus-on-the-indian-economy-2/
https://insights.blackcoffer.com/impact-of-covid-19-on-the-global-economy-2/
https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy-3/
https://insights.blackcoffer.com/should-celebrities-be-allowed-to-join-politics/
https://insights.blackcoffer.com/how-prepared-is-india-to-tackle-a-possible-covid-19-outbreak/
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work/
https://insights.blackcoffer.com/controversy-as-a-marketing-strategy/
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry/
https://insights.blackcoffer.com/coronavirus-impact-on-energy-markets/
https://insights.blackcoffer.com/what-are-the-key-policies-that-will-mitigate-the-impacts-of-covid-19-on-the-world-of-work/
https://insights.blackcoffer.com/marketing-drives-results-with-a-focus-on-problems/
https://insights.blackcoffer.com/continued-demand-for-sustainability/
https://insights.blackcoffer.com/coronavirus-disease-covid-19-effect-the-impact-and-role-of-mass-media-during-the-pandemic/
https://insights.blackcoffer.com/should-people-wear-fabric-gloves-seeking-evidence-regarding-the-differential-transfer-of-covid-19-or-coronaviruses-generally-between-surfaces/
https://insights.blackcoffer.com/why-is-there-a-severe-immunological-and-inflammatory-explosion-in-those-affected-by-sarms-covid-19/
https://insights.blackcoffer.com/what-do-you-think-is-the-lesson-or-lessons-to-be-learned-with-covid-19/
https://insights.blackcoffer.com/coronavirus-the-unexpected-challenge-for-the-european-union/
https://insights.blackcoffer.com/industrial-revolution-4-0-pros-and-cons/
https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy/
https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy-2/
https://insights.blackcoffer.com/impact-of-covid-19coronavirus-on-the-indian-economy/
https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-global-economy/
https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
https://insights.blackcoffer.com/blockchain-in-fintech/
https://insights.blackcoffer.com/blockchain-for-payments/
https://insights.blackcoffer.com/the-future-of-investing/
https://insights.blackcoffer.com/big-data-analytics-in-healthcare/
https://insights.blackcoffer.com/business-analytics-in-the-healthcare-industry/
https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/'''
url_id = url_id.replace('\n',',')
url_id = url_id.split(',')
url_id

['https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/',
 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/',
 'https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/',
 'https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/',
 'https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/',
 'https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/',
 'https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/',
 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/',
 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/',
 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/',
 'https://insights.blackcoffer.com/how-to-protect-fu

In [22]:
for i in url_id:
    response = requests.get(i)
    soup = BeautifulSoup(response.content, 'html.parser')
    title_tag = soup.find('h1', class_='tdb-title-text')
    title = title_tag.text.strip() if title_tag else ''
    article_tag = soup.find_all('p', class_="")
    text = ' '.join([p.text.strip() for p in soup])
    text = re.sub('<[^<]+?>', '', text)
    simpleStr = ""
    for p in article_tag:
        import re
        clean = re.compile('<.*?>')
        raw= re.sub(clean, '', str(p))
        simpleStr += raw + " "
        
    import nltk
    sentences = nltk.sent_tokenize(simpleStr)
    
    total_number_of_sentences = len(sentences)
    print('total number of sentence in url',total_number_of_sentences)
    
    corpus=[]
    for i in range(len(sentences)):
        rp = re.sub('[^a-zA-Z]',' ', sentences[i])
        rp = rp.lower()
        rp = rp.split()
        rp = [ word for word in rp if not word in set(stopwords)]
        rp = ' '.join(rp)
        corpus.append(rp)
        
    total_number_of_words = 0
    for text in corpus:
        words = text.split()
        total_number_of_words += len(words)
    print('Total number of words: ',total_number_of_words)
    
    positive_score=0
    n_score=0
    for i in words:
        if i in set(positive_words):
            positive_score = positive_score + 1
        elif i in set(negative_words):
            n_score = negative_score - 1
    negative_score = n_score * (-1)
    print('positive score: ',positive_score)
    print('negative score: ',negative_score)
    
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    print('Polarity score: ',polarity_score)
    
    subjectivity_score = (positive_score + negative_score) / (total_number_of_words + 0.000001)
    print('Subjectivity score: ',subjectivity_score)
    
    try:
        average_sentence_length = total_number_of_words / total_number_of_sentences
    except ZeroDivisionError:
        average_sentence_length = 0
    print('Average Sentence Length: ',average_sentence_length)
    
    def count_syllables(word):
        vowels = "aeiouy"
        count = 0
        prev_char = ''
        for char in word:
            if char.lower() in vowels and prev_char not in vowels:
                count += 1
            prev_char = char
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count = 1
        return count
    words = simpleStr.split()
    complex_count = 0
    for word in words:
        if count_syllables(word) > 2:
            complex_count += 1
    print('The number of complex words in the text: ',complex_count)
    total_count = len(words)
    
    try:
        complex_percentage = complex_count / total_count * 100
    except ZeroDivisionError:
        complex_percentage = 0
    print("The percentage of complex words in the text is:", complex_percentage, "%")
    
    fog_index = 0.4 * ( average_sentence_length + complex_percentage)
    print('Fog Index = ',  fog_index)
    
    try:
        average_number_of_words_per_sentence = total_number_of_words / total_number_of_sentences
    except ZeroDivisionError:
        average_number_of_words_per_sentence = 0
    print('Average Number of Words Per Sentence = ',  average_number_of_words_per_sentence )
    
    
    
    import re
    pattern = r"\b(I|we|my|ours|us)\b"
    total_number_of_personal_pronouns = len(re.findall(pattern, simpleStr, flags=re.IGNORECASE))
    print("The number of personal pronouns in the text is:", total_number_of_personal_pronouns)
    print(' '*125)
    print('*' * 125)
    print(' '*125)


total number of sentence in url 75
Total number of words:  1029
positive score:  3
negative score:  0
Polarity score:  0.9999996666667778
Subjectivity score:  0.0029154518922104447
Average Sentence Length:  13.72
The number of complex words in the text:  376
The percentage of complex words in the text is: 21.206993795826286 %
Fog Index =  13.970797518330514
Average Number of Words Per Sentence =  13.72
The number of personal pronouns in the text is: 1
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 79
Total number of words:  608
positive score:  0
negative score:  0
Polarity score:  0.0
Subjectivity score:  0.0
Average Sentence Le

total number of sentence in url 90
Total number of words:  927
positive score:  0
negative score:  0
Polarity score:  0.0
Subjectivity score:  0.0
Average Sentence Length:  10.3
The number of complex words in the text:  299
The percentage of complex words in the text is: 16.294277929155314 %
Fog Index =  10.637711171662126
Average Number of Words Per Sentence =  10.3
The number of personal pronouns in the text is: 1
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 57
Total number of words:  598
positive score:  2
negative score:  0
Polarity score:  0.99999950000025
Subjectivity score:  0.003344481599758392
Average Sentence Length: 

total number of sentence in url 0
Total number of words:  0
positive score:  2
negative score:  0
Polarity score:  0.99999950000025
Subjectivity score:  2000000.0
Average Sentence Length:  0
The number of complex words in the text:  0
The percentage of complex words in the text is: 0 %
Fog Index =  0.0
Average Number of Words Per Sentence =  0
The number of personal pronouns in the text is: 0
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 55
Total number of words:  670
positive score:  0
negative score:  1
Polarity score:  -0.9999990000010001
Subjectivity score:  0.0014925373112051681
Average Sentence Length:  12.181818181818182


total number of sentence in url 55
Total number of words:  341
positive score:  2
negative score:  0
Polarity score:  0.99999950000025
Subjectivity score:  0.005865102622096473
Average Sentence Length:  6.2
The number of complex words in the text:  95
The percentage of complex words in the text is: 11.889862327909889 %
Fog Index =  7.235944931163956
Average Number of Words Per Sentence =  6.2
The number of personal pronouns in the text is: 19
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 9
Total number of words:  75
positive score:  2
negative score:  1
Polarity score:  0.33333322222225925
Subjectivity score:  0.0399999994666666

total number of sentence in url 16
Total number of words:  170
positive score:  1
negative score:  1
Polarity score:  0.0
Subjectivity score:  0.01176470581314879
Average Sentence Length:  10.625
The number of complex words in the text:  59
The percentage of complex words in the text is: 12.79826464208243 %
Fog Index =  9.369305856832971
Average Number of Words Per Sentence =  10.625
The number of personal pronouns in the text is: 8
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 69
Total number of words:  803
positive score:  2
negative score:  0
Polarity score:  0.99999950000025
Subjectivity score:  0.0024906600218049064
Average

...

total number of sentence in url 27
Total number of words:  243
positive score:  0
negative score:  1
Polarity score:  -0.9999990000010001
Subjectivity score:  0.004115226320513472
Average Sentence Length:  9.0
The number of complex words in the text:  58
The percentage of complex words in the text is: 10.841121495327103 %
Fog Index =  7.936448598130841
Average Number of Words Per Sentence =  9.0
The number of personal pronouns in the text is: 4
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 30
Total number of words:  440
positive score:  0
negative score:  0
Polarity score:  0.0
Subjectivity score:  0.0
Average Sentence Length:  

total number of sentence in url 66
Total number of words:  888
positive score:  0
negative score:  0
Polarity score:  0.0
Subjectivity score:  0.0
Average Sentence Length:  13.454545454545455
The number of complex words in the text:  279
The percentage of complex words in the text is: 16.127167630057805 %
Fog Index =  11.832685233841303
Average Number of Words Per Sentence =  13.454545454545455
The number of personal pronouns in the text is: 13
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 77
Total number of words:  495
positive score:  0
negative score:  0
Polarity score:  0.0
Subjectivity score:  0.0
Average Sentence Length:  

total number of sentence in url 54
Total number of words:  643
positive score:  2
negative score:  0
Polarity score:  0.99999950000025
Subjectivity score:  0.003110419901850047
Average Sentence Length:  11.907407407407407
The number of complex words in the text:  214
The percentage of complex words in the text is: 16.550657385924207 %
Fog Index =  11.383225917332647
Average Number of Words Per Sentence =  11.907407407407407
The number of personal pronouns in the text is: 7
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 15
Total number of words:  125
positive score:  1
negative score:  1
Polarity score:  0.0
Subjectivity score:  0

total number of sentence in url 7
Total number of words:  100
positive score:  0
negative score:  0
Polarity score:  0.0
Subjectivity score:  0.0
Average Sentence Length:  14.285714285714286
The number of complex words in the text:  35
The percentage of complex words in the text is: 15.765765765765765 %
Fog Index =  12.020592020592021
Average Number of Words Per Sentence =  14.285714285714286
The number of personal pronouns in the text is: 4
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 14
Total number of words:  109
positive score:  0
negative score:  1
Polarity score:  -0.9999990000010001
Subjectivity score:  0.009174311842437

total number of sentence in url 49
Total number of words:  565
positive score:  1
negative score:  0
Polarity score:  0.9999990000010001
Subjectivity score:  0.001769911501292192
Average Sentence Length:  11.53061224489796
The number of complex words in the text:  171
The percentage of complex words in the text is: 15.659340659340659 %
Fog Index =  10.875981161695448
Average Number of Words Per Sentence =  11.53061224489796
The number of personal pronouns in the text is: 2
                                                                                                                             
*****************************************************************************************************************************
                                                                                                                             
total number of sentence in url 65
Total number of words:  577
positive score:  2
negative score:  1
Polarity score:  0.33333322222225925
Subjec